# <font color='blue'>Modelo de Transferência do Conhecimento</font>
## <font color='blue'>Conversão de Arquivos do Lattes para Carga no Qlik Sense</font>

## <font color='blue'>Processo ETL: Parsing XML to CSV</font>

https://pandas.pydata.org/

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.6.1


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal:
!pip install pandas==1.4.2

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

  Could not fetch URL https://pypi.python.org/simple/pandas/: There was a problem confirming the ssl certificate: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:749) - skipping


  Could not find a version that satisfies the requirement pandas==1.4.2 (from versions: )
No matching distribution found for pandas==1.4.2
  Could not find a version that satisfies the requirement watermark (from versions: )
No matching distribution found for watermark


In [3]:
# Importando o módulo Pandas
import pandas as pd
import xml.etree.ElementTree as ET  # o conteúdo está sob a forma de dicionário
import re
import os
import csv
from random import randrange, uniform
import uuid

In [4]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Beatriz H. Neto" --iversions

ModuleNotFoundError: No module named 'watermark'

# I. Dados Gerais

In [ ]:
# Coleta os dados do element 'DADOS-GERAIS' para o dicionário dados_gerais.
def extrai_dados_gerais(curriculo_vitae):
    id_lattes = curriculo_vitae
    dados_gerais = {} 
    dados_gerais.update(id_lattes)                              

    for filho in root.iter('DADOS-GERAIS'):
        dados_gerais.update(filho.attrib)
        if 'NOME-EM-CITACOES-BIBLIOGRAFICAS' in dados_gerais:
            del dados_gerais['NOME-EM-CITACOES-BIBLIOGRAFICAS']
        if 'PERMISSAO-DE-DIVULGACAO' in dados_gerais:
            del dados_gerais['PERMISSAO-DE-DIVULGACAO']
        if 'OUTRAS-INFORMACOES-RELEVANTES' in dados_gerais:
            del dados_gerais['OUTRAS-INFORMACOES-RELEVANTES']
        if 'TEXTO-RESUMO-CV-RH' in dados_gerais:
            del dados_gerais['TEXTO-RESUMO-CV-RH']
        if 'NOME-COMPLETO-CIVIL' in dados_gerais:
            dados_gerais['NOME-COMPLETO'] = dados_gerais['NOME-COMPLETO-CIVIL']
            del dados_gerais['NOME-COMPLETO-CIVIL']
        return(dados_gerais)

In [ ]:
# Cria novas chaves e apaga as chaves antigas no dicionário dados_gerais
def atualiza_chaves_dados_gerais(dados_gerais_sem_atualizar):
    dados_gerais = {}
    dados_gerais = dados_gerais_sem_atualizar
    dados_gerais['ID_LATTES'] = dados_gerais.pop('ID_LATTES')
    dados_gerais['NOME_COMPLETO'] = dados_gerais.pop('NOME-COMPLETO')
    dados_gerais['NACIONALIDADE'] = dados_gerais.pop('NACIONALIDADE')
    dados_gerais['PAIS_NASCIMENTO'] = dados_gerais.pop('PAIS-DE-NASCIMENTO')
    dados_gerais['UF_NASCIMENTO'] = dados_gerais.pop('UF-NASCIMENTO')
    dados_gerais['CIDADE_NASCIMENTO'] = dados_gerais.pop('CIDADE-NASCIMENTO')
    dados_gerais['DATA_FALECIMENTO'] = dados_gerais.pop('DATA-FALECIMENTO')
    dados_gerais['SIGLA_PAIS_NACIONALIDADE'] = dados_gerais.pop('SIGLA-PAIS-NACIONALIDADE')
    dados_gerais['PAIS_NACIONALIDADE'] = dados_gerais.pop('PAIS-DE-NACIONALIDADE')
    dados_gerais['ORCID_ID'] = dados_gerais.pop('ORCID-ID')
    return(dados_gerais)

# II. Formação Acadêmica

## 2.1. Ensino Médio

In [ ]:
# Coleta os dados do element 'ENSINO-MEDIO-SEGUNDO-GRAU' para o dicionário segundo_grau. 
def extrai_segundo_grau():
    segundo_grau = {}
    competencia_ini = ''
    competencia_ini_seg_grau = ''
    competencia_fim = ''
    competencia_fim_seg_grau = ''

    for filho in root.iter('ENSINO-MEDIO-SEGUNDO-GRAU'):
        #segundo_grau = filho.attrib
        segundo_grau.update(filho.attrib)
        #print(filho.attrib)
        
        # Formato Ano Mês: YYYYMM
        competencia_ini = segundo_grau['ANO-DE-INICIO'] + '01' 
        competencia_ini_seg_grau = {'COMP_INI': competencia_ini}
        segundo_grau.update(competencia_ini_seg_grau)
       
        competencia_fim = segundo_grau['ANO-DE-CONCLUSAO'] + '12'
        competencia_fim_seg_grau = {'COMP_FIM': competencia_fim}
        segundo_grau.update(competencia_fim_seg_grau)
        
    if len(segundo_grau) < 1:
        segundo_grau.update({'SEQUENCIA-FORMACAO': 'NI', 
                             'COMP_INI': 'NI', 
                             'COMP_FIM': 'NI', 
                             'NIVEL': 'NI', 
                             'CODIGO-INSTITUICAO': 'NI', 
                             'NOME-INSTITUICAO': 'NI', 
                             'STATUS-DO-CURSO': 'NI', 
                             'ANO-DE-INICIO': 'NI', 
                             'ANO-DE-CONCLUSAO': 'NI'})    
    return(segundo_grau)


In [ ]:
# Cria novas chaves e apaga as chaves antigas no dicionário segundo_grau.

def atualiza_chaves_segundo_grau(segundo_grau_sem_atualizar):
    segundo_grau = {}
    segundo_grau = segundo_grau_sem_atualizar
    segundo_grau['SEQUENCIA_SEG_GRAU'] = segundo_grau.pop('SEQUENCIA-FORMACAO')
    segundo_grau['COMP_INI_SEG_GRAU'] = segundo_grau.pop('COMP_INI')
    segundo_grau['COMP_FIM_SEG_GRAU'] = segundo_grau.pop('COMP_FIM')
    segundo_grau['NIVEL_SEG_GRAU'] = segundo_grau.pop('NIVEL')
    segundo_grau['COD_INSTITUICAO_SEG_GRAU'] = segundo_grau.pop('CODIGO-INSTITUICAO')
    segundo_grau['NOME_INSTITUICAO_SEG_GRAU'] = segundo_grau.pop('NOME-INSTITUICAO')
    segundo_grau['STATUS_SEG_GRAU'] = segundo_grau.pop('STATUS-DO-CURSO')
    segundo_grau['ANO_INICIO_SEG_GRAU'] = segundo_grau.pop('ANO-DE-INICIO')
    segundo_grau['ANO_CONCLUSAO_SEG_GRAU'] = segundo_grau.pop('ANO-DE-CONCLUSAO')
    return(segundo_grau)

## 2.2. Curso Técnico

In [ ]:
# Coleta os dados do element 'CURSO-TECNICO-PROFISSIONALIZANTE' para o dicionário curso_tecnico.

def extrai_curso_tecnico():
    curso_tecnico = {}
    competencia_ini = ''
    competencia_ini_curso_tec = ''
    competencia_fim = ''
    competencia_fim_curso_tec = ''

    for filho in root.iter('CURSO-TECNICO-PROFISSIONALIZANTE'):
        curso_tecnico = filho.attrib
        if 'CODIGO-ORGAO' in curso_tecnico:
            del curso_tecnico['CODIGO-ORGAO']
        if 'NOME-ORGAO' in curso_tecnico:
            del curso_tecnico['NOME-ORGAO']
        if 'FLAG-BOLSA' in curso_tecnico:
            del curso_tecnico['FLAG-BOLSA']
        if 'CODIGO-AGENCIA-FINANCIADORA' in curso_tecnico:
            del curso_tecnico['CODIGO-AGENCIA-FINANCIADORA']
        if 'NOME-AGENCIA' in curso_tecnico:
            del curso_tecnico['NOME-AGENCIA']
        if 'NOME-CURSO-INGLES' in curso_tecnico:
            del curso_tecnico['NOME-CURSO-INGLES']

        # Formato Ano Mês: YYYYMM
        competencia_ini = curso_tecnico['ANO-DE-INICIO'] + '01' 
        competencia_ini_curso_tec = {'COMP_INI': competencia_ini}
        curso_tecnico.update(competencia_ini_curso_tec)

        competencia_fim = curso_tecnico['ANO-DE-CONCLUSAO'] +  '12'
        competencia_fim_curso_tec = {'COMP_FIM': competencia_fim}
        curso_tecnico.update(competencia_fim_curso_tec)

    # Se não há curso técnico preenche as colunas com branco.
    if len(curso_tecnico) < 1:
        curso_tecnico.update({'SEQUENCIA-FORMACAO': 'NI', 
                              'COMP_INI': 'NI', 
                              'COMP_FIM': 'NI', 
                              'NIVEL': 'NI', 
                              'CODIGO-INSTITUICAO': 'NI', 
                              'NOME-INSTITUICAO': 'NI', 
                              'CODIGO-CURSO': 'NI', 
                              'NOME-CURSO': 'NI', 
                              'STATUS-DO-CURSO': 'NI', 
                              'ANO-DE-INICIO': 'NI', 
                              'ANO-DE-CONCLUSAO': 'NI'})
    return(curso_tecnico)

In [ ]:
# Cria novas chaves e apaga as chaves antigas no dicionário curso_tecnico.

def atualiza_chaves_curso_tecnico(curso_tecnico_sem_atualizar):
    curso_tecnico = {}
    curso_tecnico = curso_tecnico_sem_atualizar
    curso_tecnico['SEQUENCIA_TEC_PROF'] = curso_tecnico.pop('SEQUENCIA-FORMACAO')
    curso_tecnico['COMP_INI_TEC_PROF'] = curso_tecnico.pop('COMP_INI')
    curso_tecnico['COMP_FIM_TEC_PROF'] = curso_tecnico.pop('COMP_FIM')
    curso_tecnico['NIVEL_TEC_PROF'] = curso_tecnico.pop('NIVEL')
    curso_tecnico['COD_INSTITUICAO_TEC_PROF'] = curso_tecnico.pop('CODIGO-INSTITUICAO')
    curso_tecnico['NOME_INSTITUICAO_TEC_PROF'] = curso_tecnico.pop('NOME-INSTITUICAO')
    curso_tecnico['COD_CURSO_TEC_PROF'] = curso_tecnico.pop('CODIGO-CURSO')
    curso_tecnico['NOME_CURSO_TEC_PROF'] = curso_tecnico.pop('NOME-CURSO')
    curso_tecnico['STATUS_CURSO_TEC_PROF'] = curso_tecnico.pop('STATUS-DO-CURSO')
    curso_tecnico['ANO_INICIO_TEC_PROF'] = curso_tecnico.pop('ANO-DE-INICIO')
    curso_tecnico['ANO_CONCLUSAO_TEC_PROF'] = curso_tecnico.pop('ANO-DE-CONCLUSAO')
    return(curso_tecnico)

## 2.3. Graduação

In [ ]:
# Coleta os dados do element 'GRADUACAO' para o dicionário graduacao.

def extrai_graduacao():
    graduacao = {}
    competencia_ini = ''
    competencia_ini_grad = ''
    competencia_fim = ''
    competencia_fim_grad = ''

    for filho in root.iter('GRADUACAO'):
        graduacao.update(filho.attrib)

        # Formato Ano Mês: YYYYMM
        competencia_ini = graduacao['ANO-DE-INICIO'] + '01'
        competencia_ini_grad = {'COMP_INI': competencia_ini}
        graduacao.update(competencia_ini_grad)

        competencia_fim = graduacao['ANO-DE-CONCLUSAO'] + '12'
        competencia_fim_grad = {'COMP_FIM': competencia_fim}
        graduacao.update(competencia_fim_grad)

        if 'CODIGO-ORGAO' in graduacao:
            del graduacao['CODIGO-ORGAO']
        if 'NOME-ORGAO' in graduacao:
            del graduacao['NOME-ORGAO']
        if 'CODIGO-AGENCIA-FINANCIADORA' in graduacao:
            del graduacao['CODIGO-AGENCIA-FINANCIADORA'] 
        if 'NOME-AGENCIA' in graduacao:
            del graduacao['NOME-AGENCIA']  
        if 'NUMERO-ID-ORIENTADOR' in graduacao:
            del graduacao['NUMERO-ID-ORIENTADOR'] 
        if 'CODIGO-CURSO-CAPES' in graduacao:
            del graduacao['CODIGO-CURSO-CAPES'] 
        if 'CODIGO-AREA-CURSO' in graduacao:
            del graduacao['CODIGO-AREA-CURSO'] 
        if 'TITULO-DO-TRABALHO-DE-CONCLUSAO-DE-CURSO-INGLES' in graduacao:
            del graduacao['TITULO-DO-TRABALHO-DE-CONCLUSAO-DE-CURSO-INGLES']
        if 'NOME-CURSO-INGLES' in graduacao:
            del graduacao['NOME-CURSO-INGLES']
        if 'CODIGO-INSTITUICAO-OUTRA-GRAD' in graduacao:
            del graduacao['CODIGO-INSTITUICAO-OUTRA-GRAD']
        if 'NOME-INSTITUICAO-OUTRA-GRAD' in graduacao:
            del graduacao['NOME-INSTITUICAO-OUTRA-GRAD']
        if 'NOME-ORIENTADOR-GRAD' in graduacao:
            del graduacao['NOME-ORIENTADOR-GRAD']
        if 'FLAG-BOLSA' in graduacao:
            del graduacao['FLAG-BOLSA']
        if 'CODIGO-INSTITUICAO-GRAD' in graduacao:
            del graduacao['CODIGO-INSTITUICAO-GRAD']
        if 'NOME-INSTITUICAO-GRAD' in graduacao:
            del graduacao['NOME-INSTITUICAO-GRAD']
        if 'FORMACAO-ACADEMICA-TITULACAO' in graduacao:
            del graduacao['FORMACAO-ACADEMICA-TITULACAO'] 
        if 'TIPO-GRADUACAO' in graduacao:
            del graduacao['TIPO-GRADUACAO'] 
    if len(graduacao) < 1:
        graduacao.update({'SEQUENCIA-FORMACAO': 'NI', 
                          'COMP_INI': 'NI', 
                          'COMP_FIM': 'NI', 
                          'NIVEL': 'NI', 
                          'TITULO-DO-TRABALHO-DE-CONCLUSAO-DE-CURSO': 'NI', 
                          'NOME-DO-ORIENTADOR': 'NI', 
                          'CODIGO-INSTITUICAO': 'NI', 
                          'NOME-INSTITUICAO': 'NI', 
                          'CODIGO-CURSO': 'NI', 
                          'NOME-CURSO': 'NI', 
                          'STATUS-DO-CURSO': 'NI', 
                          'ANO-DE-INICIO': 'NI', 
                          'ANO-DE-CONCLUSAO': 'NI'})
    return(graduacao)

In [ ]:
# Cria novas chaves e apaga as chaves antigas no dicionário graduacao.

def atualiza_chaves_graduacao(graduacao_sem_atualizar):
    graduacao = {}
    graduacao = graduacao_sem_atualizar
    graduacao['SEQUENCIA_FORMACAO_GRAD'] = graduacao.pop('SEQUENCIA-FORMACAO')
    graduacao['COMP_INI_GRAD'] = graduacao.pop('COMP_INI')
    graduacao['COMP_FIM_GRAD'] = graduacao.pop('COMP_FIM')
    graduacao['NIVEL_GRAD'] = graduacao.pop('NIVEL')
    graduacao['TITULO_TRABALHO_GRAD'] = graduacao.pop('TITULO-DO-TRABALHO-DE-CONCLUSAO-DE-CURSO')
    graduacao['NOME_ORIENTADOR_GRAD'] = graduacao.pop('NOME-DO-ORIENTADOR')
    graduacao['COD_INSTITUICAO_GRAD'] = graduacao.pop('CODIGO-INSTITUICAO')
    graduacao['NOME_INSTITUICAO_GRAD'] = graduacao.pop('NOME-INSTITUICAO')
    graduacao['COD_CURSO_GRAD'] = graduacao.pop('CODIGO-CURSO')
    graduacao['NOME_CURSO_GRAD'] = graduacao.pop('NOME-CURSO')
    graduacao['STATUS_CURSO_GRAD'] = graduacao.pop('STATUS-DO-CURSO')
    graduacao['ANO_INICIO_GRAD'] = graduacao.pop('ANO-DE-INICIO')
    graduacao['ANO_CONCLUSAO_GRAD'] = graduacao.pop('ANO-DE-CONCLUSAO')
    return(graduacao)

## 2.4. Curso de Aperfeiçoamento

In [ ]:
# Coleta os dados do element 'APERFEICOAMENTO' para o dicionário aperfeiçoamento.

def extrai_aperfeiçoamento():
    aperfeiçoamento = {}
    competencia_ini = ''
    competencia_ini_aperf = ''
    competencia_fim = ''
    competencia_fim_aperf = ''

    for filho in root.iter('APERFEICOAMENTO'):
        aperfeiçoamento.update(filho.attrib)

        # Formato Ano Mês: YYYYMM
        competencia_ini = aperfeiçoamento['ANO-DE-INICIO'] + '01'
        competencia_ini_aperf = {'COMP_INI': competencia_ini}
        aperfeiçoamento.update(competencia_ini_aperf)

        competencia_fim = aperfeiçoamento['ANO-DE-CONCLUSAO'] + '12'
        competencia_fim_aperf = {'COMP_FIM': competencia_fim}
        aperfeiçoamento.update(competencia_fim_aperf)

        if 'CODIGO-ORGAO' in aperfeiçoamento:
            del aperfeiçoamento['CODIGO-ORGAO']
        if 'NOME-ORGAO' in aperfeiçoamento:
            del aperfeiçoamento['NOME-ORGAO']
        if 'CODIGO-AREA-CURSO' in aperfeiçoamento:
            del aperfeiçoamento['CODIGO-AREA-CURSO']
        if 'FLAG-BOLSA' in aperfeiçoamento:
            del aperfeiçoamento['FLAG-BOLSA']
        if 'CODIGO-AGENCIA-FINANCIADORA' in aperfeiçoamento:
            del aperfeiçoamento['CODIGO-AGENCIA-FINANCIADORA'] 
        if 'NOME-AGENCIA' in aperfeiçoamento:
            del aperfeiçoamento['NOME-AGENCIA']
        if 'CARGA-HORARIA' in aperfeiçoamento:
            del aperfeiçoamento['CARGA-HORARIA']
        if 'TITULO-DA-MONOGRAFIA-INGLES' in aperfeiçoamento:
            del aperfeiçoamento['TITULO-DA-MONOGRAFIA-INGLES']
        if 'NOME-CURSO-INGLES' in aperfeiçoamento:
            del aperfeiçoamento['NOME-CURSO-INGLES']
    if len(aperfeiçoamento) < 1:
        aperfeiçoamento.update({'SEQUENCIA-FORMACAO': 'NI', 
                                'COMP_INI': 'NI', 
                                'COMP_FIM': 'NI', 
                                'NIVEL': 'NI', 
                                'TITULO-DA-MONOGRAFIA': 'NI', 
                                'NOME-DO-ORIENTADOR': 'NI', 
                                'CODIGO-INSTITUICAO': 'NI', 
                                'NOME-INSTITUICAO': 'NI', 
                                'CODIGO-CURSO': 'NI', 
                                'NOME-CURSO': 'NI',
                                'STATUS-DO-CURSO': 'NI', 
                                'ANO-DE-INICIO': 'NI', 
                                'ANO-DE-CONCLUSAO': 'NI'})
    return(aperfeiçoamento)

In [ ]:
# Cria novas chaves e apaga as chaves antigas no dicionário aperfeiçoamento.

def atualiza_chaves_aperfeiçoamento(aperfeiçoamento_sem_atualizar):
    aperfeiçoamento = {}
    aperfeiçoamento = aperfeiçoamento_sem_atualizar
    aperfeiçoamento['SEQUENCIA_FORMACAO_APERF'] = aperfeiçoamento.pop('SEQUENCIA-FORMACAO')
    aperfeiçoamento['COMP_INI_APERF'] = aperfeiçoamento.pop('COMP_INI')
    aperfeiçoamento['COMP_FIM_APERF'] = aperfeiçoamento.pop('COMP_FIM')
    aperfeiçoamento['NIVEL_APERF'] = aperfeiçoamento.pop('NIVEL')
    aperfeiçoamento['TITULO_TRABALHO_APERF'] = aperfeiçoamento.pop('TITULO-DA-MONOGRAFIA')
    aperfeiçoamento['NOME_ORIENTADOR_APERF'] = aperfeiçoamento.pop('NOME-DO-ORIENTADOR')
    aperfeiçoamento['COD_INSTITUICAO_APERF'] = aperfeiçoamento.pop('CODIGO-INSTITUICAO')
    aperfeiçoamento['NOME_INSTITUICAO_APERF'] = aperfeiçoamento.pop('NOME-INSTITUICAO')
    aperfeiçoamento['COD_CURSO_APERF'] = aperfeiçoamento.pop('CODIGO-CURSO')
    aperfeiçoamento['NOME_CURSO_APERF'] = aperfeiçoamento.pop('NOME-CURSO')
    aperfeiçoamento['STATUS_CURSO_APERF'] = aperfeiçoamento.pop('STATUS-DO-CURSO')
    aperfeiçoamento['ANO_INICIO_APERF'] = aperfeiçoamento.pop('ANO-DE-INICIO')
    aperfeiçoamento['ANO_CONCLUSAO_APERF'] = aperfeiçoamento.pop('ANO-DE-CONCLUSAO')
    return(aperfeiçoamento)

## 2.5. Curso de Especialização

In [ ]:
# Coleta os dados do element 'ESPECIALIZACAO' para o dicionário aperfeiçoamento.

def extrai_especializacao():
    especializacao = {}
    competencia_ini = ''
    competencia_ini_espec = ''
    competencia_fim = ''
    competencia_fim_espec = ''

    for filho in root.iter('ESPECIALIZACAO'):
        especializacao.update(filho.attrib)

        # Formato Ano Mês: YYYYMM
        competencia_ini = especializacao['ANO-DE-INICIO'] + '01'
        competencia_ini_espec = {'COMP_INI': competencia_ini}
        especializacao.update(competencia_ini_espec)

        competencia_fim = especializacao['ANO-DE-CONCLUSAO'] + '12'
        competencia_fim_espec = {'COMP_FIM': competencia_fim}
        especializacao.update(competencia_fim_espec)
        if 'CODIGO-ORGAO' in especializacao:
            del especializacao['CODIGO-ORGAO']
        if 'NOME-ORGAO' in especializacao:
            del especializacao['NOME-ORGAO']    
        if 'FLAG-BOLSA' in especializacao:
            del especializacao['FLAG-BOLSA']
        if 'CODIGO-AGENCIA-FINANCIADORA' in especializacao:
            del especializacao['CODIGO-AGENCIA-FINANCIADORA']
        if 'NOME-AGENCIA' in especializacao:
            del especializacao['NOME-AGENCIA']
        if 'CARGA-HORARIA' in especializacao:
            del especializacao['CARGA-HORARIA'] 
        if 'TITULO-DA-MONOGRAFIA-INGLES' in especializacao:
            del especializacao['TITULO-DA-MONOGRAFIA-INGLES']
        if 'NOME-CURSO-INGLES' in especializacao:
            del especializacao['NOME-CURSO-INGLES']
    if len(especializacao) < 1:
        especializacao.update({'SEQUENCIA-FORMACAO': 'NI', 
                               'COMP_INI': 'NI', 
                               'COMP_FIM': 'NI', 
                               'NIVEL': 'NI', 
                               'TITULO-DA-MONOGRAFIA': 'NI', 
                               'NOME-DO-ORIENTADOR': 'NI', 
                               'CODIGO-INSTITUICAO': 'NI', 
                               'NOME-INSTITUICAO': 'NI', 
                               'CODIGO-CURSO': 'NI', 
                               'NOME-CURSO': 'NI', 
                               'STATUS-DO-CURSO': 'NI',
                               'ANO-DE-INICIO': 'NI', 
                               'ANO-DE-CONCLUSAO': 'NI'})
    return(especializacao)

In [ ]:
# Cria novas chaves e apaga as chaves antigas no dicionário especializacao.

def atualiza_chaves_especializacao(especializacao_sem_atualizar):
    especializacao = {}
    especializacao = especializacao_sem_atualizar
    especializacao['SEQUENCIA_FORMACAO_ESPEC'] = especializacao.pop('SEQUENCIA-FORMACAO')
    especializacao['COMP_INI_ESPEC'] = especializacao.pop('COMP_INI')
    especializacao['COMP_FIM_ESPEC'] = especializacao.pop('COMP_FIM')
    especializacao['NIVEL_ESPEC'] = especializacao.pop('NIVEL')
    especializacao['TITULO_TRABALHO_ESPEC'] = especializacao.pop('TITULO-DA-MONOGRAFIA')
    especializacao['NOME_ORIENTADOR_ESPEC'] = especializacao.pop('NOME-DO-ORIENTADOR')
    especializacao['COD_INSTITUICAO_ESPEC'] = especializacao.pop('CODIGO-INSTITUICAO')
    especializacao['NOME_INSTITUICAO_ESPEC'] = especializacao.pop('NOME-INSTITUICAO')
    especializacao['COD_CURSO_ESPEC'] = especializacao.pop('CODIGO-CURSO')
    especializacao['NOME_CURSO_ESPEC'] = especializacao.pop('NOME-CURSO')
    especializacao['STATUS_CURSO_ESPEC'] = especializacao.pop('STATUS-DO-CURSO')
    especializacao['ANO_INICIO_ESPEC'] = especializacao.pop('ANO-DE-INICIO')
    especializacao['ANO_CONCLUSAO_ESPEC'] = especializacao.pop('ANO-DE-CONCLUSAO')
    return(especializacao)

## 2.6. Mestrado

In [ ]:
# Coleta os dados do element 'MESTRADO' para o dicionário mestrado.

def extrai_mestrado():
    mestrado = {}
    competencia_ini = ''
    competencia_ini_mestr = ''
    competencia_fim = ''
    competencia_fim_mestr = ''

    for filho in root.iter('MESTRADO'):
        mestrado.update(filho.attrib)

        # Formato Ano Mês: YYYYMM
        competencia_ini = mestrado['ANO-DE-INICIO'] + '01'
        competencia_ini_mestr = {'COMP_INI': competencia_ini}
        mestrado.update(competencia_ini_mestr)

        competencia_fim = mestrado['ANO-DE-CONCLUSAO'] + '12'
        competencia_fim_mestr = {'COMP_FIM': competencia_fim}
        mestrado.update(competencia_fim_mestr)

        if 'CODIGO-ORGAO' in mestrado:
            del mestrado['CODIGO-ORGAO']
        if 'NOME-ORGAO' in mestrado:
            del mestrado['NOME-ORGAO'] 
        if 'CODIGO-AREA-CURSO' in mestrado:
            del mestrado['CODIGO-AREA-CURSO']
        if 'FLAG-BOLSA' in mestrado:
            del mestrado['FLAG-BOLSA'] 
        if 'CODIGO-AGENCIA-FINANCIADORA' in mestrado:
            del mestrado['CODIGO-AGENCIA-FINANCIADORA']
        if 'NOME-AGENCIA' in mestrado:
            del mestrado['NOME-AGENCIA']
        if 'CODIGO-CURSO-CAPES' in mestrado:
            del mestrado['CODIGO-CURSO-CAPES']
        if 'TITULO-DA-DISSERTACAO-TESE-INGLES' in mestrado:
            del mestrado['TITULO-DA-DISSERTACAO-TESE-INGLES']
        if 'NOME-CURSO-INGLES' in mestrado:
            del mestrado['NOME-CURSO-INGLES']
        if 'NOME-DO-CO-ORIENTADOR' in mestrado:
            del mestrado['NOME-DO-CO-ORIENTADOR'] 
        if 'CODIGO-INSTITUICAO-DOUT' in mestrado:
            del mestrado['CODIGO-INSTITUICAO-DOUT'] 
        if 'NOME-INSTITUICAO-DOUT' in mestrado:
            del mestrado['NOME-INSTITUICAO-DOUT']
        if 'CODIGO-INSTITUICAO-OUTRA-DOUT' in mestrado:
            del mestrado['CODIGO-INSTITUICAO-OUTRA-DOUT'] 
        if 'NOME-INSTITUICAO-OUTRA-DOUT' in mestrado:
            del mestrado['NOME-INSTITUICAO-OUTRA-DOUT']
        if 'NOME-ORIENTADOR-DOUT' in mestrado:
            del mestrado['NOME-ORIENTADOR-DOUT']   
    if len(mestrado) < 1:
        mestrado.update({'SEQUENCIA-FORMACAO': 'NI', 
                         'COMP_INI': 'NI', 
                         'COMP_FIM': 'NI', 
                         'NIVEL': 'NI', 
                         'CODIGO-INSTITUICAO': 'NI', 
                         'NOME-INSTITUICAO': 'NI', 
                         'CODIGO-CURSO': 'NI', 
                         'NOME-CURSO': 'NI', 
                         'STATUS-DO-CURSO': 'NI', 
                         'ANO-DE-INICIO': 'NI', 
                         'ANO-DE-CONCLUSAO': 'NI', 
                         'ANO-DE-OBTENCAO-DO-TITULO': 'NI', 
                         'TITULO-DA-DISSERTACAO-TESE': 'NI', 
                         'NOME-COMPLETO-DO-ORIENTADOR': 'NI', 
                         'TIPO-MESTRADO': 'NI', 
                         'NUMERO-ID-ORIENTADOR': 'NI'})
    return(mestrado)

In [ ]:
# Cria novas chaves e apaga as chaves antigas no dicionário mestrado.

def atualiza_chaves_mestrado(mestrado_sem_atualizar):
    mestrado = {}
    mestrado = mestrado_sem_atualizar
    mestrado['SEQUENCIA_FORMACAO_MEST'] = mestrado.pop('SEQUENCIA-FORMACAO')
    mestrado['COMP_INI_MEST'] = mestrado.pop('COMP_INI')
    mestrado['COMP_FIM_MEST'] = mestrado.pop('COMP_FIM')
    mestrado['NIVEL_MEST'] = mestrado.pop('NIVEL')
    mestrado['COD_INSTITUICAO_MEST'] = mestrado.pop('CODIGO-INSTITUICAO')
    mestrado['NOME_INSTITUICAO_MEST'] = mestrado.pop('NOME-INSTITUICAO')
    mestrado['COD_CURSO_MEST'] = mestrado.pop('CODIGO-CURSO')
    mestrado['NOME_CURSO_MEST'] = mestrado.pop('NOME-CURSO')
    mestrado['STATUS_CURSO_MEST'] = mestrado.pop('STATUS-DO-CURSO')
    mestrado['ANO_INICIO_MEST'] = mestrado.pop('ANO-DE-INICIO')
    mestrado['ANO_CONCLUSAO_MEST'] = mestrado.pop('ANO-DE-CONCLUSAO')
    mestrado['ANO_OBTENCAO_TITULO_MEST'] = mestrado.pop('ANO-DE-OBTENCAO-DO-TITULO')
    mestrado['TITULO_DISSERTACAO_MEST'] = mestrado.pop('TITULO-DA-DISSERTACAO-TESE')
    mestrado['NOME_ORIENTADOR_MEST'] = mestrado.pop('NOME-COMPLETO-DO-ORIENTADOR')
    mestrado['TIPO_MEST'] = mestrado.pop('TIPO-MESTRADO')
    mestrado['ID_ORIENTADOR-MEST'] = mestrado.pop('NUMERO-ID-ORIENTADOR')
    return(mestrado)

## 2.7. Mestrado Profissional

In [ ]:
# Coleta os dados do element 'MESTRADO-PROFISSIONALIZANTE' para o dicionário mestrado_prof.

def extrai_mestrado_prof():
    mestrado_prof = {}
    competencia_ini = ''
    competencia_ini_mestr_prof = ''
    competencia_fim = ''
    competencia_fim_mestr_prof = ''

    for filho in root.iter('MESTRADO-PROFISSIONALIZANTE'):
        mestrado_prof.update(filho.attrib)

        # Formato Ano Mês: YYYYMM
        competencia_ini = mestrado_prof['ANO-DE-INICIO'] + '01'
        competencia_ini_mestr_prof = {'COMP_INI': competencia_ini}
        mestrado_prof.update(competencia_ini_mestr_prof)

        competencia_fim = mestrado_prof['ANO-DE-CONCLUSAO'] + '12'
        competencia_fim_mestr_prof = {'COMP_FIM': competencia_fim}
        mestrado_prof.update(competencia_fim_mestr_prof)

        if 'CODIGO-ORGAO' in mestrado_prof:
            del mestrado_prof['CODIGO-ORGAO']
        if 'NOME-ORGAO' in mestrado_prof:
            del mestrado_prof['NOME-ORGAO']
        if 'CODIGO-AREA-CURSO' in mestrado_prof:
            del mestrado_prof['CODIGO-AREA-CURSO']
        if 'FLAG-BOLSA' in mestrado_prof:
            del mestrado_prof['FLAG-BOLSA']
        if 'CODIGO-AGENCIA-FINANCIADORA' in mestrado_prof:
            del mestrado_prof['CODIGO-AGENCIA-FINANCIADORA']
        if 'NOME-AGENCIA' in mestrado_prof:
            del mestrado_prof['NOME-AGENCIA']
        if 'CODIGO-CURSO-CAPES' in mestrado_prof:
            del mestrado_prof['CODIGO-CURSO-CAPES'] 
        if 'TITULO-DA-DISSERTACAO-TESE-INGLES' in mestrado_prof:
            del mestrado_prof['TITULO-DA-DISSERTACAO-TESE-INGLES']
        if 'NOME-CURSO-INGLES' in mestrado_prof:
            del mestrado_prof['NOME-CURSO-INGLES']
        if 'NOME-DO-CO-ORIENTADOR' in mestrado_prof:
            del mestrado_prof['NOME-DO-CO-ORIENTADOR']   
    if len(mestrado_prof) < 1:
        mestrado_prof.update({'SEQUENCIA-FORMACAO': 'NI', 
                              'COMP_INI': 'NI', 
                              'COMP_FIM': 'NI', 
                              'NIVEL': 'NI', 
                              'CODIGO-INSTITUICAO': 'NI', 
                              'NOME-INSTITUICAO': 'NI', 
                              'CODIGO-CURSO': 'NI', 
                              'NOME-CURSO': 'NI', 
                              'STATUS-DO-CURSO': 'NI', 
                              'ANO-DE-INICIO': 'NI', 
                              'ANO-DE-CONCLUSAO': 'NI', 
                              'ANO-DE-OBTENCAO-DO-TITULO': 'NI', 
                              'TITULO-DA-DISSERTACAO-TESE': 'NI', 
                              'NOME-COMPLETO-DO-ORIENTADOR': 'NI', 
                              'TIPO-MESTRADO': 'NI', 
                              'NUMERO-ID-ORIENTADOR': 'NI'})
    return(mestrado_prof)

In [ ]:
# Cria novas chaves e apaga as chaves antigas no dicionário mestrado_prof.

def atualiza_chaves_mestrado_prof(mestrado_prof_sem_atualizar):
    mestrado_prof = {}
    mestrado_prof = mestrado_prof_sem_atualizar
    mestrado_prof['SEQUENCIA_FORMACAO_MEST_PROF'] = mestrado_prof.pop('SEQUENCIA-FORMACAO')
    mestrado_prof['COMP_INI_MEST_PROF'] = mestrado_prof.pop('COMP_INI')
    mestrado_prof['COMP_FIM_MEST_PROF'] = mestrado_prof.pop('COMP_FIM')
    mestrado_prof['NIVEL_MEST_PROF'] = mestrado_prof.pop('NIVEL')
    mestrado_prof['COD_INST_MEST_PROF'] = mestrado_prof.pop('CODIGO-INSTITUICAO')
    mestrado_prof['NOME_INST_MEST_PROF'] = mestrado_prof.pop('NOME-INSTITUICAO')
    mestrado_prof['COD_CURSO_MEST_PROF'] = mestrado_prof.pop('CODIGO-CURSO')
    mestrado_prof['NOME_CURSO_MEST_PROF'] = mestrado_prof.pop('NOME-CURSO')
    mestrado_prof['STATUS_CURSO_MEST_PROF'] = mestrado_prof.pop('STATUS-DO-CURSO')
    mestrado_prof['ANO_INICIO_MEST_PROF'] = mestrado_prof.pop('ANO-DE-INICIO')
    mestrado_prof['ANO_CONCLUSAO_MEST_PROF'] = mestrado_prof.pop('ANO-DE-CONCLUSAO')
    mestrado_prof['ANO_OBTENCAO_TITULO_MEST_PROF'] = mestrado_prof.pop('ANO-DE-OBTENCAO-DO-TITULO')
    mestrado_prof['TITULO_DISSERTACAO_MEST_PROF'] = mestrado_prof.pop('TITULO-DA-DISSERTACAO-TESE')
    mestrado_prof['NOME_ORIENTADOR_MEST_PROF'] = mestrado_prof.pop('NOME-COMPLETO-DO-ORIENTADOR')
    mestrado_prof['TIPO_MEST_PROF'] = mestrado_prof.pop('TIPO-MESTRADO')
    mestrado_prof['ID_ORIENTADOR_MEST_PROF'] = mestrado_prof.pop('NUMERO-ID-ORIENTADOR')
    return(mestrado_prof)

## 2.8. Doutorado

In [ ]:
# Coleta os dados do element 'DOUTORADO' para o dicionário doutorado.

def extrai_doutorado():
    doutorado = {}
    competencia_ini = ''
    competencia_ini_dout = ''
    competencia_fim = ''
    competencia_fim_dout = ''

    for filho in root.iter('DOUTORADO'):
        doutorado.update(filho.attrib)

        # Formato Ano Mês: YYYYMM
        competencia_ini = doutorado['ANO-DE-INICIO'] + '01'
        competencia_ini_dout = {'COMP_INI': competencia_ini}
        doutorado.update(competencia_ini_dout)

        competencia_fim = doutorado['ANO-DE-CONCLUSAO'] + '12'
        competencia_fim_dout = {'COMP_FIM': competencia_fim}
        doutorado.update(competencia_fim_dout)

        if 'CODIGO-ORGAO' in doutorado:
            del doutorado['CODIGO-ORGAO']
        if 'NOME-ORGAO' in doutorado:
            del doutorado['NOME-ORGAO']
        if 'CODIGO-AREA-CURSO' in doutorado:
            del doutorado['CODIGO-AREA-CURSO'] 
        if 'FLAG-BOLSA' in doutorado:
            del doutorado['FLAG-BOLSA']
        if 'CODIGO-AGENCIA-FINANCIADORA' in doutorado:
            del doutorado['CODIGO-AGENCIA-FINANCIADORA']
        if 'NOME-AGENCIA' in doutorado:
            del doutorado['NOME-AGENCIA']
        if 'CODIGO-INSTITUICAO-OUTRA-DOUT' in doutorado:
            del doutorado['CODIGO-INSTITUICAO-OUTRA-DOUT']
        if 'NOME-INSTITUICAO-OUTRA-DOUT' in doutorado:
            del doutorado['NOME-INSTITUICAO-OUTRA-DOUT']
        if 'CODIGO-CURSO-CAPES' in doutorado:
            del doutorado['CODIGO-CURSO-CAPES']
        if 'TITULO-DA-DISSERTACAO-TESE-INGLES' in doutorado:
            del doutorado['TITULO-DA-DISSERTACAO-TESE-INGLES'] 
        if 'NOME-CURSO-INGLES' in doutorado:
            del doutorado['NOME-CURSO-INGLES'] 
        if 'NOME-DO-ORIENTADOR-CO-TUTELA' in doutorado:
            del doutorado['NOME-DO-ORIENTADOR-CO-TUTELA'] 
        if 'CODIGO-INSTITUICAO-OUTRA-CO-TUTELA' in doutorado:
            del doutorado['CODIGO-INSTITUICAO-OUTRA-CO-TUTELA']
        if 'CODIGO-INSTITUICAO-CO-TUTELA' in doutorado:
            del doutorado['CODIGO-INSTITUICAO-CO-TUTELA']
        if 'NOME-DO-ORIENTADOR-SANDUICHE' in doutorado:
            del doutorado['NOME-DO-ORIENTADOR-SANDUICHE']
        if 'CODIGO-INSTITUICAO-OUTRA-SANDUICHE' in doutorado:
            del doutorado['CODIGO-INSTITUICAO-OUTRA-SANDUICHE']
        if 'CODIGO-INSTITUICAO-SANDUICHE' in doutorado:
            del doutorado['CODIGO-INSTITUICAO-SANDUICHE']
        if 'NOME-DO-CO-ORIENTADOR' in doutorado:
            del doutorado['NOME-DO-CO-ORIENTADOR']
    if len(doutorado) < 1:
        doutorado.update({'SEQUENCIA-FORMACAO': 'NI', 
                          'COMP_INI': 'NI', 
                          'COMP_FIM': 'NI', 
                          'NIVEL': 'NI', 
                          'CODIGO-INSTITUICAO': 'NI', 
                          'NOME-INSTITUICAO': 'NI', 
                          'CODIGO-CURSO': 'NI', 
                          'NOME-CURSO': 'NI', 
                          'STATUS-DO-CURSO': 'NI', 
                          'ANO-DE-INICIO': 'NI', 
                          'ANO-DE-CONCLUSAO': 'NI',
                          'ANO-DE-OBTENCAO-DO-TITULO': 'NI', 
                          'TITULO-DA-DISSERTACAO-TESE': 'NI', 
                          'NOME-COMPLETO-DO-ORIENTADOR': 'NI', 
                          'TIPO-DOUTORADO': 'NI', 
                          'CODIGO-INSTITUICAO-DOUT': 'NI', 
                          'NOME-INSTITUICAO-DOUT': 'NI', 
                          'NOME-ORIENTADOR-DOUT': 'NI', 
                          'NUMERO-ID-ORIENTADOR': 'NI'})
    return(doutorado)

In [ ]:
# Cria novas chaves e apaga as chaves antigas no dicionário doutorado.

def atualiza_chaves_doutorado(doutorado_sem_atualizar):
    doutorado = {}
    doutorado = doutorado_sem_atualizar
    doutorado['SEQUENCIA_FORMACAO_DOUT'] = doutorado.pop('SEQUENCIA-FORMACAO')
    doutorado['COMP_INI_DOUT'] = doutorado.pop('COMP_INI')
    doutorado['COMP_FIM_DOUT'] = doutorado.pop('COMP_FIM')
    doutorado['NIVEL_DOUT'] = doutorado.pop('NIVEL')
    doutorado['COD_INST_DOUT'] = doutorado.pop('CODIGO-INSTITUICAO')
    doutorado['NOME_INST_DOUT'] = doutorado.pop('NOME-INSTITUICAO')
    doutorado['COD_CURSO_DOUT'] = doutorado.pop('CODIGO-CURSO')
    doutorado['NOME_CURSO_DOUT'] = doutorado.pop('NOME-CURSO')
    doutorado['STATUS_CURSO_DOUT'] = doutorado.pop('STATUS-DO-CURSO')
    doutorado['ANO_INICIO_DOUT'] = doutorado.pop('ANO-DE-INICIO')
    doutorado['ANO_CONCLUSAO_DOUT'] = doutorado.pop('ANO-DE-CONCLUSAO')
    doutorado['ANO_OBTENCAO_TITULO_DOUT'] = doutorado.pop('ANO-DE-OBTENCAO-DO-TITULO')
    doutorado['TITULO_TESE_DOUT'] = doutorado.pop('TITULO-DA-DISSERTACAO-TESE')
    doutorado['NOME_COMPLETO_ORIENTADOR_DOUT'] = doutorado.pop('NOME-COMPLETO-DO-ORIENTADOR')
    doutorado['TIPO_DOUT'] = doutorado.pop('TIPO-DOUTORADO')
    doutorado['COD_INST_DOUT_SAND'] = doutorado.pop('CODIGO-INSTITUICAO-DOUT')
    doutorado['NOME_INST_DOUT_SAND'] = doutorado.pop('NOME-INSTITUICAO-DOUT')
    doutorado['NOME_ORIENTADOR_DOUT_SAND'] = doutorado.pop('NOME-ORIENTADOR-DOUT')
    doutorado['NR_ID_ORIENTADOR_DOUT'] = doutorado.pop('NUMERO-ID-ORIENTADOR')
    return(doutorado)

## 2.9. Livre Docência

In [ ]:
# Coleta os dados do element 'LIVRE-DOCENCIA' para o dicionário livre_docencia.

def extrai_livre_docencia():
    livre_docencia = {}
    competencia_ini = ''
    competencia_ini_livre_doc = ''
    competencia_fim = ''
    competencia_fim_livre_doc = ''

    for filho in root.iter('LIVRE-DOCENCIA'):
        livre_docencia.update(filho.attrib)

        # Formato Ano Mês: YYYYMM
        competencia_ini = livre_docencia['ANO-DE-INICIO'] + '01'
        competencia_ini_livre_doc = {'COMP_INI': competencia_ini}
        livre_docencia.update(competencia_ini_livre_doc)

        competencia_fim = livre_docencia['ANO-DE-CONCLUSAO'] +  '12'
        competencia_fim_livre_doc = {'COMP_FIM': competencia_fim}
        livre_docencia.update(competencia_fim_livre_doc)

        if 'TITULO-DO-TRABALHO-INGLES' in livre_docencia:
            del livre_docencia["TITULO-DO-TRABALHO-INGLES"]
    if len(livre_docencia) < 1:
        livre_docencia.update({'SEQUENCIA-FORMACAO': 'NI', 
                               'COMP_INI': 'NI', 
                               'COMP_FIM': 'NI', 
                               'NIVEL': 'NI', 
                               'CODIGO-INSTITUICAO': 'NI', 
                               'NOME-INSTITUICAO': 'NI',
                               'ANO-DE-OBTENCAO-DO-TITULO': 'NI', 
                               'TITULO-DO-TRABALHO': 'NI'})
    return(livre_docencia)

In [ ]:
# Cria novas chaves e apaga as chaves antigas no dicionário livre_docencia.

def atualiza_chaves_livre_docencia(livre_docencia_sem_atualizar):
    livre_docencia = {}
    livre_docencia = livre_docencia_sem_atualizar
    livre_docencia['SEQUENCIA_FORMACAO_LIV_DOC'] = livre_docencia.pop('SEQUENCIA-FORMACAO')
    livre_docencia['COMP_INI_LIV_DOC'] = livre_docencia.pop('COMP_INI')
    livre_docencia['COMP_FIM_LIV_DOC'] = livre_docencia.pop('COMP_FIM')
    livre_docencia['NIVEL_LIV_DOC'] = livre_docencia.pop('NIVEL')
    livre_docencia['COD_INST_LIV_DOC'] = livre_docencia.pop('CODIGO-INSTITUICAO')
    livre_docencia['NOME_INST_LIV_DOC'] = livre_docencia.pop('NOME-INSTITUICAO')
    livre_docencia['ANO_OBTENCAO_TITULO_LIV_DOC'] = livre_docencia.pop('ANO-DE-OBTENCAO-DO-TITULO')
    livre_docencia['TITULO_DO_TRABALHO_LIV_DOC'] = livre_docencia.pop('TITULO-DO-TRABALHO')
    return(livre_docencia)

## 2.10. Pós-Doutorado

In [ ]:
# Coleta os dados do element 'POS-DOUTORADO' para o dicionário pos_doutorado.

def extrai_pos_doutorado():
    pos_doutorado = {}
    competencia_ini = ''
    competencia_ini_pos_dout = ''
    competencia_fim = ''
    competencia_fim_os_dout = ''

    for filho in root.iter('POS-DOUTORADO'):
        pos_doutorado.update(filho.attrib)

        # Formato Ano Mês: YYYYMM
        competencia_ini = pos_doutorado['ANO-DE-INICIO'] + '01'
        competencia_ini_pos_dout = {'COMP_INI': competencia_ini}
        pos_doutorado.update(competencia_ini_pos_dout)

        competencia_fim = pos_doutorado['ANO-DE-CONCLUSAO'] +  '12'
        competencia_fim_pos_dout = {'COMP_FIM': competencia_fim}
        pos_doutorado.update(competencia_fim_pos_dout)

        if 'FLAG-BOLSA' in pos_doutorado:
            del pos_doutorado['FLAG-BOLSA'] 
        if 'CODIGO-AGENCIA-FINANCIADORA' in pos_doutorado:
            del pos_doutorado['CODIGO-AGENCIA-FINANCIADORA']
        if 'NOME-AGENCIA' in pos_doutorado:
            del pos_doutorado['NOME-AGENCIA']
        if 'CODIGO-CURSO-CAPES' in pos_doutorado:
            del pos_doutorado['CODIGO-CURSO-CAPES'] 
        if 'TITULO-DO-TRABALHO-INGLES' in pos_doutorado:
            del pos_doutorado['TITULO-DO-TRABALHO-INGLES']
        if 'NOME-CURSO-INGLES' in pos_doutorado:
            del pos_doutorado['NOME-CURSO-INGLES'] 
    if len(pos_doutorado) < 1:
        pos_doutorado.update({'SEQUENCIA-FORMACAO': 'NI', 
                              'COMP_INI': 'NI', 
                              'COMP_FIM': 'NI', 
                              'NIVEL': 'NI', 
                              'CODIGO-INSTITUICAO': 'NI', 
                              'NOME-INSTITUICAO': 'NI', 
                              'ANO-DE-INICIO': 'NI', 
                              'ANO-DE-CONCLUSAO': 'NI', 
                              'ANO-DE-OBTENCAO-DO-TITULO': 'NI',
                              'STATUS-DO-ESTAGIO': 'NI', 
                              'STATUS-DO-CURSO': 'NI', 
                              'NUMERO-ID-ORIENTADOR': 'NI',
                              'TITULO-DO-TRABALHO': 'NI'})
    return(pos_doutorado)

In [ ]:
# Cria novas chaves e apaga as chaves antigas no dicionário pos_doutorado

def atualiza_chaves_pos_doutorado(pos_doutorado_sem_atualizar):
    pos_doutorado = {}
    pos_doutorado = pos_doutorado_sem_atualizar
    pos_doutorado['SEQUENCIA_FORMACAO_POS_DOUT'] = pos_doutorado.pop('SEQUENCIA-FORMACAO')
    pos_doutorado['COMP_INI_POS_DOUT'] = pos_doutorado.pop('COMP_INI')
    pos_doutorado['COMP_FIM_POS_DOUT'] = pos_doutorado.pop('COMP_FIM')
    pos_doutorado['NIVEL_POS_DOUT'] = pos_doutorado.pop('NIVEL')
    pos_doutorado['COD_INST_POS_DOUT'] = pos_doutorado.pop('CODIGO-INSTITUICAO')
    pos_doutorado['NOME_INST_POS_DOUT'] = pos_doutorado.pop('NOME-INSTITUICAO')
    pos_doutorado['ANO_INICIO_POS_DOUT'] = pos_doutorado.pop('ANO-DE-INICIO')
    pos_doutorado['ANO_CONCLUSAO_POS_DOUT'] = pos_doutorado.pop('ANO-DE-CONCLUSAO')
    pos_doutorado['ANO_OBTENCAO_TITULO_POS_DOUT'] = pos_doutorado.pop('ANO-DE-OBTENCAO-DO-TITULO')
    pos_doutorado['STATUS_ESTAGIO_POS_DOUT'] = pos_doutorado.pop('STATUS-DO-ESTAGIO')
    pos_doutorado['STATUS_CURSO_POS_DOUT'] = pos_doutorado.pop('STATUS-DO-CURSO')
    pos_doutorado['NR_ID_ORIENTADOR_POS_DOUT'] = pos_doutorado.pop('NUMERO-ID-ORIENTADOR')
    pos_doutorado['TITULO_TRABALHO_POS_DOUT'] = pos_doutorado.pop('TITULO-DO-TRABALHO')
    return(pos_doutorado)

In [ ]:
def consolida_formacao_academica(dados_gerais, curso_tecnico, segundo_grau, 
                                 graduacao, aperfeiçoamento, especializacao, 
                                 mestrado, mestrado_prof, doutorado, 
                                 livre_docencia, pos_doutorado):
    # contém a formação acadêmica de um arquivo XML
    formacao_academica = {}
    
    # Junção de dicionários com **. 
    #Cria uma linha que será acrescentada na lista_formacao_academica
    formacao_academica = {**dados_gerais, **segundo_grau, **curso_tecnico, 
                          **graduacao, **aperfeiçoamento, **especializacao,
                          **mestrado, **mestrado_prof, **doutorado, 
                          **livre_docencia, **pos_doutorado}
    return(formacao_academica)

#  III. Produção Bibliográfica

## 3.1. Trabalho

In [ ]:
# Coleta os dados do element TRABALHO-EM-EVENTOS do dicionário XML Tree.
# Cria as listas trabalhos e autores_trabalhos para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_trabalhos(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    trabalhos = []
    autores_trabalhos = []
    sequencia_prod_trabalho = {}
    mes = 0
    mes_competencia_trabalho = ''
    competencia_trabalho = ''
    id_prod_biblio = ''

    for filho1 in root.iter('TRABALHO-EM-EVENTOS'):
        trabalhos.append(id_lattes)
        sequencia_prod_trabalho = filho1.attrib
        trabalhos.append(sequencia_prod_trabalho)    
        for filho2 in filho1.iter('DADOS-BASICOS-DO-TRABALHO'):        
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            trabalhos.append(id_prod_biblio)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_trabalho = '0' + str(mes)
            else:
                mes_competencia_trabalho = str(mes)       
            
            # Formato Ano Mês: YYYYMM
            competencia_trabalho = filho2.attrib['ANO-DO-TRABALHO'] + mes_competencia_trabalho
            competencia_trabalho = {'COMPETENCIA': competencia_trabalho}
            trabalhos.append(competencia_trabalho)
            trabalhos.append(filho2.attrib)
            if 'HOME-PAGE-DO-TRABALHO' in trabalhos:
                del trabalhos['HOME-PAGE-DO-TRABALHO']
            if 'FLAG-RELEVANCIA' in trabalhos:
                del trabalhos['FLAG-RELEVANCIA']
            if 'TITULO-DO-TRABALHO-INGLES' in trabalhos:
                del trabalhos['TITULO-DO-TRABALHO-INGLES']
            if 'FLAG-DIVULGACAO-CIENTIFICA' in trabalhos:
                del trabalhos['FLAG-DIVULGACAO-CIENTIFICA']
            for filho3 in filho1.iter('DETALHAMENTO-DO-TRABALHO'):
                trabalhos.append(filho3.attrib)
                if 'NOME-DO-EVENTO-INGLES' in trabalhos:
                    del trabalhos['NOME-DO-EVENTO-INGLES']
                for filho4 in filho1.iter('AUTORES'):
                    autores_trabalhos.append(id_lattes)                    
                    if filho4.attrib['NOME-COMPLETO-DO-AUTOR'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}                                    
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}                                   
                    autores_trabalhos.append(id_autor)
                    autores_trabalhos.append(id_parent_node)
                    autores_trabalhos.append(sequencia_prod_trabalho)
                    autores_trabalhos.append(id_prod_biblio)
                    autores_trabalhos.append(competencia_trabalho)
                    autores_trabalhos.append(filho4.attrib)
                    if 'NRO-ID-CNPQ' in autores_trabalhos:
                        del autores_trabalhos['NRO-ID-CNPQ']
    return(trabalhos, autores_trabalhos)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_trabalho.
# Inclui os dicionários atualizados na lista_trabalhos.

def atualiza_chaves_trabalhos(trabalhos_sem_atualizar):
    trabalhos = []
    trabalhos = trabalhos_sem_atualizar
    qtde_trabalhos = len(trabalhos)//6
    i = 0
    itens_trabalho = {}
    lista_trabalhos = []

    if qtde_trabalhos >= 1:
        while (i < qtde_trabalhos):
            itens_trabalho = dict(
            ID_LATTES = trabalhos[i*6]['ID_LATTES'], 
            SEQUENCIA = trabalhos[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = trabalhos[(i*6)+2]['ID-PROD-BIBLIO'],
            COD_NIVEL = 8, 
            COMPETENCIA = trabalhos[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_TRABALHO = trabalhos[(i*6)+4]['NATUREZA'], 
            TITULO_TRABALHO = trabalhos[(i*6)+4]['TITULO-DO-TRABALHO'],
            ANO_TRABALHO = trabalhos[(i*6)+4]['ANO-DO-TRABALHO'], 
            PAIS_EVENTO_TRAB = trabalhos[(i*6)+4]['PAIS-DO-EVENTO'], 
            IDIOMA_TRABALHO = trabalhos[(i*6)+4]['IDIOMA'], 
            MEIO_DIVULGACAO_TRAB = trabalhos[(i*6)+4]['MEIO-DE-DIVULGACAO'], 
            DOI_TRABALHO = trabalhos[(i*6)+4]['DOI'], 
            CLASSIFICACAO_EVENTO_TRAB = trabalhos[(i*6)+5]['CLASSIFICACAO-DO-EVENTO'], 
            NOME_EVENTO_TRAB = trabalhos[(i*6)+5]['NOME-DO-EVENTO'], 
            CIDADE_EVENTO_TRAB = trabalhos[(i*6)+5]['CIDADE-DO-EVENTO'], 
            ANO_REALIZACAO_TRAB = trabalhos[(i*6)+5]['ANO-DE-REALIZACAO'], 
            TITULO_ANAIS_PROC_TRAB = trabalhos[(i*6)+5]['TITULO-DOS-ANAIS-OU-PROCEEDINGS'], 
            VOLUME_TRAB = trabalhos[(i*6)+5]['VOLUME'], 
            FASCICULO_TRAB = trabalhos[(i*6)+5]['FASCICULO'], 
            SERIE_TRAB = trabalhos[(i*6)+5]['SERIE'], 
            PAG_INICIAL_TRAB = trabalhos[(i*6)+5]['PAGINA-INICIAL'], 
            PAG_FINAL_TRAB = trabalhos[(i*6)+5]['PAGINA-FINAL'], 
            ISBN_TRAB = trabalhos[(i*6)+5]['ISBN'], 
            NOME_EDITORA_TRAB = trabalhos[(i*6)+5]['NOME-DA-EDITORA'], 
            CIDADE_EDITORA_TRAB = trabalhos[(i*6)+5]['CIDADE-DA-EDITORA'])   
            lista_trabalhos.append(itens_trabalho)
            itens_trabalho = {}
            i += 1
    return(lista_trabalhos)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_trabalho.
# Inclui os dicionários atualizados na lista_autores_trabalhos.

def atualiza_chaves_autores_trabalhos(autores_trabalhos_sem_atualizar):
    autores_trabalhos = []
    autores_trabalhos = autores_trabalhos_sem_atualizar
    qtde_autores_trabalho = len(autores_trabalhos)//7
    i = 0
    autor_trabalho = {}
    lista_autores_trabalhos = []

    if qtde_autores_trabalho >= 1:
        while (i < qtde_autores_trabalho):
            autor_trabalho = dict(
            ID_LATTES = autores_trabalhos[i*7]['ID_LATTES'], 
            ID_AUTOR = autores_trabalhos[(i*7)+1]['ID_AUTOR'], 
            ID_AUTOR_PAI = autores_trabalhos[(i*7)+2]['ID_AUTOR_PAI'], 
            SEQUENCIA = autores_trabalhos[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_trabalhos[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 8, 
            COMPETENCIA = autores_trabalhos[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Produção Bibliográfica', 
            TIPO_NIVEL_2 = 'Trabalho em Evento', 
            NOME_AUTOR = autores_trabalhos[(i*7)+6]['NOME-COMPLETO-DO-AUTOR'], 
            NOME_CITACAO = autores_trabalhos[(i*7)+6]['NOME-PARA-CITACAO'], 
            ORDEM_AUTORIA = autores_trabalhos[(i*7)+6]['ORDEM-DE-AUTORIA'])
            lista_autores_trabalhos.append(autor_trabalho)
            autor_trabalho = {}
            i += 1
    return(lista_autores_trabalhos)

## 3.2. Artigo

In [ ]:
# Coleta os dados do element ARTIGO-PUBLICADO do dicionário XML Tree.
# Cria as listas artigos e autores_artigos para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_artigos(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    artigos = []
    autores_artigos = []
    sequencia_prod_artigo = []
    mes = 0
    mes_competencia_artigo = ''
    competencia_artigo = ''
    id_prod_biblio = ''

    for filho1 in root.iter('ARTIGO-PUBLICADO'):
        #print(filho1.tag, filho1.attrib)
        artigos.append(id_lattes)
        sequencia_prod_artigo = filho1.attrib
        artigos.append(sequencia_prod_artigo)
        if 'ORDEM-IMPORTANCIA' in artigos:
            del artigos['ORDEM-IMPORTANCIA']
        for filho2 in filho1.iter('DADOS-BASICOS-DO-ARTIGO'):
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            artigos.append(id_prod_biblio)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_artigo = '0' + str(mes)
            else:
                mes_competencia_artigo = str(mes)       

            # Formato Ano Mês: YYYYMM
            competencia_artigo = filho2.attrib['ANO-DO-ARTIGO'] + mes_competencia_artigo
            competencia_artigo = {'COMPETENCIA': competencia_artigo}
            artigos.append(competencia_artigo)
            artigos.append(filho2.attrib)
            if 'HOME-PAGE-DO-TRABALHO' in artigos:
                del artigos['HOME-PAGE-DO-TRABALHO']
            if 'FLAG-RELEVANCIA' in artigos:
                del artigos['FLAG-RELEVANCIA']
            if 'TITULO-DO-ARTIGO-INGLES' in artigos:
                del artigos['TITULO-DO-ARTIGO-INGLES'] 
            if 'FLAG-DIVULGACAO-CIENTIFICA' in artigos:
                del artigos['FLAG-DIVULGACAO-CIENTIFICA']
            for filho3 in filho1.iter('DETALHAMENTO-DO-ARTIGO'):
                artigos.append(filho3.attrib)
                for filho4 in filho1.iter('AUTORES'):
                    autores_artigos.append(id_lattes)
                    if filho4.attrib['NOME-COMPLETO-DO-AUTOR'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}            
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}
                    autores_artigos.append(id_autor)
                    autores_artigos.append(id_parent_node)
                    autores_artigos.append(sequencia_prod_artigo)
                    autores_artigos.append(id_prod_biblio)
                    autores_artigos.append(competencia_artigo)
                    autores_artigos.append(filho4.attrib)
                    if 'FLAG-DIVULGACAO-CIENTIFICA' in autores_artigos:
                        del autores_artigos['NRO-ID-CNPQ']
    return(artigos, autores_artigos)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_artigo.
# Inclui os dicionários atualizados na lista_artigos.

def atualiza_chaves_artigos(artigos_sem_atualizar):
    artigos = []
    artigos = artigos_sem_atualizar
    qtde_artigos = len(artigos)//6
    i = 0
    itens_artigo = {}
    lista_artigos = []

    if qtde_artigos >= 1:
        while (i < qtde_artigos):
            itens_artigo = dict(
            ID_LATTES = artigos[i*6]['ID_LATTES'], 
            SEQUENCIA = artigos[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = artigos[(i*6)+2]['ID-PROD-BIBLIO'],
            COD_NIVEL = 1, 
            COMPETENCIA = artigos[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_ARTIGO = artigos[(i*6)+4]['NATUREZA'], 
            TITULO_ARTIGO = artigos[(i*6)+4]['TITULO-DO-ARTIGO'], 
            ANO_ARTIGO = artigos[(i*6)+4]['ANO-DO-ARTIGO'], 
            PAIS_PUBLICACAO_ARTIGO = artigos[(i*6)+4]['PAIS-DE-PUBLICACAO'], 
            IDIOMA_ARTIGO = artigos[(i*6)+4]['IDIOMA'], 
            MEIO_DIVULGACAO_ARTIGO = artigos[(i*6)+4]['MEIO-DE-DIVULGACAO'], 
            DOI_ARTIGO = artigos[(i*6)+4]['DOI'], 
            TITULO_PERIODICO_REVISTA = artigos[(i*6)+5]['TITULO-DO-PERIODICO-OU-REVISTA'],
            ISSN_ARTIGO = artigos[(i*6)+5]['ISSN'], 
            VOLUME_ARTIGO = artigos[(i*6)+5]['VOLUME'], 
            FASCICULO_ARTIGO = artigos[(i*6)+5]['FASCICULO'], 
            SERIE_ARTIGO = artigos[(i*6)+5]['SERIE'], 
            PAG_INICIAL_ARTIGO = artigos[(i*6)+5]['PAGINA-INICIAL'], 
            PAG_FINAL_ARTIGO = artigos[(i*6)+5]['PAGINA-FINAL'], 
            LOCAL_PUBLICACAO_ARTIGO = artigos[(i*6)+5]['LOCAL-DE-PUBLICACAO'])   
            lista_artigos.append(itens_artigo)
            itens_artigo = {}
            i += 1
    return(lista_artigos) 

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_artigo.
# Inclui os dicionários atualizados na lista_autores_artigos.

def atualiza_chaves_autores_artigos(autores_artigos_sem_atualizar):
    autores_artigos = []
    autores_artigos = autores_artigos_sem_atualizar
    qtde_autores_artigo = len(autores_artigos)//7
    i = 0
    autor_artigo = {}
    lista_autores_artigos = []

    if qtde_autores_artigo >= 1:
        while (i < qtde_autores_artigo):
            autor_artigo = dict(
            ID_LATTES = autores_artigos[i*7]['ID_LATTES'], 
            ID_AUTOR = autores_artigos[(i*7)+1]['ID_AUTOR'], 
            ID_AUTOR_PAI = autores_artigos[(i*7)+2]['ID_AUTOR_PAI'], 
            SEQUENCIA = autores_artigos[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_artigos[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 1, 
            COMPETENCIA = autores_artigos[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Produção Bibliográfica', 
            TIPO_NIVEL_2 = 'Artigo Publicado', 
            NOME_AUTOR = autores_artigos[(i*7)+6]['NOME-COMPLETO-DO-AUTOR'], 
            NOME_CITACAO = autores_artigos[(i*7)+6]['NOME-PARA-CITACAO'], 
            ORDEM_AUTORIA = autores_artigos[(i*7)+6]['ORDEM-DE-AUTORIA'])
            lista_autores_artigos.append(autor_artigo)
            autor_artigo = {}
            i += 1
    return(lista_autores_artigos)

## 3.3. Livro

In [ ]:
# Coleta os dados do element LIVRO-PUBLICADO-OU-ORGANIZADO do dicionário XML Tree.
# Cria as listas livros e autores_livros para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.


def extrai_livros(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    livros = []
    autores_livros = []
    sequencia_prod_livro = []
    mes = 0
    mes_competencia_livro = ''
    competencia_livro = ''
    id_prod_biblio = ''

    for filho1 in root.iter('LIVRO-PUBLICADO-OU-ORGANIZADO'):
        #print(filho1.tag, filho1.attrib)
        livros.append(id_lattes)
        sequencia_prod_livro = filho1.attrib
        livros.append(sequencia_prod_livro)
        for filho2 in filho1.iter('DADOS-BASICOS-DO-LIVRO'):
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            livros.append(id_prod_biblio)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_livro = '0' + str(mes)
            else:
                mes_competencia_livro = str(mes)       
            
            # Formato Ano Mês: YYYYMM
            competencia_livro = filho2.attrib['ANO'] + mes_competencia_livro
            competencia_livro = {'COMPETENCIA': competencia_livro}
            livros.append(competencia_livro)
            livros.append(filho2.attrib)
            if 'HOME-PAGE-DO-TRABALHO' in livros:
                del livros['HOME-PAGE-DO-TRABALHO']
            if 'FLAG-RELEVANCIA' in livros:
                del livros['FLAG-RELEVANCIA'] 
            if 'TITULO-DO-LIVRO-INGLES' in livros:
                del livros['TITULO-DO-LIVRO-INGLES'] 
            if 'FLAG-DIVULGACAO-CIENTIFICA' in livros:
                del livros['FLAG-DIVULGACAO-CIENTIFICA']
            for filho3 in filho1.iter('DETALHAMENTO-DO-LIVRO'):
                livros.append(filho3.attrib)
                for filho4 in filho1.iter('AUTORES'):
                    autores_livros.append(id_lattes)
                    if filho4.attrib['NOME-COMPLETO-DO-AUTOR'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}                                    
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}                                   
                    autores_livros.append(id_autor)
                    autores_livros.append(id_parent_node)
                    autores_livros.append(sequencia_prod_livro)
                    autores_livros.append(id_prod_biblio)
                    autores_livros.append(competencia_livro)
                    autores_livros.append(filho4.attrib)
                    if 'NRO-ID-CNPQ' in autores_livros:
                        del autores_livros['NRO-ID-CNPQ']
    return(livros, autores_livros)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_livro.
# Inclui os dicionários atualizados na lista_livros.

def atualiza_chaves_livros(livros_sem_atualizar):
    livros = []
    livros = livros_sem_atualizar
    qtde_livros = len(livros)//6
    i = 0
    itens_livro = {}
    lista_livros = []

    if qtde_livros >= 1:
        while (i < qtde_livros):
            itens_livro = dict(
            ID_LATTES = livros[i*6]['ID_LATTES'], 
            SEQUENCIA = livros[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = livros[(i*6)+2]['ID-PROD-BIBLIO'],
            COD_NIVEL = 4, 
            COMPETENCIA = livros[(i*6)+3]['COMPETENCIA'], 
            TIPO_LIVRO = livros[(i*6)+4]['TIPO'], 
            NATUREZA_LIVRO = livros[(i*6)+4]['NATUREZA'], 
            TITULO_LIVRO = livros[(i*6)+4]['TITULO-DO-LIVRO'], 
            ANO_LIVRO = livros[(i*6)+4]['ANO'], 
            PAIS_PUBLICACAO_LIVRO = livros[(i*6)+4]['PAIS-DE-PUBLICACAO'], 
            IDIOMA_LIVRO = livros[(i*6)+4]['IDIOMA'], 
            MEIO_DIVULGACAO_LIVRO = livros[(i*6)+4]['MEIO-DE-DIVULGACAO'], 
            DOI_LIVRO = livros[(i*6)+4]['DOI'], 
            NR_VOLUMES_LIVRO = livros[(i*6)+5]['NUMERO-DE-VOLUMES'], 
            NR_PAGINAS_LIVRO = livros[(i*6)+5]['NUMERO-DE-PAGINAS'], 
            ISBN_LIVRO = livros[(i*6)+5]['ISBN'], 
            NR_EDICAO_REVISAO_LIVRO = livros[(i*6)+5]['NUMERO-DA-EDICAO-REVISAO'], 
            NR_SERIE_LIVRO = livros[(i*6)+5]['NUMERO-DA-SERIE'], 
            CIDADE_EDITORA_LIVRO = livros[(i*6)+5]['CIDADE-DA-EDITORA'], 
            NOME_EDITORA_LIVRO = livros[(i*6)+5]['NOME-DA-EDITORA']) 
            lista_livros.append(itens_livro)
            itens_livro = {}
            i += 1
    return(lista_livros) 

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_livro.
# Inclui os dicionários atualizados na lista_autores_livros.

def atualiza_chaves_autores_livros(autores_livros_sem_atualizar):
    autores_livros = []
    autores_livros = autores_livros_sem_atualizar
    qtde_autores_livros = len(autores_livros)//7
    i = 0
    autor_livro = {}
    lista_autores_livros = []

    if qtde_autores_livros >= 1:
        while (i < qtde_autores_livros):
            autor_livro = dict(
            ID_LATTES = autores_livros[i*7]['ID_LATTES'], 
            ID_AUTOR = autores_trabalhos[(i*7)+1]['ID_AUTOR'], 
            ID_AUTOR_PAI = autores_trabalhos[(i*7)+2]['ID_AUTOR_PAI'], 
            SEQUENCIA = autores_livros[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_livros[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 4, 
            COMPETENCIA = autores_livros[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Produção Bibliográfica', 
            TIPO_NIVEL_2 = 'Livro Publicado ou Organizado', 
            NOME_AUTOR = autores_livros[(i*7)+6]['NOME-COMPLETO-DO-AUTOR'], 
            NOME_CITACAO = autores_livros[(i*7)+6]['NOME-PARA-CITACAO'], 
            ORDEM_AUTORIA = autores_livros[(i*7)+6]['ORDEM-DE-AUTORIA'])
            lista_autores_livros.append(autor_livro)
            autor_livro = {}
            i += 1
    return(lista_autores_livros)

## 3.4. Capítulo de Livro

In [ ]:
# Coleta os dados do element CAPITULO-DE-LIVRO-PUBLICADO do dicionário XML Tree.
# Cria as listas cap_livros e autores_cap_livro para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_cap_livros(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    cap_livros = []
    autores_cap_livro = []
    sequencia_prod_cap_livro = []
    mes = 0
    mes_competencia_cap_livro = ''
    competencia_cap_livro = ''
    id_prod_biblio = ''

    for filho1 in root.iter('CAPITULO-DE-LIVRO-PUBLICADO'):
        #print(filho1.tag, filho1.attrib)
        cap_livros.append(id_lattes)
        sequencia_prod_cap_livro = filho1.attrib
        cap_livros.append(sequencia_prod_cap_livro)
        for filho2 in filho1.iter('DADOS-BASICOS-DO-CAPITULO'):
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            cap_livros.append(id_prod_biblio)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_cap_livro = '0' + str(mes)
            else:
                mes_competencia_cap_livro = str(mes)       
            # Formato Ano Mês: YYYYMM
            competencia_cap_livro = filho2.attrib['ANO'] + mes_competencia_cap_livro
            competencia_cap_livro = {'COMPETENCIA': competencia_cap_livro}
            cap_livros.append(competencia_cap_livro)
            cap_livros.append(filho2.attrib)
            if 'FLAG-RELEVANCIA' in cap_livros:
                del cap_livros['HOME-PAGE-DO-TRABALHO']
            if 'FLAG-RELEVANCIA' in cap_livros:
                del cap_livros['FLAG-RELEVANCIA']
            if 'FLAG-RELEVANCIA' in cap_livros:
                del cap_livros['TITULO-DO-CAPITULO-DO-LIVRO-INGLES'] 
            if 'FLAG-RELEVANCIA' in cap_livros:
                del cap_livros['FLAG-DIVULGACAO-CIENTIFICA'] 
            for filho3 in filho1.iter('DETALHAMENTO-DO-CAPITULO'):
                cap_livros.append(filho3.attrib)
                for filho4 in filho1.iter('AUTORES'):
                    autores_cap_livro.append(id_lattes)
                    if filho4.attrib['NOME-COMPLETO-DO-AUTOR'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}                                    
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}                                   
                    autores_cap_livro.append(id_autor)
                    autores_cap_livro.append(id_parent_node)
                    autores_cap_livro.append(sequencia_prod_cap_livro)
                    autores_cap_livro.append(id_prod_biblio)
                    autores_cap_livro.append(competencia_cap_livro)
                    autores_cap_livro.append(filho4.attrib)
                    if 'FLAG-RELEVANCIA' in autores_cap_livro:
                        del autores_cap_livro['NRO-ID-CNPQ']
    return(cap_livros, autores_cap_livro)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_cap_livro.
# Inclui os dicionários atualizados na lista_cap_livros.

def atualiza_chaves_cap_livros(cap_livros_sem_atualizar):
    cap_livros = []
    cap_livros = cap_livros_sem_atualizar
    qtde_cap_livro = len(cap_livros)//6
    i = 0
    itens_cap_livro = {}
    lista_cap_livros = []

    if qtde_cap_livro >= 1:
        while (i < qtde_cap_livro):
            itens_cap_livro = dict(
            ID_LATTES = cap_livros[i*6]['ID_LATTES'], 
            SEQUENCIA = cap_livros[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = cap_livros[(i*6)+2]['ID-PROD-BIBLIO'],
            COD_NIVEL = 2, 
            COMPETENCIA = cap_livros[(i*6)+3]['COMPETENCIA'], 
            TIPO_CAP_LIVRO = cap_livros[(i*6)+4]['TIPO'], 
            TITULO_CAP_LIVRO = cap_livros[(i*6)+4]['TITULO-DO-CAPITULO-DO-LIVRO'],
            ANO_LIVRO = cap_livros[(i*6)+4]['ANO'], 
            PAIS_PUBLICACAO_CAP_LIVRO = cap_livros[(i*6)+4]['PAIS-DE-PUBLICACAO'], 
            IDIOMA_CAP_LIVRO = cap_livros[(i*6)+4]['IDIOMA'], 
            MEIO_DIVULGACAO_CAP_LIVRO = cap_livros[(i*6)+4]['MEIO-DE-DIVULGACAO'], 
            DOI_CAP_LIVRO = cap_livros[(i*6)+4]['DOI'], 
            TITULO_LIVRO = cap_livros[(i*6)+5]['TITULO-DO-LIVRO'], 
            NR_VOLUMES_CAP_LIVRO = cap_livros[(i*6)+5]['NUMERO-DE-VOLUMES'], 
            PAG_INICIAL_CAP_LIVRO = cap_livros[(i*6)+5]['PAGINA-INICIAL'], 
            PAG_FINAL_CAP_LIVRO = cap_livros[(i*6)+5]['PAGINA-FINAL'], 
            ISBN_CAP_LIVRO = cap_livros[(i*6)+5]['ISBN'], 
            ORGANIZADORES_CAP_LIVRO = cap_livros[(i*6)+5]['ORGANIZADORES'], 
            NR_EDICAO_REVISAO_CAP_LIVRO = cap_livros[(i*6)+5]['NUMERO-DA-EDICAO-REVISAO'], 
            NR_SERIE_CAP_LIVRO = cap_livros[(i*6)+5]['NUMERO-DA-SERIE'], 
            CIDADE_EDITORA_CAP_LIVRO = cap_livros[(i*6)+5]['CIDADE-DA-EDITORA'], 
            NOME_EDITORA_CAP_LIVRO = cap_livros[(i*6)+5]['NOME-DA-EDITORA'])
            lista_cap_livros.append(itens_cap_livro)
            itens_cap_livro = {}
            i += 1
    return(lista_cap_livros) 

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_cap_livro.
# Inclui os dicionários atualizados na lista_autores_cap_livros.

def atualiza_chaves_autores_cap_livros(autores_cap_livro_sem_atualizar):
    autores_cap_livro = []
    autores_cap_livro = autores_cap_livro_sem_atualizar
    qtde_autores_cap_livro = len(autores_cap_livro)//7
    i = 0
    autor_cap_livro = {}
    lista_autores_cap_livros = []

    if qtde_autores_cap_livro >= 1:
        while (i < qtde_autores_cap_livro):
            autor_cap_livro = dict(
            ID_LATTES = autores_cap_livro[i*7]['ID_LATTES'], 
            ID_AUTOR = autores_cap_livro[(i*7)+1]['ID_AUTOR'], 
            ID_AUTOR_PAI = autores_cap_livro[(i*7)+2]['ID_AUTOR_PAI'], 
            SEQUENCIA = autores_cap_livro[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_cap_livro[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 2, 
            COMPETENCIA = autores_cap_livro[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Produção Bibliográfica', 
            TIPO_NIVEL_2 = 'Capítulo de Livro Publicado', 
            NOME_AUTOR = autores_cap_livro[(i*7)+6]['NOME-COMPLETO-DO-AUTOR'], 
            NOME_CITACAO = autores_cap_livro[(i*7)+6]['NOME-PARA-CITACAO'], 
            ORDEM_AUTORIA = autores_cap_livro[(i*7)+6]['ORDEM-DE-AUTORIA'])
            lista_autores_cap_livros.append(autor_cap_livro)
            autor_cap_livro = {}
            i += 1
    return(lista_autores_cap_livros)

# 3.5. Texto

In [ ]:
# Coleta os dados do element TEXTO-EM-JORNAL-OU-REVISTA do dicionário XML Tree.
# Cria as listas textos e autores_texto para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_textos(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    textos = []
    autores_texto = []
    sequencia_prod_texto = []
    mes = 0
    mes_competencia_texto = ''
    competencia_texto = ''
    id_prod_biblio = ''

    for filho1 in root.iter('TEXTO-EM-JORNAL-OU-REVISTA'):
        #print(filho1.tag, filho1.attrib)
        textos.append(id_lattes)
        sequencia_prod_texto = filho1.attrib
        textos.append(sequencia_prod_texto)
        for filho2 in filho1.iter('DADOS-BASICOS-DO-TEXTO'):
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            textos.append(id_prod_biblio)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_texto = '0' + str(mes)
            else:
                mes_competencia_texto = str(mes)       

            # Formato Ano Mês: YYYYMM
            competencia_texto = filho2.attrib['ANO-DO-TEXTO'] + mes_competencia_texto
            competencia_texto = {'COMPETENCIA': competencia_texto}
            textos.append(competencia_texto)
            textos.append(filho2.attrib)
            if 'HOME-PAGE-DO-TRABALHO' in textos:
                del textos['HOME-PAGE-DO-TRABALHO']
            if 'FLAG-RELEVANCIA' in textos:
                del textos['FLAG-RELEVANCIA']
            if 'TITULO-DO-TEXTO-INGLES' in textos:
                del textos['TITULO-DO-TEXTO-INGLES']
            if 'FLAG-DIVULGACAO-CIENTIFICA' in textos:
                del textos['FLAG-DIVULGACAO-CIENTIFICA']
            if 'FORMATO_DATA_PUBLICACAO' in textos:
                del textos['FORMATO_DATA_PUBLICACAO']
            for filho3 in filho1.iter('DETALHAMENTO-DO-TEXTO'):
                textos.append(filho3.attrib)
                for filho4 in filho1.iter('AUTORES'):
                    autores_texto.append(id_lattes)
                    if filho4.attrib['NOME-COMPLETO-DO-AUTOR'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}                                    
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}                                   
                    autores_texto.append(id_autor)
                    autores_texto.append(id_parent_node)
                    autores_texto.append(sequencia_prod_texto)
                    autores_texto.append(id_prod_biblio)
                    autores_texto.append(competencia_texto)
                    autores_texto.append(filho4.attrib)
                    if 'NRO-ID-CNPQ' in autores_texto:
                        del autores_texto['NRO-ID-CNPQ']
    return(textos, autores_texto)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_texto.
# Inclui os dicionários atualizados na lista_textos.

def atualiza_chaves_textos(textos_sem_atualizar):
    textos = []
    textos = textos_sem_atualizar
    qtde_textos = len(textos)//6
    i = 0
    itens_texto = {}
    lista_textos = []

    if qtde_textos >= 1:
        while (i < qtde_textos):
            itens_texto = dict(
            ID_LATTES = textos[i*6]['ID_LATTES'], 
            SEQUENCIA = textos[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = textos[(i*6)+2]['ID-PROD-BIBLIO'],
            COD_NIVEL = 7, 
            COMPETENCIA = textos[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_TEXTO = textos[(i*6)+4]['NATUREZA'], 
            TITULO_TEXTO = textos[(i*6)+4]['TITULO-DO-TEXTO'], 
            ANO_TEXTO = textos[(i*6)+4]['ANO-DO-TEXTO'], 
            PAIS_PUBLICACAO_TEXTO = textos[(i*6)+4]['PAIS-DE-PUBLICACAO'], 
            IDIOMA_TEXTO = textos[(i*6)+4]['IDIOMA'], 
            MEIO_DIVULGACAO_TEXTO = textos[(i*6)+4]['MEIO-DE-DIVULGACAO'], 
            DOI_TEXTO = textos[(i*6)+4]['DOI'], 
            TITULO_JORNAL_REVISTA = textos[(i*6)+5]['TITULO-DO-JORNAL-OU-REVISTA'], 
            ISSN_TEXTO = textos[(i*6)+5]['ISSN'], 
            DATA_PUBLICACAO_TEXTO = textos[(i*6)+5]['DATA-DE-PUBLICACAO'], 
            VOLUME_TEXTO = textos[(i*6)+5]['VOLUME'], 
            PAG_INICIAL_TEXTO = textos[(i*6)+5]['PAGINA-INICIAL'], 
            PAG_FINAL_TEXTO = textos[(i*6)+5]['PAGINA-FINAL'], 
            LOCAL_PUBLICACAO_TEXTO = textos[(i*6)+5]['LOCAL-DE-PUBLICACAO'])
            lista_textos.append(itens_texto)
            itens_texto = {}
            i += 1
    return(lista_textos) 

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_texto.
# Inclui os dicionários atualizados na lista_autores_textos.

def atualiza_chaves_autores_textos(autores_textos_sem_atualizar):
    autores_texto = []
    autores_texto = autores_textos_sem_atualizar
    qtde_autores_texto = len(autores_texto)//7
    i = 0
    autor_texto = {}
    lista_autores_textos = []

    if qtde_autores_texto >= 1:
        while (i < qtde_autores_texto):
            autor_texto = dict(
            ID_LATTES = autores_texto[i*7]['ID_LATTES'], 
            ID_AUTOR = autores_texto[(i*7)+1]['ID_AUTOR'], 
            ID_AUTOR_PAI = autores_texto[(i*7)+2]['ID_AUTOR_PAI'], 
            SEQUENCIA = autores_texto[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_texto[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 7, 
            COMPETENCIA = autores_texto[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Produção Bibliográfica', 
            TIPO_NIVEL_2 = 'Texto em Jornal ou Revista', 
            NOME_AUTOR = autores_texto[(i*7)+6]['NOME-COMPLETO-DO-AUTOR'], 
            NOME_CITACAO = autores_texto[(i*7)+6]['NOME-PARA-CITACAO'], 
            ORDEM_AUTORIA = autores_texto[(i*7)+6]['ORDEM-DE-AUTORIA'])
            lista_autores_textos.append(autor_texto)
            autor_texto = {}
            i += 1   
    return(lista_autores_textos)

# 3.6. Outra Produção

In [ ]:
# Coleta os dados do element OUTRA-PRODUCAO-BIBLIOGRAFICA do dicionário XML Tree.
# Cria as listas outras_prod e autores_outras_prod para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_outras_prod(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    outras_prod = []
    autores_outras_prod = []
    sequencia_prod_outras = []
    mes = 0
    mes_competencia_outras_prod = ''
    competencia_outras_prod = ''
    id_prod_biblio = ''

    for filho1 in root.iter('OUTRA-PRODUCAO-BIBLIOGRAFICA'):
        #print(filho1.tag, filho1.attrib)
        sequencia_prod_outras = filho1.attrib
        outras_prod.append(id_lattes)
        outras_prod.append(sequencia_prod_outras)
        for filho2 in filho1.iter('DADOS-BASICOS-DE-OUTRA-PRODUCAO'):
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            outras_prod.append(id_prod_biblio)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_outras_prod = '0' + str(mes)
            else:
                mes_competencia_outras_prod = str(mes)       

            # Formato Ano Mês: YYYYMM
            competencia_outras_prod = filho2.attrib['ANO'] + mes_competencia_outras_prod
            competencia_outras_prod = {'COMPETENCIA': competencia_outras_prod}
            outras_prod.append(competencia_outras_prod)
            outras_prod.append(filho2.attrib)
            if 'HOME-PAGE-DO-TRABALHO' in outras_prod:
                del outras_prod['HOME-PAGE-DO-TRABALHO']
            if 'HOME-PAGE-DO-TRABALHO' in outras_prod:
                del outras_prod['FLAG-RELEVANCIA']
            if 'HOME-PAGE-DO-TRABALHO' in outras_prod:
                del outras_prod['TITULO-INGLES']
            if 'HOME-PAGE-DO-TRABALHO' in outras_prod:
                del outras_prod['NATUREZA-INGLES']
            if 'HOME-PAGE-DO-TRABALHO' in outras_prod:
                del outras_prod['FLAG-DIVULGACAO-CIENTIFICA']
            for filho3 in filho1.iter('DETALHAMENTO-DE-OUTRA-PRODUCAO'):
                outras_prod.append(filho3.attrib)
                for filho4 in filho1.iter('AUTORES'):
                    autores_outras_prod.append(id_lattes)                    
                    if filho4.attrib['NOME-COMPLETO-DO-AUTOR'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}                                    
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}                                   
                    autores_outras_prod.append(id_autor)
                    autores_outras_prod.append(id_parent_node) 
                    autores_outras_prod.append(sequencia_prod_outras)
                    autores_outras_prod.append(id_prod_biblio)
                    autores_outras_prod.append(competencia_outras_prod)
                    autores_outras_prod.append(filho4.attrib)
                    if 'HOME-PAGE-DO-TRABALHO' in autores_outras_prod:
                        del autores_outras_prod['NRO-ID-CNPQ']
    return(outras_prod, autores_outras_prod)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_outra_prod.
# Inclui os dicionários atualizados na lista_outras_prod.

def atualiza_chaves_outras_prod(outras_prod_sem_atualizar):
    outras_prod = []
    outras_prod = outras_prod_sem_atualizar
    qtde_outras_prod = len(outras_prod)//6
    i = 0
    itens_outra_prod = {}
    lista_outras_prod = []

    if qtde_outras_prod >= 1:
        while (i < qtde_outras_prod):
            itens_outra_prod = dict(
            ID_LATTES = outras_prod[i*6]['ID_LATTES'], 
            SEQUENCIA = outras_prod[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = outras_prod[(i*6)+2]['ID-PROD-BIBLIO'],
            COD_NIVEL = 5, 
            COMPETENCIA = outras_prod[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_OUTRA_PROD = outras_prod[(i*6)+4]['NATUREZA'], 
            TITULO_OUTRA_PROD = outras_prod[(i*6)+4]['TITULO'], 
            ANO_OUTRA_PROD = outras_prod[(i*6)+4]['ANO'], 
            PAIS_PUBLICACAO_OUTRA_PROD = outras_prod[(i*6)+4]['PAIS-DE-PUBLICACAO'], 
            IDIOMA_OUTRA_PROD = outras_prod[(i*6)+4]['IDIOMA'], 
            MEIO_DIVULGACAO_OUTRA_PROD = outras_prod[(i*6)+4]['MEIO-DE-DIVULGACAO'], 
            DOI_OUTRA_PROD = outras_prod[(i*6)+4]['DOI'], 
            EDITORA_OUTRA_PROD = outras_prod[(i*6)+5]['EDITORA'], 
            CIDADE_EDITORA_OUTRA_PROD = outras_prod[(i*6)+5]['CIDADE-DA-EDITORA'], 
            NR_PAG_OUTRA_PROD = outras_prod[(i*6)+5]['NUMERO-DE-PAGINAS'], 
            ISSN_ISBN_OUTRA_PROD = outras_prod[(i*6)+5]['ISSN-ISBN'])
            lista_outras_prod.append(itens_outra_prod)
            itens_outra_prod = {}
            i += 1
    return(lista_outras_prod) 

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_outra_prod.
# Inclui os dicionários atualizados na lista_autores_outras_prod.

def atualiza_chaves_autores_outras_prod(autores_outras_prod_sem_atualizar):
    autores_outras_prod = []
    autores_outras_prod = autores_outras_prod_sem_atualizar
    qtde_autores_outras_prod = len(autores_outras_prod)//7
    i = 0
    autor_outra_prod = {}
    lista_autores_outras_prod = []

    if qtde_autores_outras_prod >= 1:
        while (i < qtde_autores_outras_prod):
            autor_outra_prod = dict(
            ID_LATTES = autores_outras_prod[i*7]['ID_LATTES'],
            ID_AUTOR = autores_outras_prod[(i*7)+1]['ID_AUTOR'], 
            ID_AUTOR_PAI = autores_outras_prod[(i*7)+2]['ID_AUTOR_PAI'], 
            SEQUENCIA = autores_outras_prod[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_outras_prod[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 5, 
            COMPETENCIA = autores_outras_prod[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Produção Bibliográfica', 
            TIPO_NIVEL_2 = 'Outra Produção Bibliográfica', 
            NOME_AUTOR = autores_outras_prod[(i*7)+6]['NOME-COMPLETO-DO-AUTOR'], 
            NOME_CITACAO = autores_outras_prod[(i*7)+6]['NOME-PARA-CITACAO'], 
            ORDEM_AUTORIA = autores_outras_prod[(i*7)+6]['ORDEM-DE-AUTORIA'])
            lista_autores_outras_prod.append(autor_outra_prod)
            autor_outra_prod = {}
            i += 1
    return(lista_autores_outras_prod)

# 3.7. Demais Trabalhos

In [ ]:
# Coleta os dados do element DEMAIS-TRABALHOS do dicionário XML Tree.
# Cria as listas demais_trabalhos e autores_demais_trabalhos para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_demais_trabalhos(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    demais_trabalhos = []
    autores_demais_trabalhos = []
    sequencia_demais_trabalhos = []
    mes = 0
    mes_competencia_demais_trabalhos = ''
    competencia_demais_trabalhos = ''
    id_prod_biblio = ''

    for filho1 in root.iter('DEMAIS-TRABALHOS'):
        #print(filho1.tag, filho1.attrib)
        demais_trabalhos.append(id_lattes)
        sequencia_demais_trabalhos = filho1.attrib
        demais_trabalhos.append(sequencia_demais_trabalhos)
        for filho2 in filho1.iter('DADOS-BASICOS-DE-DEMAIS-TRABALHOS'):
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            demais_trabalhos.append(id_prod_biblio)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_demais_trabalhos = '0' + str(mes)
            else:
                mes_competencia_demais_trabalhos = str(mes)       

            # Formato Ano Mês: YYYYMM
            competencia_demais_trabalhos = filho2.attrib['ANO'] + mes_competencia_demais_trabalhos
            competencia_demais_trabalhos = {'COMPETENCIA': competencia_demais_trabalhos}
            demais_trabalhos.append(competencia_demais_trabalhos)
            demais_trabalhos.append(filho2.attrib)
            if 'HOME-PAGE' in demais_trabalhos:
                del demais_trabalhos['HOME-PAGE']
            if 'FLAG-RELEVANCIA' in demais_trabalhos:
                del demais_trabalhos['FLAG-RELEVANCIA']
            if 'TITULO-INGLES' in demais_trabalhos:
                del demais_trabalhos['TITULO-INGLES']
            if 'NATUREZA-INGLES' in demais_trabalhos:
                del demais_trabalhos['NATUREZA-INGLES']
            for filho3 in filho1.iter('DETALHAMENTO-DE-DEMAIS-TRABALHOS'):
                #print(filho3.tag, filho3.attrib)
                demais_trabalhos.append(filho3.attrib)
                for filho4 in filho1.iter('AUTORES'):
                    autores_demais_trabalhos.append(id_lattes)
                    if filho4.attrib['NOME-COMPLETO-DO-AUTOR'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}                                    
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}                                   
                    autores_demais_trabalhos.append(id_autor)
                    autores_demais_trabalhos.append(id_parent_node)
                    autores_demais_trabalhos.append(sequencia_demais_trabalhos)
                    autores_demais_trabalhos.append(id_prod_biblio)
                    autores_demais_trabalhos.append(competencia_demais_trabalhos)
                    autores_demais_trabalhos.append(filho4.attrib)
                    if 'HOME-PAGE-DO-TRABALHO' in autores_demais_trabalhos:
                        del autores_demais_trabalhos['NRO-ID-CNPQ']
    return(demais_trabalhos, autores_demais_trabalhos)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_demais_trabalhos.
# Inclui os dicionários atualizados na lista_demais_trabalhos.

def atualiza_chaves_demais_trabalhos(demais_trabalhos_sem_atualizar):
    demais_trabalhos = []
    demais_trabalhos = demais_trabalhos_sem_atualizar
    qtde_demais_trabalhos = len(demais_trabalhos)//6
    i = 0
    itens_demais_trabalhos = {}
    lista_demais_trabalhos = []

    if qtde_demais_trabalhos >= 1:
        while (i < qtde_demais_trabalhos):
            itens_demais_trabalhos = dict(
            ID_LATTES = demais_trabalhos[i*6]['ID_LATTES'], 
            SEQUENCIA = demais_trabalhos[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = demais_trabalhos[(i*6)+2]['ID-PROD-BIBLIO'],
            COD_NIVEL = 3, 
            COMPETENCIA = demais_trabalhos[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_DEMAIS_TRAB = demais_trabalhos[(i*6)+4]['NATUREZA'], 
            TITULO_DEMAIS_TRAB = demais_trabalhos[(i*6)+4]['TITULO'], 
            ANO_DEMAIS_TRAB = demais_trabalhos[(i*6)+4]['ANO'], 
            PAIS_DEMAIS_TRAB = demais_trabalhos[(i*6)+4]['PAIS'], 
            IDIOMA_DEMAIS_TRAB = demais_trabalhos[(i*6)+4]['IDIOMA'], 
            DOI_DEMAIS_TRAB = demais_trabalhos[(i*6)+4]['DOI'], 
            FINALIDADE_DEMAIS_TRAB = demais_trabalhos[(i*6)+5]['FINALIDADE'], 
            LOCAL_DEMAIS_TRAB = demais_trabalhos[(i*6)+5]['LOCAL'])
            lista_demais_trabalhos.append(itens_demais_trabalhos)
            itens_demais_trabalhos = {}
            i += 1
    return(lista_demais_trabalhos)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_demais_trabalhos.
# Inclui os dicionários atualizados na lista_autores_demais_trabalhos.

def atualiza_chaves_autores_demais_trabalhos(autores_demais_trabalhos_sem_atualizar):
    autores_demais_trabalhos = []
    autores_demais_trabalhos = autores_demais_trabalhos_sem_atualizar
    qtde_autores_demais_trabalhos = len(autores_demais_trabalhos)//7
    i = 0
    autor_demais_trabalhos = {}
    lista_autores_demais_trabalhos = []

    if qtde_autores_demais_trabalhos >= 1:
        while (i < qtde_autores_demais_trabalhos):
            autor_demais_trabalhos = dict(
            ID_LATTES = autores_demais_trabalhos[i*7]['ID_LATTES'], 
            ID_AUTOR = autores_demais_trabalhos[(i*7)+1]['ID_AUTOR'], 
            ID_AUTOR_PAI = autores_demais_trabalhos[(i*7)+2]['ID_AUTOR_PAI'],
            SEQUENCIA = autores_demais_trabalhos[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_demais_trabalhos[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 3, 
            COMPETENCIA = autores_demais_trabalhos[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Produção Bibliográfica', 
            TIPO_NIVEL_2 = 'Demais Trabalhos', 
            NOME_AUTOR = autores_demais_trabalhos[(i*7)+6]['NOME-COMPLETO-DO-AUTOR'], 
            NOME_CITACAO = autores_demais_trabalhos[(i*7)+6]['NOME-PARA-CITACAO'], 
            ORDEM_AUTORIA = autores_demais_trabalhos[(i*7)+6]['ORDEM-DE-AUTORIA'])
            lista_autores_demais_trabalhos.append(autor_demais_trabalhos)
            autor_demais_trabalhos = {}
            i += 1
    return(lista_autores_demais_trabalhos)

# IV. Orientação

## 4.1. Orientação Mestrado

In [ ]:
# Coleta os dados do element ORIENTACOES-CONCLUIDAS-PARA-MESTRADO do dicionário XML Tree.
# Cria a lista orientacao_mestrado para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_orientacao_mestrado(curriculo_vitae):
    id_lattes = curriculo_vitae
    orientacao_mestrado = []
    sequencia_orient_mestrado = []
    mes = 0
    mes_competencia_orient_mestrado = ''
    competencia_orient_mestrado = ''
    id_orient_mestrado = ''

    for filho1 in root.iter('ORIENTACOES-CONCLUIDAS-PARA-MESTRADO'):
        sequencia_orient_mestrado = filho1.attrib
        orientacao_mestrado.append(id_lattes)
        orientacao_mestrado.append(sequencia_orient_mestrado)
        for filho2 in filho1.iter('DADOS-BASICOS-DE-ORIENTACOES-CONCLUIDAS-PARA-MESTRADO'):
            id_orient_mestrado = uuid.uuid4()            
            id_orient_mestrado = {'ID-ORIENT-MEST': id_orient_mestrado}
            orientacao_mestrado.append(id_orient_mestrado)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_orient_mestrado = '0' + str(mes)
            else:
                mes_competencia_orient_mestrado = str(mes)       
            # Formato Ano Mês: YYYYMM
            competencia_orient_mestrado = filho2.attrib['ANO'] + mes_competencia_orient_mestrado
            competencia_orient_mestrado = {'COMPETENCIA': competencia_orient_mestrado}
            orientacao_mestrado.append(competencia_orient_mestrado)
            orientacao_mestrado.append(filho2.attrib)
            if 'TIPO' in orientacao_mestrado:
                del orientacao_mestrado['TIPO']
            if 'HOME-PAGE' in orientacao_mestrado:
                del orientacao_mestrado['HOME-PAGE']
            if 'FLAG-RELEVANCIA' in orientacao_mestrado:
                del orientacao_mestrado['FLAG-RELEVANCIA']
            if 'TITULO-INGLES' in orientacao_mestrado:
                del orientacao_mestrado['TITULO-INGLES']
            for filho3 in filho1.iter('DETALHAMENTO-DE-ORIENTACOES-CONCLUIDAS-PARA-MESTRADO'):
                orientacao_mestrado.append(filho3.attrib)
                if 'CODIGO-ORGAO' in orientacao_mestrado:
                    del orientacao_mestrado['CODIGO-ORGAO']
                if 'NOME-ORGAO' in orientacao_mestrado:
                    del orientacao_mestrado['NOME-ORGAO']
                if 'FLAG-BOLSA' in orientacao_mestrado:
                    del orientacao_mestrado['FLAG-BOLSA']
                if 'CODIGO-AGENCIA-FINANCIADORA' in orientacao_mestrado:
                    del orientacao_mestrado['CODIGO-AGENCIA-FINANCIADORA']
                if 'NOME-DA-AGENCIA' in orientacao_mestrado:
                    del orientacao_mestrado['NOME-DA-AGENCIA']
                if 'NOME-DO-CURSO-INGLES' in orientacao_mestrado:
                    del orientacao_mestrado['NOME-DO-CURSO-INGLES']
    return(orientacao_mestrado)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_orient_mestrado.
# Inclui os dicionários atualizados na lista_orient_mestrado.

def atualiza_chaves_orientacao_mestrado(orientacao_mestrado_sem_atualizar):
    orientacao_mestrado = []
    orientacao_mestrado = orientacao_mestrado_sem_atualizar
    qtde_orient_mestrado = len(orientacao_mestrado)//6
    i = 0
    itens_orient_mestrado = {}
    lista_orient_mestrado = []

    if qtde_orient_mestrado >= 1:
        while (i < qtde_orient_mestrado):
            itens_orient_mestrado = dict(
            ID_LATTES = orientacao_mestrado[i*6]['ID_LATTES'], 
            SEQUENCIA = orientacao_mestrado[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_ORIENTACAO = orientacao_mestrado[(i*6)+2]['ID-ORIENT-MEST'],
            COMPETENCIA = orientacao_mestrado[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_ORIENTACAO = orientacao_mestrado[(i*6)+4]['NATUREZA'], 
            TITULO_ORIENTACAO = orientacao_mestrado[(i*6)+4]['TITULO'], 
            ANO_ORIENTACAO = orientacao_mestrado[(i*6)+4]['ANO'], 
            PAIS_ORIENTACAO = orientacao_mestrado[(i*6)+4]['PAIS'], 
            IDIOMA_ORIENTACAO = orientacao_mestrado[(i*6)+4]['IDIOMA'], 
            DOI_ORIENTACAO = orientacao_mestrado[(i*6)+4]['DOI'], 
            ORIENTACAO = 'Mestrado', 
            TIPO_ORIENTACAO = orientacao_mestrado[(i*6)+5]['TIPO-DE-ORIENTACAO'], 
            NOME_ORIENTADO = orientacao_mestrado[(i*6)+5]['NOME-DO-ORIENTADO'], 
            COD_INSTITUICAO_ORIENTACAO = orientacao_mestrado[(i*6)+5]['CODIGO-INSTITUICAO'], 
            NOME_INSTITUICAO_ORIENTACAO = orientacao_mestrado[(i*6)+5]['NOME-DA-INSTITUICAO'],
            COD_CURSO_ORIENTACAO = orientacao_mestrado[(i*6)+5]['CODIGO-CURSO'],
            NOME_CURSO_ORIENTACAO = orientacao_mestrado[(i*6)+5]['NOME-DO-CURSO'],
            NR_PAG_ORIENTACAO = orientacao_mestrado[(i*6)+5]['NUMERO-DE-PAGINAS'],
            ID_ORIENTADO = orientacao_mestrado[(i*6)+5]['NUMERO-ID-ORIENTADO'])
            lista_orient_mestrado.append(itens_orient_mestrado)
            itens_orient_mestrado = {}
            i += 1
    return(lista_orient_mestrado)

## 4.2. Orientação Doutorado

In [ ]:
# Coleta os dados do element ORIENTACOES-CONCLUIDAS-PARA-DOUTORADO do dicionário XML Tree.
# Cria a lista orientacao_doutorado para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_orientacao_doutorado(curriculo_vitae):
    id_lattes = curriculo_vitae
    orientacao_doutorado = []
    sequencia_orient_doutorado = []
    mes = 0
    mes_competencia_orient_doutorado = ''
    competencia_orient_doutorado = ''
    id_orient_doutorado = ''

    for filho1 in root.iter('ORIENTACOES-CONCLUIDAS-PARA-DOUTORADO'):
        #print(filho1.tag, filho1.attrib)
        orientacao_doutorado.append(id_lattes)
        sequencia_orient_doutorado = filho1.attrib
        orientacao_doutorado.append(sequencia_orient_doutorado)
        for filho2 in filho1.iter('DADOS-BASICOS-DE-ORIENTACOES-CONCLUIDAS-PARA-DOUTORADO'):
            id_orient_doutorado = uuid.uuid4()            
            id_orient_doutorado = {'ID-ORIENT-DOUT': id_orient_doutorado}
            orientacao_doutorado.append(id_orient_doutorado)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_orient_doutorado = '0' + str(mes)
            else:
                mes_competencia_orient_doutorado = str(mes)       
            # Formato Ano Mês: YYYYMM
            competencia_orient_doutorado = filho2.attrib['ANO'] + mes_competencia_orient_doutorado
            competencia_orient_doutorado = {'COMPETENCIA': competencia_orient_doutorado}
            orientacao_doutorado.append(competencia_orient_doutorado)
            orientacao_doutorado.append(filho2.attrib)
            if 'HOME-PAGE' in orientacao_doutorado:
                del orientacao_doutorado['HOME-PAGE']
            if 'FLAG-RELEVANCIA' in orientacao_doutorado:
                del orientacao_doutorado['FLAG-RELEVANCIA']
            if 'TITULO-INGLES' in orientacao_doutorado:
                del orientacao_doutorado['TITULO-INGLES']
            for filho3 in filho1.iter('DETALHAMENTO-DE-ORIENTACOES-CONCLUIDAS-PARA-DOUTORADO'):
                orientacao_doutorado.append(filho3.attrib)
                if 'CODIGO-ORGAO' in orientacao_doutorado:
                    del orientacao_doutorado['CODIGO-ORGAO']
                if 'NOME-ORGAO' in orientacao_doutorado:
                    del orientacao_doutorado['NOME-ORGAO']
                if 'FLAG-BOLSA' in orientacao_doutorado:
                    del orientacao_doutorado['FLAG-BOLSA']
                if 'CODIGO-AGENCIA-FINANCIADORA' in orientacao_doutorado:
                    del orientacao_doutorado['CODIGO-AGENCIA-FINANCIADORA']
                if 'NOME-DA-AGENCIA' in orientacao_doutorado:
                    del orientacao_doutorado['NOME-DA-AGENCIA']
                if 'NOME-DO-CURSO-INGLES' in orientacao_doutorado:
                    del orientacao_doutorado['NOME-DO-CURSO-INGLES']
    return(orientacao_doutorado)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_orient_doutorado.
# Inclui os dicionários atualizados na lista_orient_doutorado.

def atualiza_chaves_orientacao_doutorado(orientacao_doutorado_sem_atualizar):
    orientacao_doutorado = []
    orientacao_doutorado = orientacao_doutorado_sem_atualizar
    qtde_orient_doutorado = len(orientacao_doutorado)//6
    i = 0
    itens_orient_doutorado = {}
    lista_orient_doutorado = []

    if qtde_orient_doutorado >= 1:
        while (i < qtde_orient_doutorado):
            itens_orient_doutorado = dict(
            ID_LATTES = orientacao_doutorado[i*6]['ID_LATTES'], 
            SEQUENCIA = orientacao_doutorado[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_ORIENTACAO = orientacao_doutorado[(i*6)+2]['ID-ORIENT-DOUT'],
            COMPETENCIA = orientacao_doutorado[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_ORIENTACAO = orientacao_doutorado[(i*6)+4]['NATUREZA'], 
            TITULO_ORIENTACAO = orientacao_doutorado[(i*6)+4]['TITULO'], 
            ANO_ORIENTACAO = orientacao_doutorado[(i*6)+4]['ANO'], 
            PAIS_ORIENTACAO = orientacao_doutorado[(i*6)+4]['PAIS'], 
            IDIOMA_ORIENTACAO = orientacao_doutorado[(i*6)+4]['IDIOMA'], 
            DOI_ORIENTACAO = orientacao_doutorado[(i*6)+4]['DOI'], 
            ORIENTACAO = 'Doutorado', 
            TIPO_ORIENTACAO = orientacao_doutorado[(i*6)+5]['TIPO-DE-ORIENTACAO'], 
            NOME_ORIENTADO = orientacao_doutorado[(i*6)+5]['NOME-DO-ORIENTADO'], 
            COD_INSTITUICAO_ORIENTACAO = orientacao_doutorado[(i*6)+5]['CODIGO-INSTITUICAO'], 
            NOME_INSTITUICAO_ORIENTACAO = orientacao_doutorado[(i*6)+5]['NOME-DA-INSTITUICAO'],
            COD_CURSO_ORIENTACAO = orientacao_doutorado[(i*6)+5]['CODIGO-CURSO'],
            NOME_CURSO_ORIENTACAO = orientacao_doutorado[(i*6)+5]['NOME-DO-CURSO'],
            NR_PAG_ORIENTACAO = orientacao_doutorado[(i*6)+5]['NUMERO-DE-PAGINAS'],
            ID_ORIENTADO = orientacao_doutorado[(i*6)+5]['NUMERO-ID-ORIENTADO'])
            lista_orient_doutorado.append(itens_orient_doutorado)
            itens_orient_doutorado = {}
            i += 1
    return(lista_orient_doutorado)

## 4.3. Orientação Pós-Doutorado

In [ ]:
# Coleta os dados da produção bibliográfica do element 'ORIENTACOES-CONCLUIDAS-PARA-POS-DOUTORADO'
# Cria as listas para incluir os dicionários. 
# As listas são utilizadas porque os dicionários não podem ter chaves iguais.

def extrai_orientacao_pos_doutorado(curriculo_vitae):
    id_lattes = curriculo_vitae
    orientacao_pos_doutorado = []
    sequencia_orient_pos_doutorado = []
    mes = 0
    mes_competencia_orient_pos_doutorado = ''
    competencia_orient_pos_doutorado = ''
    id_orient_doutorado = ''

    for filho1 in root.iter('ORIENTACOES-CONCLUIDAS-PARA-POS-DOUTORADO'):
        #print(filho1.tag, filho1.attrib)
        orientacao_pos_doutorado.append(id_lattes)
        sequencia_orient_pos_doutorado = filho1.attrib
        orientacao_pos_doutorado.append(sequencia_orient_pos_doutorado)
        for filho2 in filho1.iter('DADOS-BASICOS-DE-ORIENTACOES-CONCLUIDAS-PARA-POS-DOUTORADO'):
            id_orient_pos_doutorado = uuid.uuid4()            
            id_orient_pos_doutorado = {'ID-ORIENT-POS-DOUT': id_orient_pos_doutorado}
            orientacao_pos_doutorado.append(id_orient_pos_doutorado)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_orient_pos_doutorado = '0' + str(mes)
            else:
                mes_competencia_orient_pos_doutorado = str(mes)       
            # Formato Ano Mês: YYYYMM
            competencia_orient_pos_doutorado = filho2.attrib['ANO'] + mes_competencia_orient_pos_doutorado
            competencia_orient_pos_doutorado = {'COMPETENCIA': competencia_orient_pos_doutorado}
            orientacao_pos_doutorado.append(competencia_orient_pos_doutorado)
            orientacao_pos_doutorado.append(filho2.attrib)
            if 'HOME-PAGE' in orientacao_pos_doutorado:
                del orientacao_pos_doutorado['HOME-PAGE']
            if 'FLAG-RELEVANCIA' in orientacao_pos_doutorado:
                del orientacao_pos_doutorado['FLAG-RELEVANCIA']
            if 'TITULO-INGLES' in orientacao_pos_doutorado:
                del orientacao_pos_doutorado['TITULO-INGLES']
            for filho3 in filho1.iter('DETALHAMENTO-DE-ORIENTACOES-CONCLUIDAS-PARA-POS-DOUTORADO'):
                orientacao_pos_doutorado.append(filho3.attrib)
                if 'CODIGO-ORGAO' in orientacao_pos_doutorado:
                    del orientacao_pos_doutorado['CODIGO-ORGAO']
                if 'NOME-ORGAO' in orientacao_pos_doutorado:
                    del orientacao_pos_doutorado['NOME-ORGAO']
                if 'FLAG-BOLSA' in orientacao_pos_doutorado:
                    del orientacao_pos_doutorado['FLAG-BOLSA']
                if 'CODIGO-AGENCIA-FINANCIADORA' in orientacao_pos_doutorado:
                    del orientacao_pos_doutorado['CODIGO-AGENCIA-FINANCIADORA']
                if 'NOME-DA-AGENCIA' in orientacao_pos_doutorado:
                    del orientacao_pos_doutorado['NOME-DA-AGENCIA']
                if 'NOME-DO-CURSO-INGLES' in orientacao_pos_doutorado:
                    del orientacao_pos_doutorado['NOME-DO-CURSO-INGLES']
    return(orientacao_pos_doutorado)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_orient_pos_dout.
# Inclui os dicionários atualizados na lista_orient_pos_dout.

def atualiza_chaves_orientacao_pos_doutorado(orientacao_pos_doutorado_sem_atualizar):
    orientacao_pos_doutorado = []
    orientacao_pos_doutorado = orientacao_pos_doutorado_sem_atualizar
    qtde_orient_pos_dout = len(orientacao_pos_doutorado)//6
    i = 0
    itens_orient_pos_dout = {}
    lista_orient_pos_dout = []

    if qtde_orient_pos_dout >= 1:
        while (i < qtde_orient_pos_dout):
            itens_orient_pos_dout = dict(
            ID_LATTES = orientacao_pos_doutorado[i*6]['ID_LATTES'], 
            SEQUENCIA = orientacao_pos_doutorado[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_ORIENTACAO = orientacao_pos_doutorado[(i*6)+2]['ID-ORIENT-POS-DOUT'],
            COMPETENCIA = orientacao_pos_doutorado[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_ORIENTACAO = orientacao_pos_doutorado[(i*6)+4]['NATUREZA'], 
            TITULO_ORIENTACAO = orientacao_pos_doutorado[(i*6)+4]['TITULO'], 
            ANO_ORIENTACAO = orientacao_pos_doutorado[(i*6)+4]['ANO'], 
            PAIS_ORIENTACAO = orientacao_pos_doutorado[(i*6)+4]['PAIS'], 
            IDIOMA_ORIENTACAO = orientacao_pos_doutorado[(i*6)+4]['IDIOMA'], 
            DOI_ORIENTACAO = orientacao_pos_doutorado[(i*6)+4]['DOI'], 
            ORIENTACAO = 'Pós-doutorado', 
            TIPO_ORIENTACAO = orientacao_pos_doutorado[(i*6)+5]['TIPO-DE-ORIENTACAO'], 
            NOME_ORIENTADO = orientacao_pos_doutorado[(i*6)+5]['NOME-DO-ORIENTADO'], 
            COD_INSTITUICAO_ORIENTACAO = orientacao_pos_doutorado[(i*6)+5]['CODIGO-INSTITUICAO'], 
            NOME_INSTITUICAO_ORIENTACAO = orientacao_pos_doutorado[(i*6)+5]['NOME-DA-INSTITUICAO'],
            COD_CURSO_ORIENTACAO = orientacao_pos_doutorado[(i*6)+5]['CODIGO-CURSO'],
            NOME_CURSO_ORIENTACAO = orientacao_pos_doutorado[(i*6)+5]['NOME-DO-CURSO'],
            NR_PAG_ORIENTACAO = orientacao_pos_doutorado[(i*6)+5]['NUMERO-DE-PAGINAS'],
            ID_ORIENTADO = orientacao_pos_doutorado[(i*6)+5]['NUMERO-ID-ORIENTADO'])
            lista_orientacoes.append(itens_orient_pos_dout)
            itens_orient_pos_dout = {}
            i += 1
    return(lista_orient_pos_dout)

## 4.4. Outra Orientação

In [ ]:
# Coleta os dados do element OUTRAS-ORIENTACOES-CONCLUIDAS do dicionário XML Tree.
# Cria a lista outra_orientacao para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_outra_orientacao(curriculo_vitae):
    id_lattes = curriculo_vitae
    outra_orientacao = []
    sequencia_outra_orientacao = []
    mes = 0
    mes_competencia_outra_orientacao = ''
    competencia_outra_orientacao = ''
    id_outra_orientacao = ''

    for filho1 in root.iter('OUTRAS-ORIENTACOES-CONCLUIDAS'):
        outra_orientacao.append(id_lattes)                 #inclui o id_lattes nos autores
        sequencia_outra_orientacao = filho1.attrib
        outra_orientacao.append(sequencia_outra_orientacao)
        for filho2 in filho1.iter('DADOS-BASICOS-DE-OUTRAS-ORIENTACOES-CONCLUIDAS'):
            id_outra_orientacao = uuid.uuid4()            
            id_outra_orientacao = {'ID-OUTRA-ORIENT': id_outra_orientacao}
            outra_orientacao.append(id_outra_orientacao)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_outra_orientacao = '0' + str(mes)
            else:
                mes_competencia_outra_orientacao = str(mes)       
            # Formato Ano Mês: YYYYMM
            competencia_outra_orientacao = filho2.attrib['ANO'] + mes_competencia_outra_orientacao
            competencia_outra_orientacao = {'COMPETENCIA': competencia_outra_orientacao}
            outra_orientacao.append(competencia_outra_orientacao) #inclui a competência na lista outra_orientacao
            outra_orientacao.append(filho2.attrib)
            if 'TIPO' in outra_orientacao:
                del outra_orientacao['TIPO']
            if 'HOME-PAGE' in outra_orientacao:
                del outra_orientacao['HOME-PAGE']
            if 'FLAG-RELEVANCIA' in outra_orientacao:
                del outra_orientacao['FLAG-RELEVANCIA']
            if 'TITULO-INGLES' in outra_orientacao:
                del outra_orientacao['TITULO-INGLES']
            if 'TIPO-INGLES' in outra_orientacao:
                del outra_orientacao['TIPO-INGLES']
            for filho3 in filho1.iter('DETALHAMENTO-DE-OUTRAS-ORIENTACOES-CONCLUIDAS'):
                outra_orientacao.append(filho3.attrib)
                #print(filho3.tag, filho3.attrib)
                if 'CODIGO-ORGAO' in outra_orientacao:
                    del outra_orientacao['CODIGO-ORGAO']
                if 'NOME-ORGAO' in outra_orientacao:
                    del outra_orientacao['NOME-ORGAO']
                if 'FLAG-BOLSA' in outra_orientacao:
                    del outra_orientacao['FLAG-BOLSA']
                if 'CODIGO-AGENCIA-FINANCIADORA' in outra_orientacao:
                    del outra_orientacao['CODIGO-AGENCIA-FINANCIADORA']
                if 'NOME-DA-AGENCIA' in outra_orientacao:
                    del outra_orientacao['NOME-DA-AGENCIA']
                if 'TIPO-DE-ORIENTACAO-CONCLUIDA' in outra_orientacao:
                    del outra_orientacao['TIPO-DE-ORIENTACAO-CONCLUIDA']
                if 'NOME-DO-CURSO-INGLES' in outra_orientacao:
                    del outra_orientacao['NOME-DO-CURSO-INGLES']
    return(outra_orientacao)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_outra_orientacao.
# Inclui os dicionários atualizados na lista_trabalhos.

def atualiza_chaves_outra_orientacao(outra_orientacao_sem_atualizar):
    outra_orientacao = []
    outra_orientacao = outra_orientacao_sem_atualizar
    qtde_outra_orientacao = len(outra_orientacao)//6
    i = 0
    itens_outra_orientacao = {}
    lista_outra_orientacao = []

    if qtde_outra_orientacao >= 1:
        while (i < qtde_outra_orientacao):
            itens_outra_orientacao = dict(
            ID_LATTES = outra_orientacao[i*6]['ID_LATTES'], 
            SEQUENCIA = outra_orientacao[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_ORIENTACAO = outra_orientacao[(i*6)+2]['ID-OUTRA-ORIENT'],
            COMPETENCIA = outra_orientacao[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_ORIENTACAO = outra_orientacao[(i*6)+4]['NATUREZA'], 
            TITULO_ORIENTACAO = outra_orientacao[(i*6)+4]['TITULO'], 
            ANO_ORIENTACAO = outra_orientacao[(i*6)+4]['ANO'], 
            PAIS_ORIENTACAO = outra_orientacao[(i*6)+4]['PAIS'], 
            IDIOMA_ORIENTACAO = outra_orientacao[(i*6)+4]['IDIOMA'], 
            DOI_ORIENTACAO = outra_orientacao[(i*6)+4]['DOI'], 
            ORIENTACAO = 'Outra orientação', 
            TIPO_ORIENTACAO = '', 
            NOME_ORIENTADO = outra_orientacao[(i*6)+5]['NOME-DO-ORIENTADO'], 
            COD_INSTITUICAO_ORIENTACAO = outra_orientacao[(i*6)+5]['CODIGO-INSTITUICAO'], 
            NOME_INSTITUICAO_ORIENTACAO = outra_orientacao[(i*6)+5]['NOME-DA-INSTITUICAO'],
            COD_CURSO_ORIENTACAO = outra_orientacao[(i*6)+5]['CODIGO-CURSO'],
            NOME_CURSO_ORIENTACAO = outra_orientacao[(i*6)+5]['NOME-DO-CURSO'],
            NR_PAG_ORIENTACAO = outra_orientacao[(i*6)+5]['NUMERO-DE-PAGINAS'],
            ID_ORIENTADO = outra_orientacao[(i*6)+5]['NUMERO-ID-ORIENTADO'])
            lista_outra_orientacao.append(itens_outra_orientacao)
            itens_outra_orientacao = {}
            i += 1
    return(lista_outra_orientacao)

# V. Participação

## 5.1. Participação em Congresso

In [ ]:
# Coleta os dados do element PARTICIPACAO-EM-CONGRESSO do dicionário XML Tree.
# Cria as listas participacao_congresso e autores_partic_congresso para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_participacao_congresso(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    participacao_congresso = []
    sequencia_partic_congresso = []
    autores_partic_congresso = []
    mes = 0
    mes_competencia_partic_congresso = ''
    competencia_partic_congresso = ''
    id_prod_biblio = ''

    for filho1 in root.iter('PARTICIPACAO-EM-CONGRESSO'):
        #print(filho1.tag, filho1.attrib)
        participacao_congresso.append(id_lattes)
        sequencia_partic_congresso = filho1.attrib
        participacao_congresso.append(sequencia_partic_congresso)
        for filho2 in filho1.iter('DADOS-BASICOS-DA-PARTICIPACAO-EM-CONGRESSO'):
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            participacao_congresso.append(id_prod_biblio)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_partic_congresso = '0' + str(mes)
            else:
                mes_competencia_partic_congresso = str(mes)       
            # Formato Ano Mês: YYYYMM
            competencia_partic_congresso = filho2.attrib['ANO'] + mes_competencia_partic_congresso
            competencia_partic_congresso = {'COMPETENCIA': competencia_partic_congresso}
            participacao_congresso.append(competencia_partic_congresso)
            participacao_congresso.append(filho2.attrib)
            if 'HOME-PAGE-DO-TRABALHO' in participacao_congresso:
                del participacao_congresso['HOME-PAGE-DO-TRABALHO']
            if 'FLAG-RELEVANCIA' in participacao_congresso:
                del participacao_congresso['FLAG-RELEVANCIA']
            if 'TITULO-INGLES' in participacao_congresso:
                del participacao_congresso['TITULO-INGLES']
            if 'FLAG-DIVULGACAO-CIENTIFICA' in participacao_congresso:
                del participacao_congresso['FLAG-DIVULGACAO-CIENTIFICA']
            for filho3 in filho1.iter('DETALHAMENTO-DA-PARTICIPACAO-EM-CONGRESSO'):
                participacao_congresso.append(filho3.attrib)
                if 'NOME-DO-EVENTO-INGLES' in participacao_congresso:
                    del participacao_congresso['NOME-DO-EVENTO-INGLES']
                for filho4 in filho1.iter('PARTICIPANTE-DE-EVENTOS-CONGRESSOS'):
                    autores_partic_congresso.append(id_lattes)
                    if filho4.attrib['NOME-COMPLETO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}                                    
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}                                   
                    autores_partic_congresso.append(id_autor)
                    autores_partic_congresso.append(id_parent_node)
                    autores_partic_congresso.append(sequencia_partic_congresso)
                    autores_partic_congresso.append(id_prod_biblio)
                    autores_partic_congresso.append(competencia_partic_congresso)
                    autores_partic_congresso.append(filho4.attrib)
                    if 'NRO-ID-CNPQ' in autores_partic_congresso:
                        del autores_partic_congresso['NRO-ID-CNPQ']
    return(participacao_congresso, autores_partic_congresso)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_partic_congresso.
# Inclui os dicionários atualizados na lista_partic_congresso.

def atualiza_chaves_participacao_congresso(participacao_congresso_sem_atualizar):
    participacao_congresso = []
    participacao_congresso = participacao_congresso_sem_atualizar
    qtde_partic_congresso = len(participacao_congresso)//6
    i = 0
    itens_partic_congresso = {}
    lista_partic_congresso = []

    if qtde_partic_congresso >= 1:
        while (i < qtde_partic_congresso):
            itens_partic_congresso = dict(
            ID_LATTES = participacao_congresso[i*6]['ID_LATTES'], 
            SEQUENCIA = participacao_congresso[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = participacao_congresso[(i*6)+2]['ID-PROD-BIBLIO'],
            COD_NIVEL = 9, 
            COMPETENCIA = participacao_congresso[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_PARTIC = participacao_congresso[(i*6)+4]['NATUREZA'], 
            TITULO_PARTIC = participacao_congresso[(i*6)+4]['TITULO'], 
            ANO_PARTIC = participacao_congresso[(i*6)+4]['ANO'], 
            PAIS_PARTIC = participacao_congresso[(i*6)+4]['PAIS'], 
            IDIOMA_PARTIC = participacao_congresso[(i*6)+4]['IDIOMA'], 
            MEIO_DIVULGACAO_PARTIC = participacao_congresso[(i*6)+4]['MEIO-DE-DIVULGACAO'], 
            TIPO_PARTICIPACAO = participacao_congresso[(i*6)+4]['TIPO-PARTICIPACAO'], 
            FORMA_PARTICIPACAO = participacao_congresso[(i*6)+4]['FORMA-PARTICIPACAO'], 
            DOI_PARTIC = participacao_congresso[(i*6)+4]['DOI'], 
            TIPO_PARTIC = 'Congresso', 
            NOME_EVENTO_PARTIC = participacao_congresso[(i*6)+5]['NOME-DO-EVENTO'], 
            COD_INSTITUICAO_PARTIC = participacao_congresso[(i*6)+5]['CODIGO-INSTITUICAO'], 
            NOME_INSTITUICAO_PARTIC = participacao_congresso[(i*6)+5]['NOME-INSTITUICAO'], 
            LOCAL_EVENTO_PARTIC = participacao_congresso[(i*6)+5]['LOCAL-DO-EVENTO'], 
            CIDADE_EVENTO_PARTIC = participacao_congresso[(i*6)+5]['CIDADE-DO-EVENTO'])
            lista_partic_congresso.append(itens_partic_congresso)
            itens_partic_congresso = {}
            i += 1
    return(lista_partic_congresso)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_partic_congresso.
# Inclui os dicionários atualizados na lista_autores_partic_congresso.

def atualiza_chaves_autores_participacao_congresso(autores_participacao_congresso_sem_atualizar):
    autores_partic_congresso = []
    autores_partic_congresso = autores_participacao_congresso_sem_atualizar
    qtde_autores_partic_congresso = len(autores_partic_congresso)//7
    i = 0
    autor_partic_congresso = {}
    lista_autores_partic_congresso = []

    if qtde_autores_partic_congresso >= 1:
        while (i < qtde_autores_partic_congresso):
            autor_partic_congresso = dict(
            ID_LATTES = autores_partic_congresso[i*7]['ID_LATTES'], 
            ID_AUTOR = autores_partic_congresso[(i*7)+1]['ID_AUTOR'],
            ID_AUTOR_PAI = autores_partic_congresso[(i*7)+2]['ID_AUTOR_PAI'],
            SEQUENCIA = autores_partic_congresso[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_partic_congresso[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 9, 
            COMPETENCIA = autores_partic_congresso[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Participação', 
            TIPO_NIVEL_2 = 'Participação em Congresso', 
            NOME_AUTOR = autores_partic_congresso[(i*7)+6]['NOME-COMPLETO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'], 
            NOME_CITACAO = autores_partic_congresso[(i*7)+6]['NOME-PARA-CITACAO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'], 
            ORDEM_AUTORIA = autores_partic_congresso[(i*7)+6]['ORDEM-PARTICIPANTE'])
            lista_autores_partic_congresso.append(autor_partic_congresso)
            autor_partic_congresso = {}
            i += 1
    return(lista_autores_partic_congresso)

## 5.2. Participação em Feira

In [ ]:
# Coleta os dados do element PARTICIPACAO-EM-FEIRA do dicionário XML Tree.
# Cria as listas participacao_feira e autores_partic_feira para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_participacao_feira(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    participacao_feira = []
    sequencia_partic_feira = []
    autores_partic_feira = []
    mes = 0
    mes_competencia_partic_feira = '' 
    competencia_partic_feira = ''
    id_prod_biblio = ''

    for filho1 in root.iter('PARTICIPACAO-EM-FEIRA'):
        sequencia_partic_feira = filho1.attrib
        participacao_feira.append(id_lattes)
        participacao_feira.append(sequencia_partic_feira)
        for filho2 in filho1.iter('DADOS-BASICOS-DA-PARTICIPACAO-EM-FEIRA'):
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            participacao_feira.append(id_prod_biblio)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_partic_feira = '0' + str(mes)
            else:
                mes_competencia_partic_feira = str(mes)       
            # Formato Ano Mês: YYYYMM
            competencia_partic_feira = filho2.attrib['ANO'] + mes_competencia_partic_feira
            competencia_partic_feira = {'COMPETENCIA': competencia_partic_feira}
            participacao_feira.append(competencia_partic_feira)
            participacao_feira.append(filho2.attrib)
            if 'HOME-PAGE-DO-TRABALHO' in participacao_feira:
                del participacao_feira['HOME-PAGE-DO-TRABALHO']
            if 'FLAG-RELEVANCIA' in participacao_feira:
                del participacao_feira['FLAG-RELEVANCIA']
            if 'TITULO-INGLES' in participacao_feira:
                del participacao_feira['TITULO-INGLES']
            if 'FLAG-DIVULGACAO-CIENTIFICA' in participacao_feira:
                del participacao_feira['FLAG-DIVULGACAO-CIENTIFICA']
            for filho3 in filho1.iter('DETALHAMENTO-DA-PARTICIPACAO-EM-FEIRA'):
                participacao_feira.append(filho3.attrib)
                if 'NOME-DO-EVENTO-INGLES' in participacao_feira:
                    del participacao_feira['NOME-DO-EVENTO-INGLES']
                # No XML está 'PARTICIPANTE-DE-EVENTOS-CONGRESSOS' para todos os tipos de participações.
                for filho4 in filho1.iter('PARTICIPANTE-DE-EVENTOS-CONGRESSOS'):
                    autores_partic_feira.append(id_lattes)
                    if filho4.attrib['NOME-COMPLETO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}                                    
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}                                   
                    autores_partic_feira.append(id_autor)
                    autores_partic_feira.append(id_parent_node)
                    autores_partic_feira.append(sequencia_partic_feira)
                    autores_partic_feira.append(id_prod_biblio)
                    autores_partic_feira.append(competencia_partic_feira)
                    autores_partic_feira.append(filho4.attrib)
                    if 'NRO-ID-CNPQ' in autores_partic_feira:
                        del autores_partic_feira['NRO-ID-CNPQ']
    return(participacao_feira, autores_partic_feira)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_partic_feira.
# Inclui os dicionários atualizados na lista_partic_feira.

def atualiza_chaves_participacao_feira(participacao_feira_sem_atualizar):
    participacao_feira = []
    participacao_feira = participacao_feira_sem_atualizar
    qtde_partic_feira = len(participacao_feira)//6
    i = 0
    itens_partic_feira = {}
    lista_partic_feira = []

    if qtde_partic_feira >= 1:
        while (i < qtde_partic_feira):
            itens_partic_feira = dict(
            ID_LATTES = participacao_feira[i*6]['ID_LATTES'], 
            SEQUENCIA = participacao_feira[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = participacao_feira[(i*6)+2]['ID-PROD-BIBLIO'],            
            COD_NIVEL = 10, 
            COMPETENCIA = participacao_feira[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_PARTIC = participacao_feira[(i*6)+4]['NATUREZA'], 
            TITULO_PARTIC = participacao_feira[(i*6)+4]['TITULO'], 
            ANO_PARTIC = participacao_feira[(i*6)+4]['ANO'], 
            PAIS_PARTIC = participacao_feira[(i*6)+4]['PAIS'], 
            IDIOMA_PARTIC = participacao_feira[(i*6)+4]['IDIOMA'], 
            MEIO_DIVULGACAO_PARTIC = participacao_feira[(i*6)+4]['MEIO-DE-DIVULGACAO'], 
            TIPO_PARTICIPACAO = participacao_feira[(i*6)+4]['TIPO-PARTICIPACAO'], 
            FORMA_PARTICIPACAO = participacao_feira[(i*6)+4]['FORMA-PARTICIPACAO'], 
            DOI_PARTIC = participacao_feira[(i*6)+4]['DOI'], 
            TIPO_PARTIC = 'Feira', 
            NOME_EVENTO_PARTIC = participacao_feira[(i*6)+5]['NOME-DO-EVENTO'], 
            COD_INSTITUICAO_PARTIC = participacao_feira[(i*6)+5]['CODIGO-INSTITUICAO'], 
            NOME_INSTITUICAO_PARTIC = participacao_feira[(i*6)+5]['NOME-INSTITUICAO'], 
            LOCAL_EVENTO_PARTIC = participacao_feira[(i*6)+5]['LOCAL-DO-EVENTO'], 
            CIDADE_EVENTO_PARTIC = participacao_feira[(i*6)+5]['CIDADE-DO-EVENTO'])
            lista_partic_feira.append(itens_partic_feira)
            itens_partic_feira = {}
            i += 1
    return(lista_partic_feira)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_partic_feira.
# Inclui os dicionários atualizados na lista_autores_partic_feira.

def atualiza_chaves_autores_participacao_feira(autores_participacao_feira_sem_atualizar):
    autores_partic_feira = []
    autores_partic_feira = autores_participacao_feira_sem_atualizar
    qtde_autores_partic_feira = len(autores_partic_feira)//7
    i = 0
    autor_partic_feira = {}
    lista_autores_partic_feira = []

    if qtde_autores_partic_feira >= 1:
        while (i < qtde_autores_partic_feira):
            autor_partic_feira = dict(
            ID_LATTES = autores_partic_feira[i*7]['ID_LATTES'], 
            ID_AUTOR = autores_partic_feira[(i*7)+1]['ID_AUTOR'],
            ID_AUTOR_PAI = autores_partic_feira[(i*7)+2]['ID_AUTOR_PAI'],
            SEQUENCIA = autores_partic_feira[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_partic_feira[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 10, 
            COMPETENCIA = autores_partic_feira[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Participação', 
            TIPO_NIVEL_2 = 'Participação em Feira', 
            # No XML está 'CONGRESSOS' para todos os tipos de participações.
            NOME_AUTOR = autores_partic_feira[(i*7)+6]['NOME-COMPLETO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'], 
            NOME_CITACAO = autores_partic_feira[(i*7)+6]['NOME-PARA-CITACAO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'], 
            ORDEM_AUTORIA = autores_partic_feira[(i*7)+6]['ORDEM-PARTICIPANTE'])
            lista_autores_partic_feira.append(autor_partic_feira)
            autor_partic_feira = {}
            i += 1
    return(lista_autores_partic_feira)

## 5.3. Participação em Seminário

In [ ]:
# Coleta os dados do element PARTICIPACAO-EM-SEMINARIO do dicionário XML Tree.
# Cria as listas participacao_seminario e autores_partic_seminario para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_participacao_seminario(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    participacao_seminario = []
    sequencia_partic_seminario = []
    autores_partic_seminario = []
    mes = 0
    mes_competencia_partic_seminario = ''
    competencia_partic_seminario = ''
    id_prod_biblio = ''

    for filho1 in root.iter('PARTICIPACAO-EM-SEMINARIO'):
        sequencia_partic_seminario = filho1.attrib
        participacao_seminario.append(id_lattes)
        participacao_seminario.append(sequencia_partic_seminario)
        for filho2 in filho1.iter('DADOS-BASICOS-DA-PARTICIPACAO-EM-SEMINARIO'):
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            participacao_seminario.append(id_prod_biblio)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_partic_seminario = '0' + str(mes)
            else:
                mes_competencia_partic_seminario = str(mes)       
            # Formato Ano Mês: YYYYMM
            competencia_partic_seminario = filho2.attrib['ANO'] + mes_competencia_partic_seminario
            competencia_partic_seminario = {'COMPETENCIA': competencia_partic_seminario}
            participacao_seminario.append(competencia_partic_seminario)
            participacao_seminario.append(filho2.attrib)
            if 'HOME-PAGE-DO-TRABALHO' in participacao_seminario:
                del participacao_seminario['HOME-PAGE-DO-TRABALHO']
            if 'FLAG-RELEVANCIA' in participacao_seminario:
                del participacao_seminario['FLAG-RELEVANCIA']
            if 'TITULO-INGLES' in participacao_seminario:
                del participacao_seminario['TITULO-INGLES'] 
            if 'FLAG-DIVULGACAO-CIENTIFICA' in participacao_seminario:
                del participacao_seminario['FLAG-DIVULGACAO-CIENTIFICA']
            for filho3 in filho1.iter('DETALHAMENTO-DA-PARTICIPACAO-EM-SEMINARIO'):
                participacao_seminario.append(filho3.attrib)
                if 'NOME-DO-EVENTO-INGLES' in participacao_seminario:
                    del participacao_seminario['NOME-DO-EVENTO-INGLES']
                # No XML está 'PARTICIPANTE-DE-EVENTOS-CONGRESSOS' para todos os tipos de participações.
                for filho4 in filho1.iter('PARTICIPANTE-DE-EVENTOS-CONGRESSOS'):
                    autores_partic_seminario.append(id_lattes)
                    if filho4.attrib['NOME-COMPLETO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}                                    
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}                                   
                    autores_partic_seminario.append(id_autor)
                    autores_partic_seminario.append(id_parent_node)
                    autores_partic_seminario.append(sequencia_partic_seminario)
                    autores_partic_seminario.append(id_prod_biblio)
                    autores_partic_seminario.append(competencia_partic_seminario)
                    autores_partic_seminario.append(filho4.attrib)
                    if 'NRO-ID-CNPQ' in autores_partic_seminario:
                        del autores_partic_seminario['NRO-ID-CNPQ'] 
    return(participacao_seminario, autores_partic_seminario)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_partic_seminario.
# Inclui os dicionários atualizados na lista_partic_seminario.

def atualiza_chaves_participacao_seminario(participacao_seminario_sem_atualizar):
    participacao_seminario = []
    participacao_seminario = participacao_seminario_sem_atualizar
    qtde_partic_seminario = len(participacao_seminario)//6
    i = 0
    itens_partic_seminario = {}
    lista_partic_seminario = []

    if qtde_partic_seminario >= 1:
        while (i < qtde_partic_seminario):
            itens_partic_seminario = dict(
            ID_LATTES = participacao_seminario[i*6]['ID_LATTES'], 
            SEQUENCIA = participacao_seminario[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = participacao_seminario[(i*6)+2]['ID-PROD-BIBLIO'],
            COD_NIVEL = 11, 
            COMPETENCIA = participacao_seminario[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_PARTIC = participacao_seminario[(i*6)+4]['NATUREZA'], 
            TITULO_PARTIC = participacao_seminario[(i*6)+4]['TITULO'], 
            ANO_PARTIC = participacao_seminario[(i*6)+4]['ANO'], 
            PAIS_PARTIC = participacao_seminario[(i*6)+4]['PAIS'], 
            IDIOMA_PARTIC = participacao_seminario[(i*6)+4]['IDIOMA'], 
            MEIO_DIVULGACAO_PARTIC = participacao_seminario[(i*6)+4]['MEIO-DE-DIVULGACAO'], 
            TIPO_PARTICIPACAO = participacao_seminario[(i*6)+4]['TIPO-PARTICIPACAO'], 
            FORMA_PARTICIPACAO = participacao_seminario[(i*6)+4]['FORMA-PARTICIPACAO'], 
            DOI_PARTIC = participacao_seminario[(i*6)+4]['DOI'], 
            TIPO_PARTIC = 'Seminário', 
            NOME_EVENTO_PARTIC = participacao_seminario[(i*6)+5]['NOME-DO-EVENTO'], 
            COD_INSTITUICAO_PARTIC = participacao_seminario[(i*6)+5]['CODIGO-INSTITUICAO'], 
            NOME_INSTITUICAO_PARTIC = participacao_seminario[(i*6)+5]['NOME-INSTITUICAO'], 
            LOCAL_EVENTO_PARTIC = participacao_seminario[(i*6)+5]['LOCAL-DO-EVENTO'], 
            CIDADE_EVENTO_PARTIC = participacao_seminario[(i*6)+5]['CIDADE-DO-EVENTO'])
            lista_partic_seminario.append(itens_partic_seminario)
            itens_partic_seminario = {}
            i += 1
    return(lista_partic_seminario)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_partic_seminario.
# Inclui os dicionários atualizados na lista_autores_partic_seminario.

def atualiza_chaves_autores_participacao_seminario(autores_participacao_seminario_sem_atualizar):
    autores_partic_seminario = []
    autores_partic_seminario = autores_participacao_seminario_sem_atualizar
    qtde_autores_partic_seminario = len(autores_partic_seminario)//7
    i = 0
    autor_partic_seminario = {}
    lista_autores_partic_seminario = []

    if qtde_autores_partic_seminario >= 1:
        while (i < qtde_autores_partic_seminario):
            autor_partic_seminario = dict(
            ID_LATTES = autores_partic_seminario[i*7]['ID_LATTES'], 
            ID_AUTOR = autores_partic_seminario[(i*7)+1]['ID_AUTOR'],
            ID_AUTOR_PAI = autores_partic_seminario[(i*7)+2]['ID_AUTOR_PAI'],
            SEQUENCIA = autores_partic_seminario[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_partic_seminario[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 11, 
            COMPETENCIA = autores_partic_seminario[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Participação', 
            TIPO_NIVEL_2 = 'Participação em Seminário', 
            # No XML está 'CONGRESSOS' para todos os tipos de participações.
            NOME_AUTOR = autores_partic_seminario[(i*7)+6]['NOME-COMPLETO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'], 
            NOME_CITACAO = autores_partic_seminario[(i*7)+6]['NOME-PARA-CITACAO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'], 
            ORDEM_AUTORIA = autores_partic_seminario[(i*7)+6]['ORDEM-PARTICIPANTE'])
            lista_autores_partic_seminario.append(autor_partic_seminario)
            autor_partic_seminario = {}
            i += 1
    return(lista_autores_partic_seminario)

## 5.4. Participação em Simpósio

In [ ]:
# Coleta os dados do element PARTICIPACAO-EM-SIMPOSIO do dicionário XML Tree.
# Cria as listas participacao_simposio e autores_partic_simposio para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_participacao_simposio(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    participacao_simposio = []
    sequencia_partic_simposio = []
    autores_partic_simposio = []
    mes = 0
    mes_competencia_partic_simposio = ''
    competencia_partic_simposio = ''
    id_prod_biblio = ''

    for filho1 in root.iter('PARTICIPACAO-EM-SIMPOSIO'):
        sequencia_partic_simposio = filho1.attrib
        participacao_simposio.append(id_lattes)
        participacao_simposio.append(sequencia_partic_simposio)
        for filho2 in filho1.iter('DADOS-BASICOS-DA-PARTICIPACAO-EM-SIMPOSIO'):
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            participacao_simposio.append(id_prod_biblio)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_partic_simposio = '0' + str(mes)
            else:
                mes_competencia_partic_simposio = str(mes)       
            # Formato Ano Mês: YYYYMM
            competencia_partic_simposio = filho2.attrib['ANO'] + mes_competencia_partic_simposio
            competencia_partic_simposio = {'COMPETENCIA': competencia_partic_simposio}
            participacao_simposio.append(competencia_partic_simposio)
            participacao_simposio.append(filho2.attrib)
            if 'HOME-PAGE-DO-TRABALHO' in participacao_simposio:
                del participacao_simposio['HOME-PAGE-DO-TRABALHO']
            if 'FLAG-RELEVANCIA' in participacao_simposio:
                del participacao_simposio['FLAG-RELEVANCIA']
            if 'TITULO-INGLES' in participacao_simposio:
                del participacao_simposio['TITULO-INGLES']
            if 'FLAG-DIVULGACAO-CIENTIFICA' in participacao_simposio:
                del participacao_simposio['FLAG-DIVULGACAO-CIENTIFICA']
            for filho3 in filho1.iter('DETALHAMENTO-DA-PARTICIPACAO-EM-SIMPOSIO'):
                participacao_simposio.append(filho3.attrib)
                if 'NOME-DO-EVENTO-INGLES' in participacao_simposio:
                    del participacao_simposio['NOME-DO-EVENTO-INGLES']
                # No XML está 'PARTICIPANTE-DE-EVENTOS-CONGRESSOS' para todos os tipos de participações.
                for filho4 in filho1.iter('PARTICIPANTE-DE-EVENTOS-CONGRESSOS'):
                    autores_partic_simposio.append(id_lattes)
                    if filho4.attrib['NOME-COMPLETO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}                                    
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}                                   
                    autores_partic_simposio.append(id_autor)
                    autores_partic_simposio.append(id_parent_node)
                    autores_partic_simposio.append(sequencia_partic_simposio)
                    autores_partic_simposio.append(id_prod_biblio)
                    autores_partic_simposio.append(competencia_partic_simposio)
                    autores_partic_simposio.append(filho4.attrib)
                    if 'NRO-ID-CNPQ' in autores_partic_simposio:
                        del autores_partic_simposio['NRO-ID-CNPQ']
    return(participacao_simposio, autores_partic_simposio)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_partic_simposio.
# Inclui os dicionários atualizados na lista_partic_simposio.

def atualiza_chaves_participacao_simposio(participacao_simposio_sem_atualizar):
    participacao_simposio = []
    participacao_simposio = participacao_simposio_sem_atualizar
    qtde_partic_simposio = len(participacao_simposio)//6
    i = 0
    itens_partic_simposio = {}
    lista_partic_simposio = []

    if qtde_partic_simposio >= 1:
        while (i < qtde_partic_simposio):
            itens_partic_simposio = dict(
            ID_LATTES = participacao_simposio[i*6]['ID_LATTES'], 
            SEQUENCIA = participacao_simposio[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = participacao_simposio[(i*6)+2]['ID-PROD-BIBLIO'],
            COD_NIVEL = 12, 
            COMPETENCIA = participacao_simposio[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_PARTIC = participacao_simposio[(i*6)+4]['NATUREZA'], 
            TITULO_PARTIC = participacao_simposio[(i*6)+4]['TITULO'], 
            ANO_PARTIC = participacao_simposio[(i*6)+4]['ANO'], 
            PAIS_PARTIC = participacao_simposio[(i*6)+4]['PAIS'], 
            IDIOMA_PARTIC = participacao_simposio[(i*6)+4]['IDIOMA'], 
            MEIO_DIVULGACAO_PARTIC = participacao_simposio[(i*6)+4]['MEIO-DE-DIVULGACAO'], 
            TIPO_PARTICIPACAO = participacao_simposio[(i*6)+4]['TIPO-PARTICIPACAO'], 
            FORMA_PARTICIPACAO = participacao_simposio[(i*6)+4]['FORMA-PARTICIPACAO'], 
            DOI_PARTIC = participacao_simposio[(i*6)+4]['DOI'], 
            TIPO_PARTIC = 'Simpósio', 
            NOME_EVENTO_PARTIC = participacao_simposio[(i*6)+5]['NOME-DO-EVENTO'], 
            COD_INSTITUICAO_PARTIC = participacao_simposio[(i*6)+5]['CODIGO-INSTITUICAO'], 
            NOME_INSTITUICAO_PARTIC = participacao_simposio[(i*6)+5]['NOME-INSTITUICAO'], 
            LOCAL_EVENTO_PARTIC = participacao_simposio[(i*6)+5]['LOCAL-DO-EVENTO'], 
            CIDADE_EVENTO_PARTIC = participacao_simposio[(i*6)+5]['CIDADE-DO-EVENTO'])
            lista_partic_simposio.append(itens_partic_simposio)
            itens_partic_simposio = {}
            i += 1
    return(lista_partic_simposio)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_partic_simposio.
# Inclui os dicionários atualizados na lista_autores_partic_simposio.

def atualiza_chaves_autores_participacao_simposio(autores_participacao_simposio_sem_atualizar):
    autores_partic_simposio = []
    autores_partic_simposio = autores_participacao_simposio_sem_atualizar
    qtde_autores_partic_simposio = len(autores_partic_simposio)//7
    i = 0
    autor_partic_simposio = {}
    lista_autores_partic_simposio = []

    if qtde_autores_partic_simposio >= 1:
        while (i < qtde_autores_partic_simposio):
            autor_partic_simposio = dict(
            ID_LATTES = autores_partic_simposio[i*7]['ID_LATTES'], 
            ID_AUTOR = autores_partic_simposio[(i*7)+1]['ID_AUTOR'],
            ID_AUTOR_PAI = autores_partic_simposio[(i*7)+2]['ID_AUTOR_PAI'],
            SEQUENCIA = autores_partic_simposio[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_partic_simposio[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 12, 
            COMPETENCIA = autores_partic_simposio[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Participação', 
            TIPO_NIVEL_2 = 'Participação em Simpósio', 
            # No XML está 'CONGRESSOS' para todos os tipos de participações.
            NOME_AUTOR = autores_partic_simposio[(i*7)+6]['NOME-COMPLETO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'], 
            NOME_CITACAO = autores_partic_simposio[(i*7)+6]['NOME-PARA-CITACAO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'], 
            ORDEM_AUTORIA = autores_partic_simposio[(i*7)+6]['ORDEM-PARTICIPANTE'])
            lista_autores_partic_simposio.append(autor_partic_simposio)
            autor_partic_simposio = {}
            i += 1
    return(lista_autores_partic_simposio)

## 5.5. Participação em Oficina

In [ ]:
# Coleta os dados do element PARTICIPACAO-EM-OFICINA do dicionário XML Tree.
# Cria as listas participacao_oficina e autores_partic_oficina para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_participacao_oficina(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    participacao_oficina = []
    sequencia_partic_oficina = []
    autores_partic_oficina = []
    mes = 0
    mes_competencia_partic_oficina = ''
    competencia_partic_oficina = ''
    id_prod_biblio = ''

    for filho1 in root.iter('PARTICIPACAO-EM-OFICINA'):
        sequencia_partic_oficina = filho1.attrib
        participacao_oficina.append(id_lattes)
        participacao_oficina.append(sequencia_partic_oficina)
        for filho2 in filho1.iter('DADOS-BASICOS-DA-PARTICIPACAO-EM-OFICINA'):
            mes = randrange(1, 12)
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            participacao_oficina.append(id_prod_biblio)            
            if mes < 10:
                mes_competencia_partic_oficina = '0' + str(mes)
            else:
                mes_competencia_partic_oficina = str(mes)       
            # Formato Ano Mês: YYYYMM
            competencia_partic_oficina = filho2.attrib['ANO'] + mes_competencia_partic_oficina
            competencia_partic_oficina = {'COMPETENCIA': competencia_partic_oficina}
            participacao_oficina.append(competencia_partic_oficina)
            participacao_oficina.append(filho2.attrib)
            if 'HOME-PAGE-DO-TRABALHO' in participacao_oficina:
                del participacao_oficina['HOME-PAGE-DO-TRABALHO']
            if 'FLAG-RELEVANCIA' in participacao_oficina:
                del participacao_oficina['FLAG-RELEVANCIA']
            if 'TITULO-INGLES' in participacao_oficina:
                del participacao_oficina['TITULO-INGLES']
            if 'FLAG-DIVULGACAO-CIENTIFICA' in participacao_oficina:
                del participacao_oficina['FLAG-DIVULGACAO-CIENTIFICA']
            for filho3 in filho1.iter('DETALHAMENTO-DA-PARTICIPACAO-EM-OFICINA'):
                participacao_oficina.append(filho3.attrib)
                if 'NOME-DO-EVENTO-INGLES' in participacao_oficina:
                    del participacao_oficina['NOME-DO-EVENTO-INGLES']
                # No XML está 'PARTICIPANTE-DE-EVENTOS-CONGRESSOS' para todos os tipos de participações.
                for filho4 in filho1.iter('PARTICIPANTE-DE-EVENTOS-CONGRESSOS'):
                    autores_partic_oficina.append(id_lattes)
                    if filho4.attrib['NOME-COMPLETO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}                                    
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}                                   
                    autores_partic_oficina.append(id_autor)
                    autores_partic_oficina.append(id_parent_node)
                    autores_partic_oficina.append(sequencia_partic_oficina)
                    autores_partic_oficina.append(id_prod_biblio)
                    autores_partic_oficina.append(competencia_partic_oficina)
                    autores_partic_oficina.append(filho4.attrib)
                    if 'NRO-ID-CNPQ' in autores_partic_oficina:
                        del autores_partic_oficina['NRO-ID-CNPQ']
    return(participacao_oficina, autores_partic_oficina)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_partic_oficina.
# Inclui os dicionários atualizados na lista_partic_oficina.

def atualiza_chaves_participacao_oficina(participacao_oficina_sem_atualizar):
    participacao_oficina = []
    participacao_oficina = participacao_oficina_sem_atualizar
    qtde_partic_oficina = len(participacao_oficina)//6
    i = 0
    itens_partic_oficina = {}
    lista_partic_oficina = []

    if qtde_partic_oficina >= 1:
        while (i < qtde_partic_oficina):
            itens_partic_oficina = dict(
            ID_LATTES = participacao_oficina[i*6]['ID_LATTES'], 
            SEQUENCIA = participacao_oficina[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = trabalhos[(i*6)+2]['ID-PROD-BIBLIO'],
            COD_NIVEL = 13, 
            COMPETENCIA = participacao_oficina[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_PARTIC = participacao_oficina[(i*6)+4]['NATUREZA'], 
            TITULO_PARTIC = participacao_oficina[(i*6)+4]['TITULO'], 
            ANO_PARTIC = participacao_oficina[(i*6)+4]['ANO'], 
            PAIS_PARTIC = participacao_oficina[(i*6)+4]['PAIS'], 
            IDIOMA_PARTIC = participacao_oficina[(i*6)+4]['IDIOMA'], 
            MEIO_DIVULGACAO_PARTIC = participacao_oficina[(i*6)+4]['MEIO-DE-DIVULGACAO'], 
            TIPO_PARTICIPACAO = participacao_oficina[(i*6)+4]['TIPO-PARTICIPACAO'], 
            FORMA_PARTICIPACAO = participacao_oficina[(i*6)+4]['FORMA-PARTICIPACAO'], 
            DOI_PARTIC = participacao_oficina[(i*6)+4]['DOI'], 
            TIPO_PARTIC = 'Oficina', 
            NOME_EVENTO_PARTIC = participacao_oficina[(i*6)+5]['NOME-DO-EVENTO'], 
            COD_INSTITUICAO_PARTIC = participacao_oficina[(i*6)+5]['CODIGO-INSTITUICAO'], 
            NOME_INSTITUICAO_PARTIC = participacao_oficina[(i*6)+5]['NOME-INSTITUICAO'], 
            LOCAL_EVENTO_PARTIC = participacao_oficina[(i*6)+5]['LOCAL-DO-EVENTO'], 
            CIDADE_EVENTO_PARTIC = participacao_oficina[(i*6)+5]['CIDADE-DO-EVENTO'])
            lista_partic_oficina.append(itens_partic_oficina)
            itens_partic_oficina = {}
            i += 1
    return(lista_partic_oficina)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_partic_oficina.
# Inclui os dicionários atualizados na lista_autores_partic_oficina.

def atualiza_chaves_autores_participacao_oficina(autores_participacao_oficina_sem_atualizar):
    autores_partic_oficina = []
    autores_partic_oficina = autores_participacao_oficina_sem_atualizar
    qtde_autores_partic_oficina = len(autores_partic_oficina)//7
    i = 0
    autor_partic_oficina = {}
    lista_autores_partic_oficina = []

    if qtde_autores_partic_oficina >= 1:
        while (i < qtde_autores_partic_oficina):
            autor_partic_oficina = dict(
            ID_LATTES = autores_partic_oficina[i*7]['ID_LATTES'], 
            ID_AUTOR = autores_partic_oficina[(i*7)+1]['ID_AUTOR'],
            ID_AUTOR_PAI = autores_partic_oficina[(i*7)+2]['ID_AUTOR_PAI'],
            SEQUENCIA = autores_partic_oficina[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_partic_oficina[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 13, 
            COMPETENCIA = autores_partic_oficina[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Participação', 
            TIPO_NIVEL_2 = 'Participação em Oficina', 
            # No XML está 'CONGRESSOS' para todos os tipos de participações.
            NOME_AUTOR = autores_partic_oficina[(i*7)+6]['NOME-COMPLETO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'], 
            NOME_CITACAO = autores_partic_oficina[(i*7)+6]['NOME-PARA-CITACAO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'], 
            ORDEM_AUTORIA = autores_partic_oficina[(i*7)+6]['ORDEM-PARTICIPANTE'])
            lista_autores_partic_oficina.append(autor_partic_oficina)
            autor_partic_oficina = {}
            i += 1
    return(lista_autores_partic_oficina)

## 5.6. Outra Participação

In [ ]:
# Coleta os dados do element OUTRAS-PARTICIPACOES-EM-EVENTOS-CONGRESSOS do dicionário XML Tree.
# Cria as listas outra_participacao e autores_outra_participacao para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_outra_participacao(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    outra_participacao = []
    sequencia_outra_participacao = []
    autores_outra_participacao = []
    mes = 0
    mes_competencia_outra_participacao = ''
    competencia_outra_participacao = ''
    id_prod_biblio = ''

    for filho1 in root.iter('OUTRAS-PARTICIPACOES-EM-EVENTOS-CONGRESSOS'):
        outra_participacao.append(id_lattes)
        sequencia_outra_participacao = filho1.attrib
        outra_participacao.append(sequencia_outra_participacao)
        for filho2 in filho1.iter('DADOS-BASICOS-DE-OUTRAS-PARTICIPACOES-EM-EVENTOS-CONGRESSOS'):
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            outra_participacao.append(id_prod_biblio)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_outra_participacao = '0' + str(mes)
            else:
                mes_competencia_outra_participacao = str(mes)       
            # Formato Ano Mês: YYYYMM
            competencia_outra_participacao = filho2.attrib['ANO'] + mes_competencia_outra_participacao
            competencia_outra_participacao = {'COMPETENCIA': competencia_outra_participacao}
            outra_participacao.append(competencia_outra_participacao)
            outra_participacao.append(filho2.attrib)
            if 'HOME-PAGE-DO-TRABALHO' in outra_participacao:
                del outra_participacao['HOME-PAGE-DO-TRABALHO']
            if 'FLAG-RELEVANCIA' in outra_participacao:
                del outra_participacao['FLAG-RELEVANCIA']
            if 'TITULO-INGLES' in outra_participacao:
                del outra_participacao['TITULO-INGLES']
            if 'FLAG-DIVULGACAO-CIENTIFICA' in outra_participacao:
                del outra_participacao['FLAG-DIVULGACAO-CIENTIFICA']
            for filho3 in filho1.iter('DETALHAMENTO-DE-OUTRAS-PARTICIPACOES-EM-EVENTOS-CONGRESSOS'):
                outra_participacao.append(filho3.attrib)
                if 'NOME-DO-EVENTO-INGLES' in outra_participacao:
                    del outra_participacao['NOME-DO-EVENTO-INGLES']
                # No XML está 'PARTICIPANTE-DE-EVENTOS-CONGRESSOS' para todos os tipos de participações.
                for filho4 in filho1.iter('PARTICIPANTE-DE-EVENTOS-CONGRESSOS'):
                    autores_outra_participacao.append(id_lattes)
                    if filho4.attrib['NOME-COMPLETO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}                                    
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}                                   
                    autores_outra_participacao.append(id_autor)
                    autores_outra_participacao.append(id_parent_node)
                    autores_outra_participacao.append(sequencia_outra_participacao)
                    autores_outra_participacao.append(id_prod_biblio)
                    autores_outra_participacao.append(competencia_outra_participacao)
                    autores_outra_participacao.append(filho4.attrib)
                    if 'NRO-ID-CNPQ' in autores_outra_participacao:
                        del autores_outra_participacao['NRO-ID-CNPQ']
    return(outra_participacao, autores_outra_participacao)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_outra_partic.
# Inclui os dicionários atualizados na lista_outra_partic.

def atualiza_chaves_outra_participacao(outra_participacao_sem_atualizar):
    outra_participacao = []
    outra_participacao = outra_participacao_sem_atualizar
    qtde_outra_partic = len(outra_participacao)//6
    i = 0
    itens_outra_partic = {}
    lista_outra_partic = []

    if qtde_outra_partic >= 1:
        while (i < qtde_outra_partic):
            itens_outra_partic = dict(
            ID_LATTES = outra_participacao[i*6]['ID_LATTES'], 
            SEQUENCIA = outra_participacao[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = outra_participacao[(i*6)+2]['ID-PROD-BIBLIO'],
            COD_NIVEL = 14, 
            COMPETENCIA = outra_participacao[(i*6)+3]['COMPETENCIA'], 
            NATUREZA_PARTIC = outra_participacao[(i*6)+4]['NATUREZA'], 
            TITULO_PARTIC = outra_participacao[(i*6)+4]['TITULO'], 
            ANO_PARTIC = outra_participacao[(i*6)+4]['ANO'], 
            PAIS_PARTIC = outra_participacao[(i*6)+4]['PAIS'], 
            IDIOMA_PARTIC = outra_participacao[(i*6)+4]['IDIOMA'], 
            MEIO_DIVULGACAO_PARTIC = outra_participacao[(i*6)+4]['MEIO-DE-DIVULGACAO'], 
            TIPO_PARTICIPACAO = outra_participacao[(i*6)+4]['TIPO-PARTICIPACAO'], 
            FORMA_PARTICIPACAO = outra_participacao[(i*6)+4]['FORMA-PARTICIPACAO'], 
            DOI_PARTIC = outra_participacao[(i*6)+4]['DOI'], 
            TIPO_PARTIC = 'Outra Participação', 
            NOME_EVENTO_PARTIC = outra_participacao[(i*6)+5]['NOME-DO-EVENTO'], 
            COD_INSTITUICAO_PARTIC = outra_participacao[(i*6)+5]['CODIGO-INSTITUICAO'], 
            NOME_INSTITUICAO_PARTIC = outra_participacao[(i*6)+5]['NOME-INSTITUICAO'], 
            LOCAL_EVENTO_PARTIC = outra_participacao[(i*6)+5]['LOCAL-DO-EVENTO'], 
            CIDADE_EVENTO_PARTIC = outra_participacao[(i*6)+5]['CIDADE-DO-EVENTO'])
            lista_outra_partic.append(itens_outra_partic)
            itens_outra_partic = {}
            i += 1
    return(lista_outra_partic)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_outra_participacao.
# Inclui os dicionários atualizados na lista_autores_outra_participacao.

def atualiza_chaves_autores_outra_participacao(autores_outra_participacao_sem_atualizar):
    autores_outra_participacao = []
    autores_outra_participacao = autores_outra_participacao_sem_atualizar
    qtde_autores_outra_participacao = len(autores_outra_participacao)//7
    i = 0
    autor_outra_participacao = {}
    lista_autores_outra_participacao = []

    if qtde_autores_outra_participacao >= 1:
        while (i < qtde_autores_outra_participacao):
            autor_outra_participacao = dict(
            ID_LATTES = autores_outra_participacao[i*7]['ID_LATTES'], 
            ID_AUTOR = autores_outra_participacao[(i*7)+1]['ID_AUTOR'],
            ID_AUTOR_PAI = autores_outra_participacao[(i*7)+2]['ID_AUTOR_PAI'],
            SEQUENCIA = autores_outra_participacao[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_outra_participacao[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 14, 
            COMPETENCIA = autores_outra_participacao[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Participação', 
            TIPO_NIVEL_2 = 'Outra Participação', 
            # No XML está 'CONGRESSOS' para todos os tipos de participações.
            NOME_AUTOR = autores_outra_participacao[(i*7)+6]['NOME-COMPLETO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'], 
            NOME_CITACAO = autores_outra_participacao[(i*7)+6]['NOME-PARA-CITACAO-DO-PARTICIPANTE-DE-EVENTOS-CONGRESSOS'], 
            ORDEM_AUTORIA = autores_outra_participacao[(i*7)+6]['ORDEM-PARTICIPANTE'])
            lista_autores_outra_participacao.append(autor_outra_participacao)
            autor_outra_participacao = {}
            i += 1
    return(lista_autores_outra_participacao)

# VI. Patente

In [ ]:
# Coleta os dados do element PATENTE do dicionário XML Tree.
# Cria as listas patente e mautores_patente para incluir os dicionários.
# As listas são utilizadas porque os dicionários não podem ter chaves iguais. 
# Cada linha da lista possui um dicionário com os dados coletados do element.

def extrai_patente(curriculo_vitae, id_autor, nome):
    id_lattes = curriculo_vitae
    id_autor_pai = id_autor
    nome_completo = nome
    patente = []
    sequencia_patente = []
    autores_patente = []
    mes = 0
    mes_competencia_patente = ''
    competencia_patente = ''
    id_prod_biblio = ''

    for filho1 in root.iter('PATENTE'):
        #print(filho1.tag, filho1.attrib)
        sequencia_patente = filho1.attrib
        patente.append(id_lattes)
        patente.append(sequencia_patente)
        for filho2 in filho1.iter('DADOS-BASICOS-DA-PATENTE'):
            id_prod_biblio = uuid.uuid4()            
            id_prod_biblio = {'ID-PROD-BIBLIO': id_prod_biblio}
            patente.append(id_prod_biblio)            
            mes = randrange(1, 12)
            if mes < 10:
                mes_competencia_patente = '0' + str(mes)
            else:
                mes_competencia_patente = str(mes)       
            # Formato Ano Mês: YYYYMM
            competencia_patente = filho2.attrib['ANO-DESENVOLVIMENTO'] + mes_competencia_patente
            competencia_patente = {'COMPETENCIA': competencia_patente}
            patente.append(competencia_patente)
            #print(filho2.tag, filho2.attrib)
            patente.append(filho2.attrib)
            if 'HOME-PAGE' in patente:
                del patente['HOME-PAGE']
            if 'FLAG-RELEVANCIA' in patente:
                del patente['FLAG-RELEVANCIA']
            if 'TITULO-INGLES' in patente:
                del patente['TITULO-INGLES']
            if 'FLAG-POTENCIAL-INOVACAO' in patente:
                del patente['FLAG-POTENCIAL-INOVACAO']
            for filho3 in filho1.iter('DETALHAMENTO-DA-PATENTE'):
                #print(filho3.tag, filho3.attrib)
                patente.append(filho3.attrib)
                if 'FINALIDADE-INGLES' in patente:
                    del patente['FINALIDADE-INGLES']
                for filho4 in filho1.iter('AUTORES'):
                    autores_patente.append(id_lattes)
                    if filho4.attrib['NOME-COMPLETO-DO-AUTOR'] != nome_completo:
                        id_autor = uuid.uuid4()
                        id_autor = {'ID_AUTOR': id_autor}
                        id_parent_node = {'ID_AUTOR_PAI': id_autor_pai}                                    
                    else:
                        #o autor é o autor pai
                        id_autor = {'ID_AUTOR': id_autor_pai}
                        id_parent_node = {'ID_AUTOR_PAI': ''}                                   
                    autores_patente.append(id_autor)
                    autores_patente.append(id_parent_node)
                    autores_patente.append(sequencia_patente)
                    autores_patente.append(id_prod_biblio)
                    autores_patente.append(competencia_patente)
                    autores_patente.append(filho4.attrib)
                    if 'NRO-ID-CNPQ' in autores_patente:
                        del autores_patente['NRO-ID-CNPQ']
    return(patente, autores_patente)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários itens_patente.
# Inclui os dicionários atualizados na lista_patentes.

def atualiza_chaves_patente(patente_sem_atualizar):
    patente = patente_sem_atualizar
    qtde_patentes = len(patente)//6
    i = 0
    itens_patente = {}
    lista_patentes = []

    if qtde_patentes >= 1:
        while (i < qtde_patentes):
            itens_patente = dict(
            ID_LATTES = patente[i*6]['ID_LATTES'], 
            SEQUENCIA = patente[(i*6)+1]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = patente[(i*6)+2]['ID-PROD-BIBLIO'],
            COD_NIVEL = 6, 
            COMPETENCIA = patente[(i*6)+3]['COMPETENCIA'], 
            TITULO_PATENTE = patente[(i*6)+4]['TITULO'], 
            ANO_PATENTE = patente[(i*6)+4]['ANO-DESENVOLVIMENTO'], 
            PAIS_PATENTE = patente[(i*6)+4]['PAIS'], 
            MEIO_DIVULGACAO_PATENTE = patente[(i*6)+4]['MEIO-DE-DIVULGACAO'], 
            FINALIDADE_PATENTE = patente[(i*6)+5]['FINALIDADE'], 
            INSTITUICAO_FINANCIADORA_PATENTE = patente[(i*6)+5]['INSTITUICAO-FINANCIADORA'], 
            CATEGORIA_PATENTE = patente[(i*6)+5]['CATEGORIA'])
            lista_patentes.append(itens_patente)
            itens_patente = {}
            i += 1
    return(lista_patentes)

In [ ]:
# Pré-requisito para a conversão no formato CSV.
# Cria novas chaves e apaga as chaves antigas nos dicionários autor_patente.
# Inclui os dicionários atualizados na lista_autores_patente.

def atualiza_chaves_autores_patente(autores_patentes_sem_atualizar):
    autores_patentes = []
    autores_patentes = autores_patentes_sem_atualizar
    qtde_autores_patente = len(autores_patentes)//7
    i = 0
    autor_patente = {}
    lista_autores_patente = []

    if qtde_autores_patente >= 1:
        while (i < qtde_autores_patente):
            autor_patente = dict(
            ID_LATTES = autores_patentes[i*7]['ID_LATTES'], 
            ID_AUTOR = autores_patentes[(i*7)+1]['ID_AUTOR'],
            ID_AUTOR_PAI = autores_patentes[(i*7)+2]['ID_AUTOR_PAI'],
            SEQUENCIA = autores_patentes[(i*7)+3]['SEQUENCIA-PRODUCAO'],
            ID_PROD_BIBLIO = autores_patentes[(i*7)+4]['ID-PROD-BIBLIO'],
            COD_NIVEL = 6, 
            COMPETENCIA = autores_patentes[(i*7)+5]['COMPETENCIA'], 
            TIPO_NIVEL_1 = 'Patentes', 
            TIPO_NIVEL_2 = 'Patentes', 
            NOME_AUTOR = autores_patentes[(i*7)+6]['NOME-COMPLETO-DO-AUTOR'], 
            NOME_CITACAO = autores_patentes[(i*7)+6]['NOME-PARA-CITACAO'], 
            ORDEM_AUTORIA = autores_patentes[(i*7)+6]['ORDEM-DE-AUTORIA'])
            lista_autores_patente.append(autor_patente)
            autor_patente = {}
            i += 1
    return(lista_autores_patente)

# VII. Leitura dos Arquivos XML

In [ ]:
# o r transforma em raw string
pasta = r'C:\Users\S033853157\Documents\Doutorado ATUAL\Processo_ETL\entrada'
extensoes = ['xml']

# lê os arquivos com extensão xml que estão na pasta
arquivos = os.listdir(pasta)    
total_arquivos = 0

dados_gerais = {}
curso_tecnico = {}
segundo_grau = {}
graduacao = {}
aperfeiçoamento = {}
especializacao = {}
mestrado = {}
mestrado_prof = {}
doutorado = {}
livre_docencia = {}
pos_doutorado = {}

trabalhos = []
autores_trabalhos = []

artigos = []
autores_artigos = []

livros = []
autores_livros = []

cap_livros = []
autores_cap_livros = []

textos = []
autores_textos = []

outras_prod = []
autores_outras_prod = []

demais_trabalhos = []
autores_demais_trabalhos = []

orientacao_mestrado = {}
orientacao_doutorado = {}
orientacao_pos_doutorado = {}
outra_orientacao = {}

participacao_congresso = {}
autores_participacao_congresso = []

participacao_feira = {}
autores_participacao_feira = []

participacao_seminario = {}
autores_participacao_seminario = []

participacao_simposio = {}
autores_participacao_simposio = []

participacao_oficina = {}
autores_participacao_oficina = []

outra_participacao = {}
autores_outra_participacao = []

patentes = []
autores_patentes = []

# Contém as linhas como um dicionário de todos os arquivos XML
lista_formacao_academica = []
lista_trabalhos = []
lista_artigos = []
lista_livros = []
lista_cap_livros = []
lista_textos = []
lista_outras_prod = []
lista_demais_trabalhos = []
lista_orientacoes = []          
lista_participacoes = []        
lista_patentes = []

# Contém as linhas como um dicionário de todos os autores de
# produções bibliográficas, participações e patentes.
lista_autores = []

# Contém as linhas como um dicionário para criar arquivos separados
lista_autores_trabalhos = []
lista_autores_artigos = []
lista_autores_livros = []
lista_autores_cap_livros = []
lista_autores_textos = []
lista_autores_outras_prod = []
lista_autores_demais_trabalhos = []

lista_autores_participacao = []
lista_autores_participacao_congresso = []
lista_autores_participacao_feira = []
lista_autores_participacao_seminario = []
lista_autores_participacao_simposio = []
lista_autores_participacao_oficina = []
lista_autores_participacao_outra = []
lista_autores_patente = []

In [ ]:
for i in arquivos:    
    if extensoes == []:     # se vazio, exibir todos
        print(i)
    else:
        extensao = i.split('.')[-1]
        if extensao in extensoes:
            tree = ET.parse(pasta+'/'+i)
            root = tree.getroot()
            curriculo_vitae = {}
            id_autor_pai = uuid.uuid4()
            
            # Coleta o Id lattes do element 'CURRICULO-VITAE' que é o root.tag.
            id_lattes = root.attrib['NUMERO-IDENTIFICADOR']  
            curriculo_vitae = {'ID_LATTES': id_lattes}
            #id_autor_pai = {'ID_AUTOR_PAI': id_autor_pai}            
            total_arquivos = total_arquivos + 1
            
            dados_gerais = extrai_dados_gerais(curriculo_vitae)
            dados_gerais = atualiza_chaves_dados_gerais(dados_gerais)
            
            nome_completo = dados_gerais['NOME_COMPLETO']
            #nome_completo = {'NOME_COMPLETO': nome_completo}
                        
            curso_tecnico = extrai_curso_tecnico()
            curso_tecnico = atualiza_chaves_curso_tecnico(curso_tecnico)

            segundo_grau = extrai_segundo_grau()
            segundo_grau = atualiza_chaves_segundo_grau(segundo_grau)

            graduacao = extrai_graduacao()
            graduacao = atualiza_chaves_graduacao(graduacao)

            aperfeiçoamento = extrai_aperfeiçoamento()
            aperfeiçoamento = atualiza_chaves_aperfeiçoamento(aperfeiçoamento)
            
            especializacao = extrai_especializacao()
            especializacao = atualiza_chaves_especializacao(especializacao)

            mestrado = extrai_mestrado()
            mestrado = atualiza_chaves_mestrado(mestrado)

            mestrado_prof = extrai_mestrado_prof()
            mestrado_prof = atualiza_chaves_mestrado_prof(mestrado_prof)

            doutorado = extrai_doutorado()
            doutorado = atualiza_chaves_doutorado(doutorado)

            livre_docencia = extrai_livre_docencia()
            livre_docencia = atualiza_chaves_livre_docencia(livre_docencia)
            
            pos_doutorado = extrai_pos_doutorado()
            pos_doutorado = atualiza_chaves_pos_doutorado(pos_doutorado)
            
            lista_formacao_academica.append(consolida_formacao_academica
            (dados_gerais, curso_tecnico, segundo_grau, graduacao,
             aperfeiçoamento, especializacao, mestrado, 
             mestrado_prof, doutorado, livre_docencia, pos_doutorado))

            # Uso de extend no lugar do append, pq o append cria listas dentre de listas.
            trabalhos, autores_trabalhos = extrai_trabalhos(curriculo_vitae, id_autor_pai, nome_completo)
            lista_trabalhos.extend(atualiza_chaves_trabalhos(trabalhos))
            lista_autores_trabalhos.extend(atualiza_chaves_autores_trabalhos(autores_trabalhos))
            lista_autores.extend(lista_autores_trabalhos)
           
            artigos, autores_artigos = extrai_artigos(curriculo_vitae, id_autor_pai, nome_completo)
            lista_artigos.extend(atualiza_chaves_artigos(artigos))
            lista_autores_artigos.extend(atualiza_chaves_autores_artigos(autores_artigos))
            lista_autores.extend(lista_autores_artigos)
            
            livros, autores_livros = extrai_livros(curriculo_vitae, id_autor_pai, nome_completo)
            lista_livros.extend(atualiza_chaves_livros(livros))
            lista_autores_livros.extend(atualiza_chaves_autores_livros(autores_livros))
            lista_autores.extend(lista_autores_livros)

            cap_livros, autores_cap_livros = extrai_cap_livros(curriculo_vitae, id_autor_pai, nome_completo)
            lista_cap_livros.extend(atualiza_chaves_cap_livros(cap_livros))
            lista_autores_cap_livros.extend(atualiza_chaves_autores_cap_livros(autores_cap_livros))
            lista_autores.extend(lista_autores_cap_livros)
            
            textos, autores_textos = extrai_textos(curriculo_vitae, id_autor_pai, nome_completo)
            lista_textos.extend(atualiza_chaves_textos(textos))
            lista_autores_textos.extend(atualiza_chaves_autores_textos(autores_textos))
            lista_autores.extend(lista_autores_textos)
            
            outras_prod, autores_outras_prod = extrai_outras_prod(curriculo_vitae, id_autor_pai, nome_completo)
            lista_outras_prod.extend(atualiza_chaves_outras_prod(outras_prod))
            lista_autores_outras_prod.extend(atualiza_chaves_autores_outras_prod(autores_outras_prod))
            lista_autores.extend(lista_autores_outras_prod)
            
            demais_trabalhos, autores_demais_trabalhos = extrai_demais_trabalhos(curriculo_vitae, id_autor_pai, nome_completo)
            lista_demais_trabalhos.extend(atualiza_chaves_demais_trabalhos(demais_trabalhos))
            lista_autores_demais_trabalhos.extend(atualiza_chaves_autores_demais_trabalhos(autores_demais_trabalhos))
            lista_autores.extend(lista_autores_demais_trabalhos)

            orientacao_mestrado = extrai_orientacao_mestrado(curriculo_vitae)
            lista_orientacoes.extend(atualiza_chaves_orientacao_mestrado(orientacao_mestrado)) 

            orientacao_doutorado = extrai_orientacao_doutorado(curriculo_vitae)
            lista_orientacoes.extend(atualiza_chaves_orientacao_doutorado(orientacao_doutorado))
            
            orientacao_pos_doutorado = extrai_orientacao_pos_doutorado(curriculo_vitae)
            lista_orientacoes.extend(atualiza_chaves_orientacao_pos_doutorado(orientacao_pos_doutorado))
            
            outra_orientacao = extrai_outra_orientacao(curriculo_vitae)
            lista_orientacoes.extend(atualiza_chaves_outra_orientacao(outra_orientacao))
            
            participacao_congresso, autores_participacao_congresso = extrai_participacao_congresso(curriculo_vitae, id_autor_pai, nome_completo)
            lista_participacoes.extend(atualiza_chaves_participacao_congresso(participacao_congresso))
            lista_autores_participacao_congresso.extend(atualiza_chaves_autores_participacao_congresso(autores_participacao_congresso))
            lista_autores_participacao.extend(lista_autores_participacao_congresso)
            lista_autores.extend(lista_autores_participacao_congresso)
            
            participacao_feira, autores_participacao_feira = extrai_participacao_feira(curriculo_vitae, id_autor_pai, nome_completo)
            lista_participacoes.extend(atualiza_chaves_participacao_feira(participacao_feira))
            lista_autores_participacao_feira.extend(atualiza_chaves_autores_participacao_feira(autores_participacao_feira))
            lista_autores_participacao.extend(lista_autores_participacao_feira)
            lista_autores.extend(lista_autores_participacao_feira)
            
            participacao_seminario, autores_participacao_seminario = extrai_participacao_seminario(curriculo_vitae, id_autor_pai, nome_completo)
            lista_participacoes.extend(atualiza_chaves_participacao_seminario(participacao_seminario))
            lista_autores_participacao_seminario.extend(atualiza_chaves_autores_participacao_seminario(autores_participacao_seminario))
            lista_autores_participacao.extend(lista_autores_participacao_seminario)
            lista_autores.extend(lista_autores_participacao_seminario)
           
            participacao_simposio, autores_participacao_simposio = extrai_participacao_simposio(curriculo_vitae, id_autor_pai, nome_completo)
            lista_participacoes.extend(atualiza_chaves_participacao_simposio(participacao_simposio))
            lista_autores_participacao_simposio.extend(atualiza_chaves_autores_participacao_simposio(autores_participacao_simposio))
            lista_autores_participacao.extend(lista_autores_participacao_simposio)
            lista_autores.extend(lista_autores_participacao_simposio)
            
            participacao_oficina, autores_participacao_oficina = extrai_participacao_oficina(curriculo_vitae, id_autor_pai, nome_completo)
            lista_participacoes.extend(atualiza_chaves_participacao_oficina(participacao_oficina))
            lista_autores_participacao_oficina.extend(atualiza_chaves_autores_participacao_oficina(autores_participacao_oficina))
            lista_autores_participacao.extend(lista_autores_participacao_oficina)
            lista_autores.extend(lista_autores_participacao_oficina)
            
            outra_participacao, autores_outra_participacao = extrai_outra_participacao(curriculo_vitae, id_autor_pai, nome_completo)
            lista_participacoes.extend(atualiza_chaves_outra_participacao(outra_participacao))
            lista_autores_participacao_outra.extend(atualiza_chaves_autores_outra_participacao(autores_outra_participacao))
            lista_autores_participacao.extend(lista_autores_participacao_outra)
            lista_autores.extend(lista_autores_participacao_outra)
            
            patente, autores_patente = extrai_patente(curriculo_vitae, id_autor_pai, nome_completo)
            lista_patentes.extend(atualiza_chaves_patente(patente))
            lista_autores_patente.extend(atualiza_chaves_autores_patente(autores_patente))
            lista_autores.extend(lista_autores_patente)
                        
            dados_gerais = {}
            curso_tecnico = {}
            segundo_grau = {}
            graduacao = {}
            aperfeiçoamento = {}
            especializacao = {}
            mestrado = {}
            mestrado_prof = {}
            doutorado = {}
            livre_docencia = {}
            pos_doutorado = {}

            trabalhos = []
            autores_trabalhos = []

            artigos = []
            autores_artigos = []

            livros = []
            autores_livros = []

            cap_livros = []
            autores_cap_livros = []

            textos = []
            autores_textos = []

            outras_prod = []
            autores_outras_prod = []

            demais_trabalhos = []
            autores_demais_trabalhos = []

            orientacao_mestrado = {}
            orientacao_doutorado = {}
            orientacao_pos_doutorado = {}
            outra_orientacao = {}

            autores_participacao = []

            participacao_congresso = {}
            autores_participacao_congresso = []

            participacao_feira = {}
            autores_participacao_feira = []

            participacao_seminario = {}
            autores_participacao_seminario = []

            participacao_simposio = {}
            autores_participacao_simposio = []

            participacao_oficina = {}
            autores_participacao_oficina = []

            outra_participacao = {}
            autores_outra_participacao = []

            patentes = []
            autores_patentes = []

# Se o ocorrer o erro:  "IOPub data rate exceeded.", digitar o seguinte comando:
# jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [ ]:
# Nome da tag raiz
root.tag

In [ ]:
# Dicionário que a tag raiz contém
root.attrib

# VIII. Geração dos Arquivos CSV

In [ ]:
# Exporta os dados da Produção Acadêmica de todos os arquivos XML para o arquivo CSV.

header_for_academica = ['ID_LATTES', 'NOME_COMPLETO', 'NOME_COMPLETO', 'NACIONALIDADE', 'PAIS_NASCIMENTO', 'UF_NASCIMENTO', 
                        'CIDADE_NASCIMENTO', 'DATA_FALECIMENTO', 'SIGLA_PAIS_NACIONALIDADE', 'PAIS_NACIONALIDADE', 'ORCID_ID', 
                        'SEQUENCIA_SEG_GRAU', 'COMP_INI_SEG_GRAU', 'COMP_FIM_SEG_GRAU', 'NIVEL_SEG_GRAU', 
                        'COD_INSTITUICAO_SEG_GRAU', 'NOME_INSTITUICAO_SEG_GRAU', 'STATUS_SEG_GRAU', 'ANO_INICIO_SEG_GRAU', 
                        'ANO_CONCLUSAO_SEG_GRAU',  'SEQUENCIA_TEC_PROF', 'COMP_INI_TEC_PROF', 'COMP_FIM_TEC_PROF', 
                        'NIVEL_TEC_PROF', 'COD_INSTITUICAO_TEC_PROF', 'NOME_INSTITUICAO_TEC_PROF', 'NOME_CURSO_TEC_PROF', 
                        'COD_CURSO_TEC_PROF', 'STATUS_CURSO_TEC_PROF', 'ANO_INICIO_TEC_PROF', 'ANO_CONCLUSAO_TEC_PROF', 
                        'SEQUENCIA_FORMACAO_GRAD', 'COMP_INI_GRAD', 'COMP_FIM_GRAD', 'NIVEL_GRAD', 'TITULO_TRABALHO_GRAD', 
                        'NOME_ORIENTADOR_GRAD', 'COD_INSTITUICAO_GRAD', 'NOME_INSTITUICAO_GRAD', 'COD_CURSO_GRAD', 
                        'NOME_CURSO_GRAD', 'STATUS_CURSO_GRAD', 'ANO_INICIO_GRAD', 'ANO_CONCLUSAO_GRAD', 
                        'SEQUENCIA_FORMACAO_APERF', 'COMP_INI_APERF', 'COMP_FIM_APERF', 'NIVEL_APERF', 
                        'TITULO_TRABALHO_APERF', 'NOME_ORIENTADOR_APERF', 'COD_INSTITUICAO_APERF', 'NOME_INSTITUICAO_APERF', 
                        'COD_CURSO_APERF', 'NOME_CURSO_APERF', 'STATUS_CURSO_APERF', 'ANO_INICIO_APERF', 
                        'ANO_CONCLUSAO_APERF', 'SEQUENCIA_FORMACAO_ESPEC', 'COMP_INI_ESPEC', 'COMP_FIM_ESPEC', 'NIVEL_ESPEC', 
                        'TITULO_TRABALHO_ESPEC', 'NOME_ORIENTADOR_ESPEC', 'COD_INSTITUICAO_ESPEC', 'NOME_INSTITUICAO_ESPEC', 
                        'COD_CURSO_ESPEC', 'NOME_CURSO_ESPEC', 'STATUS_CURSO_ESPEC', 'ANO_INICIO_ESPEC', 'ANO_CONCLUSAO_ESPEC', 
                        'SEQUENCIA_FORMACAO_MEST', 'COMP_INI_MEST', 'COMP_FIM_MEST', 'NIVEL_MEST', 'COD_INSTITUICAO_MEST', 
                        'NOME_INSTITUICAO_MEST', 'COD_CURSO_MEST', 'NOME_CURSO_MEST', 'STATUS_CURSO_MEST', 'ANO_INICIO_MEST', 
                        'ANO_CONCLUSAO_MEST', 'ANO_OBTENCAO_TITULO_MEST', 'TITULO_DISSERTACAO_MEST', 'NOME_ORIENTADOR_MEST', 
                        'TIPO_MEST', 'ID_ORIENTADOR-MEST', 'SEQUENCIA_FORMACAO_MEST_PROF', 'COMP_INI_MEST_PROF', 
                        'COMP_FIM_MEST_PROF', 'NIVEL_MEST_PROF', 'COD_INST_MEST_PROF', 'NOME_INST_MEST_PROF', 
                        'COD_CURSO_MEST_PROF',  'NOME_CURSO_MEST_PROF', 'STATUS_CURSO_MEST_PROF', 'ANO_INICIO_MEST_PROF', 
                        'ANO_CONCLUSAO_MEST_PROF', 'ANO_OBTENCAO_TITULO_MEST_PROF', 'TITULO_DISSERTACAO_MEST_PROF', 
                        'NOME_ORIENTADOR_MEST_PROF', 'TIPO_MEST_PROF', 'ID_ORIENTADOR_MEST_PROF', 'SEQUENCIA_FORMACAO_DOUT', 
                        'COMP_INI_DOUT', 'COMP_FIM_DOUT', 'NIVEL_DOUT', 'COD_INST_DOUT', 'NOME_INST_DOUT', 'COD_CURSO_DOUT', 
                        'NOME_CURSO_DOUT', 'STATUS_CURSO_DOUT', 'ANO_INICIO_DOUT', 'ANO_CONCLUSAO_DOUT', 
                        'ANO_OBTENCAO_TITULO_DOUT', 'TITULO_TESE_DOUT', 'NOME_COMPLETO_ORIENTADOR_DOUT', 'TIPO_DOUT', 
                        'COD_INST_DOUT_SAND', 'NOME_INST_DOUT_SAND', 'NOME_ORIENTADOR_DOUT_SAND', 'NR_ID_ORIENTADOR_DOUT', 
                        'SEQUENCIA_FORMACAO_LIV_DOC', 'COMP_INI_LIV_DOC', 'COMP_FIM_LIV_DOC', 'NIVEL_LIV_DOC', 
                        'COD_INST_LIV_DOC', 'NOME_INST_LIV_DOC', 'ANO_OBTENCAO_TITULO_LIV_DOC', 'TITULO_DO_TRABALHO_LIV_DOC', 
                        'SEQUENCIA_FORMACAO_POS_DOUT', 'COMP_INI_POS_DOUT', 'COMP_FIM_POS_DOUT', 'NIVEL_POS_DOUT', 
                        'COD_INST_POS_DOUT', 'NOME_INST_POS_DOUT', 'ANO_INICIO_POS_DOUT', 'ANO_CONCLUSAO_POS_DOUT', 
                        'ANO_OBTENCAO_TITULO_POS_DOUT', 'STATUS_ESTAGIO_POS_DOUT', 'STATUS_CURSO_POS_DOUT', 
                        'NR_ID_ORIENTADOR_POS_DOUT', 'TITULO_TRABALHO_POS_DOUT']
with open('Formacao_Academica.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_for_academica) 
    writer.writeheader() 
    writer.writerows(lista_formacao_academica) 

In [ ]:
# Exporta os dados da lista de Trabalhos de todos os arquivos XML para o arquivo CSV.

header_trabalhos = ['ID_PROD_BIBLIO', 'ID_LATTES', 'SEQUENCIA', 
                    'COD_NIVEL', 'COMPETENCIA', 'NATUREZA_TRABALHO', 
                    'TITULO_TRABALHO', 'ANO_TRABALHO','PAIS_EVENTO_TRAB', 
                    'IDIOMA_TRABALHO', 'MEIO_DIVULGACAO_TRAB', 
                    'DOI_TRABALHO', 'CLASSIFICACAO_EVENTO_TRAB', 
                    'NOME_EVENTO_TRAB', 'CIDADE_EVENTO_TRAB', 
                    'ANO_REALIZACAO_TRAB', 'TITULO_ANAIS_PROC_TRAB',
                    'VOLUME_TRAB', 'FASCICULO_TRAB', 'SERIE_TRAB', 
                    'PAG_INICIAL_TRAB', 'PAG_FINAL_TRAB', 'ISBN_TRAB', 
                    'NOME_EDITORA_TRAB', 'CIDADE_EDITORA_TRAB']
                  
with open('Trabalhos.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_trabalhos) 
    writer.writeheader() 
    writer.writerows(lista_trabalhos)
    
#texto = '\n Lista de Trabalhos: '
#print(texto + str(lista_trabalhos))
 

In [ ]:
# Exporta os dados da lista de Artigos de todos os arquivos XML para o arquivo CSV.

header_artigos = ['ID_PROD_BIBLIO', 'ID_LATTES', 'SEQUENCIA', 'COD_NIVEL',
                  'COMPETENCIA', 'NATUREZA_ARTIGO', 'TITULO_ARTIGO', 
                  'ANO_ARTIGO', 'PAIS_PUBLICACAO_ARTIGO','IDIOMA_ARTIGO', 
                  'MEIO_DIVULGACAO_ARTIGO', 'DOI_ARTIGO', 
                  'TITULO_PERIODICO_REVISTA', 'ISSN_ARTIGO', 
                  'VOLUME_ARTIGO', 'FASCICULO_ARTIGO', 'SERIE_ARTIGO', 
                  'PAG_INICIAL_ARTIGO', 'PAG_FINAL_ARTIGO', 
                  'LOCAL_PUBLICACAO_ARTIGO']
                  
with open('Artigos.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_artigos) 
    writer.writeheader() 
    writer.writerows(lista_artigos) 

In [ ]:
# Exporta os dados da lista de Livros de todos os arquivos XML para o arquivo CSV.

header_livros = ['ID_PROD_BIBLIO', 'ID_LATTES', 'SEQUENCIA', 'COD_NIVEL', 'COMPETENCIA', 
                 'TIPO_LIVRO', 'NATUREZA_LIVRO', 'TITULO_LIVRO', 
                 'ANO_LIVRO', 'PAIS_PUBLICACAO_LIVRO', 'IDIOMA_LIVRO', 
                 'MEIO_DIVULGACAO_LIVRO', 'DOI_LIVRO', 'NR_VOLUMES_LIVRO',
                 'NR_PAGINAS_LIVRO', 'ISBN_LIVRO', 'NR_EDICAO_REVISAO_LIVRO',
                 'NR_SERIE_LIVRO', 'CIDADE_EDITORA_LIVRO',
                 'NOME_EDITORA_LIVRO']
                  
with open('Livros.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_livros) 
    writer.writeheader() 
    writer.writerows(lista_livros) 

In [ ]:
# Exporta os dados da lista de Capítulos de Livros de todos os arquivos XML para o arquivo CSV.

header_cap_livros = ['ID_PROD_BIBLIO', 'ID_LATTES', 'SEQUENCIA', 
                     'COD_NIVEL', 'COMPETENCIA', 'TIPO_CAP_LIVRO', 
                     'TITULO_CAP_LIVRO', 'ANO_LIVRO', 
                     'PAIS_PUBLICACAO_CAP_LIVRO','IDIOMA_CAP_LIVRO', 
                     'MEIO_DIVULGACAO_CAP_LIVRO', 'DOI_CAP_LIVRO', 
                     'TITULO_LIVRO', 'NR_VOLUMES_CAP_LIVRO', 
                     'PAG_INICIAL_CAP_LIVRO', 'PAG_FINAL_CAP_LIVRO', 
                     'ISBN_CAP_LIVRO', 'ORGANIZADORES_CAP_LIVRO', 
                     'NR_EDICAO_REVISAO_CAP_LIVRO', 'NR_SERIE_CAP_LIVRO', 
                     'CIDADE_EDITORA_CAP_LIVRO', 'NOME_EDITORA_CAP_LIVRO']
                  
with open('Cap_livros.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_cap_livros) 
    writer.writeheader() 
    writer.writerows(lista_cap_livros) 

In [ ]:
# Exporta os dados da lista de Textos de todos os arquivos XML para o arquivo CSV.

header_textos = ['ID_PROD_BIBLIO', 'ID_LATTES', 'SEQUENCIA', 'COD_NIVEL', 
                 'COMPETENCIA', 'NATUREZA_TEXTO', 'TITULO_TEXTO', 
                 'ANO_TEXTO', 'PAIS_PUBLICACAO_TEXTO','IDIOMA_TEXTO', 
                 'MEIO_DIVULGACAO_TEXTO', 'DOI_TEXTO', 
                 'TITULO_JORNAL_REVISTA', 'ISSN_TEXTO', 
                 'DATA_PUBLICACAO_TEXTO', 'VOLUME_TEXTO', 
                 'PAG_INICIAL_TEXTO', 'PAG_FINAL_TEXTO', 
                 'LOCAL_PUBLICACAO_TEXTO']
                  
with open('Textos.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_textos) 
    writer.writeheader() 
    writer.writerows(lista_textos) 

In [ ]:
# Exporta os dados da lista de Outras Publicações de todos os arquivos XML para o arquivo CSV.

header_outra_prod = ['ID_PROD_BIBLIO', 'ID_LATTES', 'SEQUENCIA', 'COD_NIVEL', 
                     'COMPETENCIA', 'NATUREZA_OUTRA_PROD', 'TITULO_OUTRA_PROD', 
                     'ANO_OUTRA_PROD', 'PAIS_PUBLICACAO_OUTRA_PROD',
                     'IDIOMA_OUTRA_PROD', 'MEIO_DIVULGACAO_OUTRA_PROD',
                     'DOI_OUTRA_PROD', 'EDITORA_OUTRA_PROD', 
                     'CIDADE_EDITORA_OUTRA_PROD', 
                     'NR_PAG_OUTRA_PROD', 'ISSN_ISBN_OUTRA_PROD']
                  
with open('Outras_prod.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_outra_prod) 
    writer.writeheader() 
    writer.writerows(lista_outras_prod) 

In [ ]:
# Exporta os dados da lista de Demais Trabalhos de todos os arquivos XML para o arquivo CSV.

header_demais_trab = ['ID_PROD_BIBLIO', 'ID_LATTES', 'SEQUENCIA', 'COD_NIVEL', 
                      'COMPETENCIA', 'NATUREZA_DEMAIS_TRAB', 'TITULO_DEMAIS_TRAB', 
                      'ANO_DEMAIS_TRAB', 'PAIS_DEMAIS_TRAB','IDIOMA_DEMAIS_TRAB', 
                      'DOI_DEMAIS_TRAB','FINALIDADE_DEMAIS_TRAB', 
                      'LOCAL_DEMAIS_TRAB']
                  
with open('Demais_trabalhos.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_demais_trab) 
    writer.writeheader() 
    writer.writerows(lista_demais_trabalhos) 

In [ ]:
# Exporta os dados da lista de Orientações de todos os arquivos XML para o arquivo CSV.

header_orientacoes = ['ID_ORIENTACAO', 'ID_LATTES', 'SEQUENCIA', 'COMPETENCIA', 
                      'NATUREZA_ORIENTACAO', 'TITULO_ORIENTACAO', 
                      'ANO_ORIENTACAO', 'PAIS_ORIENTACAO','IDIOMA_ORIENTACAO', 
                      'DOI_ORIENTACAO', 'ORIENTACAO', 'TIPO_ORIENTACAO', 
                      'NOME_ORIENTADO', 'COD_INSTITUICAO_ORIENTACAO', 
                      'NOME_INSTITUICAO_ORIENTACAO', 'COD_CURSO_ORIENTACAO', 
                      'NOME_CURSO_ORIENTACAO', 'NR_PAG_ORIENTACAO', 'ID_ORIENTADO']
                  
with open('Orientacoes.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_orientacoes) 
    writer.writeheader() 
    writer.writerows(lista_orientacoes) 

In [ ]:
# Exporta os dados da lista de Participações de todos os arquivos XML para o arquivo CSV.

header_participacoes = ['ID_PROD_BIBLIO', 'ID_LATTES', 'SEQUENCIA', 'COD_NIVEL', 
                        'COMPETENCIA', 'NATUREZA_PARTIC', 'TITULO_PARTIC', 
                        'ANO_PARTIC', 'PAIS_PARTIC','IDIOMA_PARTIC', 
                        'MEIO_DIVULGACAO_PARTIC', 'TIPO_PARTICIPACAO', 
                        'FORMA_PARTICIPACAO', 'DOI_PARTIC', 'TIPO_PARTIC',
                        'NOME_EVENTO_PARTIC', 'COD_INSTITUICAO_PARTIC', 
                        'NOME_INSTITUICAO_PARTIC', 'LOCAL_EVENTO_PARTIC', 
                        'CIDADE_EVENTO_PARTIC']
                  
with open('Participacoes.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_participacoes) 
    writer.writeheader() 
    writer.writerows(lista_participacoes) 

In [ ]:
# Exporta os dados da lista de Patentes de todos os arquivos XML para o arquivo CSV.

header_patentes = ['ID_PROD_BIBLIO', 'ID_LATTES', 'SEQUENCIA', 'COD_NIVEL', 
                   'COMPETENCIA', 'TITULO_PATENTE', 'ANO_PATENTE', 
                   'PAIS_PATENTE', 'MEIO_DIVULGACAO_PATENTE',
                   'FINALIDADE_PATENTE', 'INSTITUICAO_FINANCIADORA_PATENTE', 
                   'CATEGORIA_PATENTE']
                  
with open('Patentes.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_patentes) 
    writer.writeheader() 
    writer.writerows(lista_patentes) 

In [ ]:
# Exporta os dados da lista de todos os Autores de todos os arquivos XML para um arquivo CSV.

header_autores = ['ID_LATTES', 'ID_AUTOR', 'ID_AUTOR_PAI',
                  'SEQUENCIA', 'ID_PROD_BIBLIO', 'COD_NIVEL', 
                  'COMPETENCIA', 'TIPO_NIVEL_1', 'TIPO_NIVEL_2', 
                  'NOME_AUTOR', 'NOME_CITACAO', 'ORDEM_AUTORIA']
                  
with open('Autores.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores) 

In [ ]:
# Exporta os dados da lista dos Autores de cada um dos arquivos XML para os seus respectivos arquivos CSV.
                  
with open('Autores_Trabalhos.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_trabalhos)
    
with open('Autores_Artigos.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_artigos)
    
with open('Autores_Livros.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_livros)
    
with open('Autores_Cap_Livros.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_cap_livros)
    
with open('Autores_Textos.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_textos)
        
with open('Autores_Outras_Prod.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_outras_prod)
    
with open('Autores_Demais_Trabalhos.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_demais_trabalhos)

with open('Autores_Participacao.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_participacao)

with open('Autores_Partic_Congresso.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_participacao_congresso)
    
with open('Autores_Partic_Feira.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_participacao_feira)
    
with open('Autores_Partic_Seminario.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_participacao_seminario)

with open('Autores_Partic_Simposio.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_participacao_simposio)
    
with open('Autores_Partic_Oficina.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_participacao_oficina)

with open('Autores_Partic_Outra.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_participacao_outra)

with open('Autores_Patente.csv', 'w', newline='') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = header_autores) 
    writer.writeheader() 
    writer.writerows(lista_autores_patente)


# Fim